# Jupyter Notebook for the evaluation of nucleon PDFs from Distillation correlation functions

## Initialization

In [ ]:
import numpy as np
import scipy.linalg as la
import scipy.optimize as scipyOpt
import math, itertools as it
import matplotlib as mpl ; import matplotlib.pyplot as plt ; plt.ion()
from matplotlib import rc
import matplotlib.gridspec as gridspec
import pickle
import h5py
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetiva']})
rc('text', usetex=True)

import warnings
warnings.filterwarnings("ignore")

print ("Done")

## Directories, Parameters, Constants

### Generic definitions

In [ ]:
# Math Pi
Pi = np.pi

valSgnPN = lambda i: ("+" if i > 0 else "") + str(i)
valSgnN  = lambda i: ("" if i > 0 else "") + str(i)


def dispTag(d):
    dL = lambda i: ("z" if i != 0 else "") + ("+" if i > 0 else "") + str(i)
    return 'disp_%s'%(dL(d))

def MomTag(zMom):
#    mSgn = lambda i: ("+" if i > 0 else "") + str(i)
    return 'pz.%s'%(valSgnPN(zMom))

def phMomTpl(phTag,zMom):
    return (phTag,MomTag(zMom))

def momFileTag(mVec):
    return 'momXYZ.%d.%d.%d' % (mVec[0],mVec[1],mVec[2])

def phasedTag(phTag,phM='unphased'):
    if phM == 'unphased':
        return phTag
    else:
        return '%s-%s'%(phTag,phM)

def phaseDir(phTag,phM='unphased'):
    if phM == 'unphased':
        return phTag
    else:
        return '%s/%s'%(phTag,phM)

def getPhasedValueStr(phM):
    return phM.split('_')[1]

def getPhasedValue(phM):
    return np.float(getPhasedValueStr(phM))

def createPhMomTag(PzStr):
    if type(PzStr) != str:
        raise ValueError('createPhMomTag: Argument must be of string type')
    return 'd001_%s'%(PzStr)
#-----------------------------------

print('\nDone\n')

In [ ]:
ensList = ['cl21_32_64_b6p3_m0p2350_m0p2050',
           'cl21_48_96_b6p3_m0p2416_m0p2050-main',
           'cl21_48_96_b6p3_m0p2416_m0p2050-1000',
           'cl21_48_96_b6p3_m0p2416_m0p2050-1200',           
           'cl21_48_96_b6p3_m0p2416_m0p2050-all']

ensTag = {'cl21_32_64_b6p3_m0p2350_m0p2050': '32c64',
          'cl21_48_96_b6p3_m0p2416_m0p2050-main': '48c96, main',
          'cl21_48_96_b6p3_m0p2416_m0p2050-1000': '48c96, str. 1000',
          'cl21_48_96_b6p3_m0p2416_m0p2050-1200': '48c96, str. 1200',
          'cl21_48_96_b6p3_m0p2416_m0p2050-all':  '48c96, all'}

havePhased = {'cl21_32_64_b6p3_m0p2350_m0p2050' : True,
              'cl21_48_96_b6p3_m0p2416_m0p2050-main' : False,
              'cl21_48_96_b6p3_m0p2416_m0p2050-1000' : False,
              'cl21_48_96_b6p3_m0p2416_m0p2050-1200' : False,              
              'cl21_48_96_b6p3_m0p2416_m0p2050-all' : False}

ensStreams = {'cl21_48_96_b6p3_m0p2416_m0p2050': ['main','1000','1200','all']}


def constructFullEns(main, stream):
    return main + '-' + stream


# (Maximum) Number of configs
Ncfg_all = {'cl21_32_64_b6p3_m0p2350_m0p2050': 349,
            'cl21_48_96_b6p3_m0p2416_m0p2050-main': 127,
            'cl21_48_96_b6p3_m0p2416_m0p2050-1000': 76,
            'cl21_48_96_b6p3_m0p2416_m0p2050-1200': 84,            
            'cl21_48_96_b6p3_m0p2416_m0p2050-all' : 287}
#----------------------------------------------

# Number of time-slices for 2pt function
Nt_2pt_all = {'cl21_32_64_b6p3_m0p2350_m0p2050': 21,
              'cl21_48_96_b6p3_m0p2416_m0p2050-main': 21,
              'cl21_48_96_b6p3_m0p2416_m0p2050-1000': 21,
              'cl21_48_96_b6p3_m0p2416_m0p2050-1200': 21,              
              'cl21_48_96_b6p3_m0p2416_m0p2050-all' : 21}


Nvec = {'cl21_32_64_b6p3_m0p2350_m0p2050': 64,
        'cl21_48_96_b6p3_m0p2416_m0p2050-main': 128,
        'cl21_48_96_b6p3_m0p2416_m0p2050-1000': 128,
        'cl21_48_96_b6p3_m0p2416_m0p2050-1200': 128,        
        'cl21_48_96_b6p3_m0p2416_m0p2050-all' : 128}



phaseTypes = {'cl21_32_64_b6p3_m0p2350_m0p2050': ['unphased', 'phased'],
              'cl21_48_96_b6p3_m0p2416_m0p2050-main': ['unphased'],
              'cl21_48_96_b6p3_m0p2416_m0p2050-1000': ['unphased'],
              'cl21_48_96_b6p3_m0p2416_m0p2050-1200': ['unphased'],              
              'cl21_48_96_b6p3_m0p2416_m0p2050-all' : ['unphased']}

SrcSnkOpTypes = {'cl21_32_64_b6p3_m0p2350_m0p2050': ['single'],
                 'cl21_48_96_b6p3_m0p2416_m0p2050-main': ['single'],
                 'cl21_48_96_b6p3_m0p2416_m0p2050-1000': ['single'],
                 'cl21_48_96_b6p3_m0p2416_m0p2050-1200': ['single'],                 
                 'cl21_48_96_b6p3_m0p2416_m0p2050-all' : ['single']}

maxDisp = {'cl21_32_64_b6p3_m0p2350_m0p2050': 8,
           'cl21_48_96_b6p3_m0p2416_m0p2050-main': 8,
           'cl21_48_96_b6p3_m0p2416_m0p2050-1000': 8,
           'cl21_48_96_b6p3_m0p2416_m0p2050-1200': 8,           
           'cl21_48_96_b6p3_m0p2416_m0p2050-all' : 8}

dispLen = {}
dispLen_ave = {}

ZmomVal = {}
TsinkList = {}
T0List = {}
ZmomVal_ave = {}
Phases_MomTags = {}
Phases_MomTags_ave = {}
for ens in ensList:
    dispLen[ens] = np.arange(-maxDisp[ens],maxDisp[ens]+1)
    dispLen_ave[ens] = np.arange(maxDisp[ens]+1)
    
    ZmomVal[ens]     = {}
    TsinkList[ens]   = {}
    T0List[ens]      = {}
    ZmomVal_ave[ens] = {}
    ZmomPh = {}
    
    Phases_MomTags[(ens,'unphased')] = ['unphased']
    Phases_MomTags_ave[(ens,'unphased')] = ['unphased']    
#------------       
    
    
# Parameters for unphased correlators with a single src-sink operator
# Note: Every combination of (momentum,t0,t_sink) is considered a 'dataset'

phTS = 'unphased'
opTS = 'single'

ens = 'cl21_32_64_b6p3_m0p2350_m0p2050'

phMTag = phasedTag(phTS)
phTpl = (phMTag,opTS)

ZmomVal[ens][phTpl]     = [0,1,2,3,-1,-2,-3] # Available momenta values
ZmomVal_ave[ens][phTpl] = [0,1,2,3]
TsinkList[ens][phTpl]   = [4,6,8,10,12,14]   # Tsink Values
T0List[ens][phTpl]      = [0,16,32,48]       # T0 Values
#------------------

ens_main = 'cl21_48_96_b6p3_m0p2416_m0p2050'
for ens_str in ensStreams[ens_main]:
    ens = constructFullEns(ens_main,ens_str)
    
    ZmomVal[ens][phTpl]     = [0,1,2,-1,-2] # Available momenta values
    ZmomVal_ave[ens][phTpl] = [0,1,2]
    TsinkList[ens][phTpl]   = [6,8]         # Tsink Values
    T0List[ens][phTpl]      = [0,24,48,72]  # T0 Values
#-----------------------------------



# Parameters for phased correlators with a single src-sink operator
opTS = 'single'
phTS = 'phased'

ens = 'cl21_32_64_b6p3_m0p2350_m0p2050'

Phases_MomTags[(ens,'phased')] = ['d001_2.00','d001_-2.00']
Phases_MomTags_ave[(ens,'phased')] = ['d001_2.00']

ZmomPh[ens] = {'d001_2.00': [4,5,6], 'd001_-2.00': [4,5,6]}

# No phased results for 'cl21_48_96_b6p3_m0p2416_m0p2050' for now

for ens in ensList:
    if not havePhased[ens]:
        continue
        
    for phM in Phases_MomTags[(ens,phTS)]:
        phMTag = phasedTag(phTS,phM)
        phTpl = (phMTag,opTS)
        TsinkList[ens][phTpl] = [4,6,8,10,12,14]    # Tsink Values
        T0List[ens][phTpl]    = [0,16,32,48]       # T0 Values

        phSgn = getPhasedValue(phM)/np.abs(getPhasedValue(phM))
        ZmomVal[ens][phTpl] = []
        ZmomVal_ave[ens][phTpl] = []
        for Pz in ZmomPh[ens][phM]:
            ZmomVal[ens][phTpl].append(int(valSgnN(int(phSgn*Pz))))
            ZmomVal_ave[ens][phTpl].append(Pz)        
#--------------------------------------------------------------



corr3ptMainDir = {}
corr2ptMainDir = {}

momTags_Dict = {}
momTags_Dict_ave = {}
momVector_Dict  = {}

Ncfg_Dict_3pt = {}
Ncfg_Dict_2pt = {}
existsDataSet_3pt = {}
existsDataSet_2pt = {}
Nt_Dict_2pt = {}

allMomenta = {}       
    
for ens in ensList:    
    corr3ptMainDir[ens] = {}
    corr2ptMainDir[ens] = {}    
   
    momTags_Dict[ens] = {}
    momTags_Dict_ave[ens] = {}
    momVector_Dict[ens]  = {}

    Ncfg_Dict_3pt[ens] = {}
    Ncfg_Dict_2pt[ens] = {}
    existsDataSet_3pt[ens] = {}
    existsDataSet_2pt[ens] = {}
    Nt_Dict_2pt[ens] = {}

    for phTS in phaseTypes[ens]:
        for phM in Phases_MomTags[(ens,phTS)]:
            phMTag = phasedTag(phTS,phM)
            phTpl = (phMTag,opTS)

            momTags_Dict[ens][phTpl] = []

            for zmom in ZmomVal[ens][phTpl]:
                mTag = MomTag(zmom)
                momTags_Dict[ens][phTpl].append(mTag)
                momVector_Dict[ens][mTag] = np.array([0,0,zmom])

                for tsnk in TsinkList[ens][phTpl]:
                    dTpl = (phMTag,opTS,mTag,tsnk)
                    Ncfg_Dict_3pt[ens][dTpl] = Ncfg_all[ens]  # All datasets have the same number of configs
                    for t0 in T0List[ens][phTpl]:
                        dTpl3pt = (phMTag,opTS,mTag,tsnk,t0)
                        existsDataSet_3pt[ens][dTpl3pt] = True  # Whether all possible datasets exist for the three-point function

                dTpl2 = (phMTag,opTS,mTag)
                Ncfg_Dict_2pt[ens][dTpl2] = Ncfg_all[ens]            
                for t0 in T0List[ens][phTpl]:
                    dTpl2pt = (phMTag,opTS,mTag,t0)
                    existsDataSet_2pt[ens][dTpl2pt] = True  # Whether all possible datasets exist for the two-point function

                Nt_Dict_2pt[ens][phMomTpl(phMTag,zmom)] = Nt_2pt_all[ens] # 2pt for all momenta have 21 time-slices        
            #----------------------------------------------

            momTags_Dict_ave[ens][phTpl] = []
            for zmom in ZmomVal_ave[ens][phTpl]:
                mTag = MomTag(zmom)
                momTags_Dict_ave[ens][phTpl].append(mTag)

            # Base Directory of two- and three-point correlation functions
            corr3ptMainDir[ens][phTpl] = 'data/%s/3pt_corr/%s'%(ens,phaseDir(phTS,phM))
            corr2ptMainDir[ens][phTpl] = 'data/%s/2pt_corr/%s'%(ens,phaseDir(phTS,phM))
    #---------------------------

    allMomenta[ens] = []
    for phTS in phaseTypes[ens]:
        for phM in Phases_MomTags_ave[(ens,phTS)]:
            phMTag = phasedTag(phTS,phM)
            phTpl = (phMTag,opTS)        
            for zmom in ZmomVal_ave[ens][phTpl]:
                mTag = MomTag(zmom)
                allMomenta[ens].append(mTag)
#----------------------------------------------


# Zero-momentum for ensemble 'cl21_32_64_b6p3_m0p2350_m0p2050' t0=48
# for unphased correlators does not exist, skip it
phMTag = phasedTag('unphased')
ens = 'cl21_32_64_b6p3_m0p2350_m0p2050'
zmom = 0
t0 = 48
mTag = MomTag(zmom)
dTpl2pt = (phMTag,opTS,mTag,t0)
existsDataSet_2pt[ens][dTpl2pt] = False
#----------------------------------------------


# Insertion gamma matrices
gMatList = {'cl21_32_64_b6p3_m0p2350_m0p2050': ['gt',
                                                'gxg5','gyg5','gzg5','gtg5',
                                                'gxgy','gxgz','gxgt','gygz','gygt','gzgt'],
            'cl21_48_96_b6p3_m0p2416_m0p2050-main': ['gt'],
            'cl21_48_96_b6p3_m0p2416_m0p2050-1000': ['gt'],
            'cl21_48_96_b6p3_m0p2416_m0p2050-1200': ['gt'],            
            'cl21_48_96_b6p3_m0p2416_m0p2050-all': ['gt']
            }
#----------------------------------------------


# Which datasets to analyze (if there's at least one t0 for each momentum and tsink that's OK)
analyzeDataSetM = {}
analyzeDataSetE = {}

for ens in ensList:
    analyzeDataSetM[ens] = {}
    analyzeDataSetE[ens] = {}

    for phTS in phaseTypes[ens]:
        for phM in Phases_MomTags[(ens,phTS)]:
            phMTag = phasedTag(phTS,phM)
            phTpl = (phMTag,opTS)

            for mTag in momTags_Dict[ens][phTpl]:
                for tsnk in TsinkList[ens][phTpl]:
                    adTplM = (phMTag,opTS,mTag,tsnk)
                    for t0 in T0List[ens][phTpl]:
                        dTpl3pt = (phMTag,opTS,mTag,tsnk,t0)
                        dTpl2pt = (phMTag,opTS,mTag,t0)
                        if existsDataSet_3pt[ens][dTpl3pt] and existsDataSet_2pt[ens][dTpl2pt]:
                            analyzeDataSetM[ens][adTplM] = True
                        else:
                            analyzeDataSetM[ens][adTplM] = False
                        break

            for mTag in momTags_Dict[ens][phTpl]:
                adTplE = (phMTag,opTS,mTag)
                for t0 in T0List[ens][phTpl]:
                    dTpl2pt = (phMTag,opTS,mTag,t0)
                    if existsDataSet_2pt[ens][dTpl2pt]:
                        analyzeDataSetE[ens][adTplE] = True
                    else:
                        analyzeDataSetE[ens][adTplE] = False
                    break        
#----------------------------------------------


ReIm = ['Re','Im']

print('Momentum Tags:\n', momTags_Dict)
print('\n')
print('Momentum Tags Averaged:\n', momTags_Dict_ave)
print('\n')
print('Momentum Vectors:\n', momVector_Dict)
print('\n')
print('All momenta list:\n', allMomenta)

print('\nDone\n')


### Read/define source-insertion-sink operators

In [ ]:

# Interpolating operators  for 3-point functions

nRow3pt_Dict = {}
nRow2pt_Dict = {}
srcOpList3pt_Dict = {}
snkOpList3pt_Dict = {}
srcOpList2pt_Dict = {}
snkOpList2pt_Dict = {}

insOptrList = {}

for ens in ensList:
    nRow3pt_Dict[ens] = {'single': {}}
    nRow2pt_Dict[ens] = {'single': {}}
    srcOpList3pt_Dict[ens] = {'single': {}}
    snkOpList3pt_Dict[ens] = {'single': {}}
    srcOpList2pt_Dict[ens] = {'single': {}}
    snkOpList2pt_Dict[ens] = {'single': {}}

    insOptrList[ens] = {}

    for phTS in phaseTypes[ens]:
        for phM in Phases_MomTags[(ens,phTS)]:
            phMTag = phasedTag(phTS,phM)
            phTpl = (phMTag,opTS)

            for zmom in ZmomVal[ens][phTpl]:
                pmTpl = phMomTpl(phMTag,zmom)


                if ens == 'cl21_32_64_b6p3_m0p2350_m0p2050':
                    # 2-point function interpolators have one row for zmom = 0
                    if zmom == 0:
                        nRow2pt_Dict[ens]['single'][pmTpl] = 1
                    else:
                        nRow2pt_Dict[ens]['single'][pmTpl] = 2
                elif 'cl21_48_96_b6p3_m0p2416_m0p2050' in ens:
                    nRow2pt_Dict[ens]['single'][pmTpl] = 1

                # 3-point function interpolators have two-rows for all momenta
                nRow3pt_Dict[ens]['single'][pmTpl] = 2

                if zmom == 0:
                    srcOpList3pt_Dict[ens]['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
                    snkOpList3pt_Dict[ens]['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
                    srcOpList2pt_Dict[ens]['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
                    snkOpList2pt_Dict[ens]['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
                else:
                    srcOpList3pt_Dict[ens]['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
                    snkOpList3pt_Dict[ens]['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
                    srcOpList2pt_Dict[ens]['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
                    snkOpList2pt_Dict[ens]['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
            #----------------------------------------------


            # Insertion operators list, that's a common file among all momenta
            insOpFname = '%s/insert_gamma_row.DA.list' % (corr3ptMainDir[ens][phTpl])

            # Read insertion operators
            insOptrList[ens][phMTag] = {'name': [], 'row': []}
            with open(insOpFname) as fp:
                line = fp.readlines()
                for t in line:
                    insOptrList[ens][phMTag]['name'].append(t.split()[0])
                    insOptrList[ens][phMTag]['row'].append(int(t.split()[1]))
            #----------------------------------------------
# End-for ensList

# Create dictionary to map actual gamma matrices to latex format
gMatLatex = {'gt'  : '\gamma_t',
             'gxg5': '\gamma_x\gamma_5',
             'gyg5': '\gamma_y\gamma_5',
             'gzg5': '\gamma_z\gamma_5',
             'gtg5': '\gamma_t\gamma_5',
             'gxgy': '\gamma_x\gamma_y',
             'gxgz': '\gamma_x\gamma_z',
             'gxgt': '\gamma_x\gamma_t',
             'gygz': '\gamma_y\gamma_z',
             'gygt': '\gamma_y\gamma_t',
             'gzgt': '\gamma_z\gamma_t'}
#----------------------------------------------

# Create dictionary to map actual gamma matrices to operator names and rows
gMatMap =  {'gt'  : ('b_b0xDA__J0_A1pP', 1),
            'gxg5': ('a_a1xDA__J1_T1pM', 1),
            'gyg5': ('a_a1xDA__J1_T1pM', 3),
            'gzg5': ('a_a1xDA__J1_T1pM', 2),
            'gtg5': ('pion_pion_2xDA__J0_A1mM', 1),
            'gxgy': ('b_b1xDA__J1_T1pP', 2),
            'gxgz': ('b_b1xDA__J1_T1pP', 3),
            'gxgt': ('rho_rho_2xDA__J1_T1mP', 3),
            'gygz': ('b_b1xDA__J1_T1pP', 1),
            'gygt': ('rho_rho_2xDA__J1_T1mP', 1),
            'gzgt': ('rho_rho_2xDA__J1_T1mP', 2)}





print('Insertion operator List:\n', insOptrList)


print('\nDone\n')


### Ensemble-related definitions

In [ ]:
L = {'cl21_32_64_b6p3_m0p2350_m0p2050': 32}
T = {'cl21_32_64_b6p3_m0p2350_m0p2050': 64}

alat_fm = {'cl21_32_64_b6p3_m0p2350_m0p2050': 0.091}
mpi_GeV = {'cl21_32_64_b6p3_m0p2350_m0p2050': 350.0}
mN_GeV  = {'cl21_32_64_b6p3_m0p2350_m0p2050': 1.160}

ens_main = 'cl21_48_96_b6p3_m0p2416_m0p2050'
for ens_str in ensStreams[ens_main]:
    ens = constructFullEns(ens_main,ens_str)
    L[ens] = 48
    T[ens] = 96
    alat_fm[ens] = 0.091
    mpi_GeV[ens] = 170.0
    mN_GeV[ens] = 0.932

uM = {}
alat_iGeV = {}
amN = {}
for ens in ensList:
    uM[ens] = 2.0*Pi/L[ens]                   # Unit momentum
    alat_iGeV[ens] = alat_fm[ens] / 0.1973    # Lattice spacing in GeV^{-1}
    amN[ens] = mN_GeV[ens] * alat_iGeV[ens]   # Lattice nucleon mass

    
    
# Dispersion relation
def dispRel(ens,am,k):
    ap = uM[ens] * k
    return np.sqrt(am**2 + np.dot(ap,ap))

print('Unit momentum is (lattice units):', uM)
print('Lattice spacing is (fm):', alat_fm)
print('Lattice spacing is (GeV^{-1}):', alat_iGeV)

print('Done\n')

### Plotting parameters (colors, markers, styles, etc)

In [ ]:
pDct   = {0: 'ro', 1: 'bs', 2: 'g^', 3: 'mp', 4: 'cd'}
cDct   = {0: 'r', 1: 'b', 2: 'g', 3: 'm', 4: 'c', 5: 'darkorange', 6: 'y', 7: 'gold', 8: 'darkviolet'}
mDct   = {0: 'o', 1: 's', 2: '^', 3: 'p', 4: 'd', 5: 'v', 6: '>', 7: 'X', 8: '<'}
osDct  = {0: 0.0 , 1: 0.15, 2: 0.3, 3: 0.45, 4: 0.6}
osDctF = {0: 0.0 , 1: 0.07, 2: 0.15, 3: 0.22, 4: 0.3,
          5: 0.37, 6: 0.45, 7: 0.52, 8: 0.60, 9: 0.67}
fsDct = {1: 'full',2:'none'}



elw=1 # Line width of error bars
ecs=6 # Capsize of error bars


### Jackknife-related functions

In [ ]:
binsize=1

def jackknife_ave(bins,Nbins,tL):

    Jax = 0
    
    if(tL==1):
        ave = np.mean(bins,dtype=bins.dtype)

        sqsum = sum(map(lambda x:x*x,ave-bins))
        fac = (Nbins -1) / float(Nbins)
        err = np.sqrt(fac*sqsum)
    else:
        ave = np.mean(bins, axis=Jax,dtype=bins.dtype)

        err = np.zeros(tL,dtype=np.float128)
        for i in range(tL):
#            sqsum = sum(map(lambda x:x*x,ave[i]-bins[:,i].real))
            sqsum = sum(map(lambda x:x*x,ave[i]-bins[:,i]))
            fac = (Nbins -1) / float(Nbins)
            err[i] = np.sqrt(fac*sqsum.real)        
            
    return (ave,err)
#-------------------------------------------------------

def jackknife_binning(corr, binsize=1):
    
    Jax=0
    
    # Define the number of bins
    mod   = Ncfg%binsize
    Nbins = int((Ncfg - mod) / binsize)
    bins = np.zeros((Nbins,Nt),dtype=corr.dtype)

    csum = np.sum(corr,axis=Jax) # Sum w.r.t to the configurations
    for t in range(Nt):
        for m in np.arange(1,mod+1):
            csum[t] -= corr[Ncfg-m,t]  # Throw away data in case there is modulo
            
        for b in range(Nbins):
            bsum = 0
            for k in range(binsize):
                bsum += corr[b*binsize+k,t]
                
            bins[b][t] = (csum[t] - bsum) / float(Ncfg - binsize - mod) # Bin averages for each bin,t

    return bins, Nbins
#-------------------------------------------------------

print('Done\n')

### Fit functions for constant and linear fits

In [ ]:
# Constant fit
def constFit(x,cVal):
    return cVal

# Chi-square for constant fit
def chiSq_const(x, y, err, cFit):
    return sum( map(lambda t:t*t, (y-constFit(x,cFit))/err) )

# Perform Constant fit
def PerformConstFit(val,err):
    Sy = sum( map(lambda x:x,val/(err**2)) )           # Sy = SUM_i val[i]/err[i]**2
    S  = sum( map(lambda x:1.0/(x*x),err) )            # S  = SUM_i ( 1/err[i] )^2
    return Sy/S
#------------------------

    
# Linear fit
def linearFit(x,M,b):
    return b + M*x

# Chi-square for Linear fit
def chiSq_linear(x, y, err, M, b):    
    Nd = len(x)
    chi = 0
    for i in range(Nd):
        chi += ( (y[i] - linearFit(x[i],M,b))/err[i] )**2
    
    return chi
#------------------------

print('Done\n')

### Fit functions for exponential fits

We perform 4 different types of fits for the summation method:

(i) $E_1$: 1-exponential fit on the summed ratio, of the form $f(x) = Z + Mx + Axe^{-\Delta Ex}$

(ii) $E_2$: 2-exponential fit on the summed ratio, of the form $f(x) = Z + Mx + Axe^{-\Delta Ex} + Bx^2e^{-\Delta Ex}$

(iii) $E_{1r}$: 1-exponential fit on the reduced-summed ratio, of the form $f(x) = M + Ae^{-\Delta Ex}$

(iv) $E_{2r}$: 2-exponential fit on the reduced-summed ratio, of the form $f(x) = M + Ae^{-\Delta Ex} + Bxe^{-\Delta Ex}$

In all the above expressions, $M$, $Z$, $A$, $B$ and $\Delta E$ are fit parameters, with $M$ the desired matrix element.
Within the Variable Projection method, we redefine the $\chi^2 \equiv \sum_k \left( \frac{y_k -f(x_k)}{\sigma} \right)^2$ so that it depends only on one of the fit parameters, $\Delta E$, and the rest of the fit parameters depend on $\Delta E$. This is expected to significantly improve the convergence of the fits.

In [ ]:
# 1 exp. fit of regular summed-ratio
def exp1Fit(x,M,Z,A,dE):
    return Z + M*x + A*x*np.exp(-dE*x)

def chiSq_exp1(x, y, err, M, Z, A, dE):    
    Nd = len(x)
    chi = 0
    for i in range(Nd):
        chi += ( (y[i] - exp1Fit(x[i],M,Z,A,dE))/err[i] )**2
    
    return chi
#------------------------


# 1 exp. fit of reduced-summed ratio
def exp1rFit(x,M,A,dE):
    return M + A*np.exp(-dE*x)

def chiSq_exp1r(x, y, err, M, A, dE):    
    Nd = len(x)
    chi = 0
    for i in range(Nd):
        chi += ( (y[i] - exp1rFit(x[i],M,A,dE))/err[i] )**2
    
    return chi
#------------------------

# 2 exp. fit of reduced-summed ratio
def exp2rFit(x,M,A,B,dE):
    return M + A*np.exp(-dE*x) + B*x*np.exp(-dE*x)

def chiSq_exp2r(x, y, err, M, A, B, dE):    
    Nd = len(x)
    chi = 0
    for i in range(Nd):
        chi += ( (y[i] - exp2rFit(x[i],M,A,B,dE))/err[i] )**2
    
    return chi
#------------------------

print('Done\n')

### Variable projection for exponential fits

In [ ]:

fitTypesVPR  = ['E1']   # Fit types for the regular-summed ratio
fitTypesVPRr = ['E1r','E2r']  # Fit types for the reduced-summed ratio

fitTypesVP = fitTypesVPR + fitTypesVPRr

# Basis functions of the models
def PhiVP0(x,dE):
    return 1
def PhiVPL(x,dE):
    return x
def PhiVP1(x,dE):
    return np.exp(-dE*x)
def PhiVP2(x,dE):
    return x * PhiVP1(x,dE)
def PhiVP3(x,dE):
    return x * PhiVP2(x,dE)

PhiVP = {'E1': [PhiVP0, PhiVPL, PhiVP2]        , 'E1r': [PhiVP0, PhiVP1],
         'E2': [PhiVP0, PhiVPL, PhiVP2, PhiVP3], 'E2r': [PhiVP0, PhiVP1, PhiVP2]}


CVP = {'E1': ['Z','M','A']    , 'E1r': ['M','A'],
       'E2': ['Z','M','A','B'], 'E2r': ['M','A','B']}


NbVP = {}
for ifitT in fitTypesVP:
    NbVP[ifitT] = len(PhiVP[ifitT])
#--------------------------------

def Y(dE):
    rY = NbVP[fitTVP]
    Yv = np.zeros(rY,dtype=np.float64)
    Nd = len(xDataVP)
    for i in range(rY):
        for k in range(Nd):
            Yv[i] += yDataVP[k] * PhiVP[fitTVP][i](xDataVP[k],dE) / errDataVP[k]**2
    return Yv

def PhiVPM(dE):
    rPhiVP = NbVP[fitTVP]
    Nd = len(xDataVP)
    mPhiVP = np.zeros((rPhiVP,rPhiVP),dtype=np.float64)
    for i in range(rPhiVP):
        for j in range(rPhiVP):
            for k in range(Nd):
                mPhiVP[i,j] += PhiVP[fitTVP][i](xDataVP[k],dE)*PhiVP[fitTVP][j](xDataVP[k],dE) / errDataVP[k]**2
    return mPhiVP

def chiSq_VP(dE):
    PhiVPInv = np.linalg.inv(PhiVPM(dE))
    PhiVPInvTr = np.transpose(PhiVPInv)
    return ( np.dot(yDataVP/errDataVP,yDataVP/errDataVP) - np.dot( np.transpose(Y(dE)), np.dot((2*PhiVPInv - PhiVPInvTr),Y(dE)) ) )


def expModel(eT, x, C):
    dE = C[-1]
    fx = 0
    for i in range(NbVP[eT]):
        fx += C[i]*PhiVP[eT][i](x,dE)
    return fx


print('\nDone\n')

### Fit functions for Variable Projection Fits using the two-state fit method

The initial form of the fitting function, arising from the two- and three-point function expressions is $\frac{A_{00}e^{-E_0t_s}+A_{11}e^{-E_1t_s}+A_{01}\left(e^{-E_0(t_s-t_{ins})}e^{-E_1t_{ins}} +e^{-E_1(t_s-t_{ins})}e^{-E_0t_{ins}} \right)}{c_0e^{-E_0t_s}+c_1 e^{-E_1t_s}}$, where $A_{00}$, $A_{01}$, $A_{11}$, $c_0$, $c_1$,  $E_0$ and $E_1$ are fit parameters. From such a fit the matrix element is obtained as $M = \frac{A_{00}}{c_0}$. With a little manipulation of this expression, one can write it as $\frac{M+A_{11}e^{-\Delta Et_s}+A_{01}\left(e^{-\Delta Et_{ins}} + e^{-\Delta E(t_s-t_{ins})}\right)}{1+c_1 e^{-\Delta Et_s}}$. One advantage of this form is that there are now five instead of seven fit parameters, $M$, $A_{01}$, $A_{11}$, $c_1$ and $\Delta E \equiv E_1 - E_0$. In this case, $M$ is directly obtained as a fit parameter instead of a ratio of two other fit parameters. In addition, this form saves the fits from interchange of fit parameters due to symmetry.

As mentioned above, the two-state fit model has the form:

$f(t_s,t_{ins}) = \frac{M+A_{11}e^{-\Delta Et_s}+A_{01}\left(e^{-\Delta Et_{ins}} + e^{-\Delta E(t_s-t_{ins})}\right)}{1+c_1 e^{-\Delta Et_s}}$

For the variable projection fits, we can write this model as $f(t_s,t_{ins}) = \sum_i p_i \Phi_i(\Delta E, c_1;t_s,t_{ins})$, where $p_0 = M$, $p_1 = A_{11}$, $p_2 = A_{01}$ and the basis functions are

$\Phi_0 = \frac{1}{1+c_1 e^{-\Delta Et_s}}$, $\Phi_1 = \frac{e^{-\Delta Et_s}}{1+c_1 e^{-\Delta Et_s}}$ and $\Phi_2 = \frac{e^{-\Delta Et_{ins}} + e^{-\Delta E(t_s-t_{ins})}}{1+c_1 e^{-\Delta Et_s}}$

In [ ]:
# Two-state fit (Factorized exp. terms, absorbed c0)
def twoStateFitOpt(T,M,A01,A11,c1,dE):
    ti,ts = T
    
    return ( (M + A11*np.exp(-dE*ts) + A01*(np.exp(-dE*ti)+np.exp(-dE*(ts-ti))) ) / 
                 (1 + c1*np.exp(-dE*ts)) )


def chiSq_twoStateOpt(T,y,err,M,A01,A11,c1,dE):
    chi = 0
    TI,TS = T
    Nd = len(TI)
    i = 0
    for i in range(Nd):
        chi += ( (y[i] - twoStateFitOpt((TI[i],TS[i]),M,A01,A11,c1,dE))/err[i] )**2
        i += 1
    return chi
#------------------------



# Two-state fit (Factorized exp. terms, absorbed c0 AND denominator)
def twoStateFitOptDen(T,M,A01,A11,dE):
    ti,ts = T
    
    return (M + A11*np.exp(-dE*ts) + A01*(np.exp(-dE*ti)+np.exp(-dE*(ts-ti))) )


def chiSq_twoStateOptDen(T,y,err,M,A01,A11,dE):
    chi = 0
    TI,TS = T
    Nd = len(TI)
    i = 0
    for i in range(Nd):
        chi += ( (y[i] - twoStateFitOptDen((TI[i],TS[i]),M,A01,A11,dE))/err[i] )**2
        i += 1
    return chi
#------------------------

print('Done\n')

### Variable Projection for the two-state fit method

In [ ]:

# Basis functions of the two-state fit model, x = t_ins
def PhiVP0_2st(x,ts,P):
    dE = P[0]
    c1 = P[1]
    return 1.0 / (1.0 + c1*np.exp(-dE*ts))    
#    return (1.0 - c1*np.exp(-dE*ts))
#    return 1.0
def PhiVP1_2st(x,ts,P):
    dE = P[0]
    c1 = P[1]
    return np.exp(-dE*ts) * PhiVP0_2st(x,ts,P)
#    return np.exp(-dE*ts)
def PhiVP2_2st(x,ts,P):
    dE = P[0]
    c1 = P[1]
    return (np.exp(-dE*x) + np.exp(-dE*(ts-x))) * PhiVP0_2st(x,ts,P)    
#    return (np.exp(-dE*x) + np.exp(-dE*(ts-x)))


PhiVP_2st = [PhiVP0_2st, PhiVP1_2st, PhiVP2_2st]

CVP_2st = ['M','A11','A01']

NbVP_2st = len(PhiVP_2st)
#--------------------------------

def Y_2st(P):
    rY = NbVP_2st
    Yv = np.zeros(rY,dtype=np.float64)
    tsL = tsList2st_VP
    for i in range(rY):
        for ts in tsL:
            xD = xData2st_VP[ts]
            yD = yData2st_VP[ts]
            eD = errData2st_VP[ts]
            Nd = len(xD)
            for k in range(Nd):
                Yv[i] += yD[k] * PhiVP_2st[i](xD[k],ts,P) / eD[k]**2
    return Yv
#-------------

def PhiMVP_2st(P):
    r = NbVP_2st
    tsL = tsList2st_VP
    mPhiVP_2st = np.zeros((r,r),dtype=np.float64)
    for i in range(r):
        for j in range(r):
            for ts in tsL:
                xD = xData2st_VP[ts]
                eD = errData2st_VP[ts]
                Nd = len(xD)
                for k in range(Nd):
                    mPhiVP_2st[i,j] += PhiVP_2st[i](xD[k],ts,P)*PhiVP_2st[j](xD[k],ts,P) / eD[k]**2
    return mPhiVP_2st
#-------------

def yTld():
    tsL = tsList2st_VP
    ydp = 0
    for ts in tsL:
        yD = yData2st_VP[ts]
        eD = errData2st_VP[ts]
        ydp += np.dot(yD/eD,yD/eD)
    return ydp
#-------------

def chiSq_VP_2st(P):
    PhiVPInv = np.linalg.inv(PhiMVP_2st(P))
    PhiVPInvTr = np.transpose(PhiVPInv)
    Yf = Y_2st(P)
    yk = yTld()
    
    return ( yk - np.dot( np.transpose(Yf), np.dot((2*PhiVPInv - PhiVPInvTr),Yf) ) )


print('\nDone\n')

## Read two-point functions, perform Jackknife sampling

In [ ]:
def get2ptFileName(phM,t0Tag,src,snk,srow,mFTag,nvec):
    filePre = 'corr_2pt.baryon.n%d'%(nvec)
    
    srcTag = 'src_%s_%d'%(src,srow)
    snkTag = 'snk_%s_%d'%(snk,srow)
    
    FileName = '%s.%s.%s.%s.%s.%s.dat'%(filePre,phM,t0Tag,srcTag,snkTag,mFTag)
    
    return FileName
#--------------------------

def get2ptFileDir(mainDir,t0Tag,mFTag):
    return '%s/%s/%s' % (mainDir,t0Tag,mFTag)
#--------------------------------------------------------------

c2pt_Allbins   = {}
c2pt_Allave    = {}
c2pt_rowAvg    = {}
c2pt_rawAllave = {}
c2pt_bins      = {}
c2pt_ave       = {}
c2pt_rawAve    = {}

Nbins_2pt_Dict = {}

effEnergy_Allbins = {}
effEnergy_Allave  = {}
effEnergy_bins    = {}
effEnergy_ave     = {}

for ens in ensList:
    
    c2pt_Allbins[ens]   = {}
    c2pt_Allave[ens]    = {}
    c2pt_rowAvg[ens]    = {}
    c2pt_rawAllave[ens] = {}
    c2pt_bins[ens]      = {}
    c2pt_ave[ens]       = {}
    c2pt_rawAve[ens]    = {}

    effEnergy_Allbins[ens] = {}
    effEnergy_Allave[ens]  = {}
    effEnergy_bins[ens]    = {}
    effEnergy_ave[ens]     = {}    
    
    Nbins_2pt_Dict[ens] = {}
    
    for opT in SrcSnkOpTypes[ens]:
        for iphT in phaseTypes[ens]:
            for phM in Phases_MomTags[(ens,iphT)]:
                phMTag = phasedTag(iphT,phM)
                phTpl = (phMTag,opT)    

                momTags = momTags_Dict[ens][phTpl]

                for mTag in momTags:
                    mVec = momVector_Dict[ens][mTag]
                    zMom = mVec[2]
                    mFTag = momFileTag(mVec)
                    pmTpl = phMomTpl(phMTag,zMom)

                    nRow = nRow2pt_Dict[ens][opT][pmTpl]

                    srcOpList2pt = srcOpList2pt_Dict[ens][opT][pmTpl]        
                    snkOpList2pt = srcOpList2pt_Dict[ens][opT][pmTpl]

                    dTpl = (phMTag,opT,mTag)
                    Ncfg = Ncfg_Dict_2pt[ens][dTpl]
                    
                    Nt = Nt_Dict_2pt[ens][pmTpl]

                    # Determine number of t0 we consider
                    Nt0 = 0
                    for t0 in T0List[ens][phTpl]:
                        dTpl2pt = (phMTag,opT,mTag,t0)
                        if existsDataSet_2pt[ens][dTpl2pt]:
                            Nt0 += 1

                    for isrc, src in enumerate(srcOpList2pt):
                        for isnk, snk in enumerate(snkOpList2pt):
                            ckey = (opT,phMTag,mTag,isrc,isnk) # All info in one tuple

                            c2pt_rawAll = np.zeros((Ncfg,Nt,nRow,Nt0),dtype=np.complex128)

                            for it0,t0 in enumerate(T0List[ens][phTpl]):
                                dTpl2pt = (phMTag,opT,mTag,t0)
                                if existsDataSet_2pt[ens][dTpl2pt]:
                                    t0Tag = 't0_%d'%(t0)

                                    for ir,irow in enumerate(range(1,nRow+1)):
                                        ckeyA = (opT,phMTag,mTag,isrc,isnk,t0,irow) # All info in one tuple

                                        c2ptFileDir  = get2ptFileDir(corr2ptMainDir[ens][phTpl],t0Tag,mFTag)
                                        c2ptFileName = get2ptFileName(phM,t0Tag,src,snk,irow,mFTag,Nvec[ens])               
                                        c2ptFile = '%s/%s' % (c2ptFileDir,c2ptFileName)
#                                        print(c2ptFile)

                                        with open(c2ptFile) as fp:
                                            line = fp.readlines()
                                            c = 0
                                            for n in line:
                                                it   = c%Nt
                                                icfg = c//Nt
                                                c2pt_rawAll[icfg,it,ir,it0] = complex(np.float64(n.split()[1]),
                                                                                      np.float64(n.split()[2]))
                                                c += 1
                                        # Reading file

                                        # Jackknife average on All two-point functions
                                        c2pt_Allbins[ens][ckeyA], Nbins_2pt_Dict[ens][dTpl] = jackknife_binning(c2pt_rawAll[:,:,ir,it0].real)
                                        Nbins_2pt = Nbins_2pt_Dict[ens][dTpl]
                                        c2pt_Allave[ens][ckeyA] = jackknife_ave(c2pt_Allbins[ens][ckeyA],Nbins_2pt,tL=Nt)

                                        # Effective energy of All t0 and rows
                                        effEnergy_Allbins[ens][ckeyA] = np.zeros((Nbins_2pt,Nt),dtype=np.float128)
                                        for it in range(Nt):
                                            effEnergy_Allbins[ens][ckeyA][:,it] = np.log(c2pt_Allbins[ens][ckeyA][:,it]/
                                                                                    c2pt_Allbins[ens][ckeyA][:,(it+1)%Nt])

                                        effEnergy_Allave[ens][ckeyA] = jackknife_ave(effEnergy_Allbins[ens][ckeyA],Nbins_2pt,Nt)
                                    # for-irow

                                    # Standard average and standard error (still average over rows)
                                    # Might need this for constructing covariance matrix
                                    ckeyB = (opT,phMTag,mTag,isrc,isnk,t0)
                                    c2pt_rowAvg[ens][ckeyB] = np.mean(c2pt_rawAll[:,:,:,it0],axis=2,dtype=np.complex128) # Average over rows
                                    c2pt_rawAllave[ens][ckeyB] = (np.mean(c2pt_rowAvg[ens][ckeyB],axis=0),
                                                             np.std(c2pt_rowAvg[ens][ckeyB],axis=0)/np.sqrt(Ncfg))
                                else:
                                    print('### Skipping dataset:', dTpl2pt)
                            # for-t0

                            # Average over rows and t0
                            c2pt_Tmp1 = np.mean(c2pt_rawAll,axis=3,dtype=np.complex128) # over t0
                            c2pt_Tmp2 = np.mean(c2pt_Tmp1  ,axis=2,dtype=np.complex128) # over rows

                            # Standard average and standard error
                            c2pt_rawAve[ens][ckey] = (np.mean(c2pt_Tmp2,axis=0),np.std(c2pt_Tmp2,axis=0)/np.sqrt(Ncfg))

                            # Jackknife-average on t0- and row-average            
                            c2pt_bins[ens][ckey], Nbins_2pt = jackknife_binning(c2pt_Tmp2.real)
                            c2pt_ave[ens][ckey] = jackknife_ave(c2pt_bins[ens][ckey],Nbins_2pt,tL=Nt)

                            effEnergy_bins[ens][ckey] = np.zeros((Nbins_2pt,Nt),dtype=np.float128)
                            for it in range(Nt):
                                effEnergy_bins[ens][ckey][:,it] = np.log(c2pt_bins[ens][ckey][:,it]/
                                                                    c2pt_bins[ens][ckey][:,(it+1)%Nt])

                            effEnergy_ave[ens][ckey] = jackknife_ave(effEnergy_bins[ens][ckey],Nbins_2pt,tL=Nt)                    

                    # for-src-snk
                    print('Done: ',ens,phMTag, ', Momentum %s'%(mTag))
                # for momentum

           
print('\nDone\n')

### Plot Effective energy

In [ ]:
# Two-point function
fig1, ax1 = plt.subplots(figsize=[12,7.5])

iSrc = 0 #srcOpList.index('NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1')
iSnk = 0 #snkOpList.index('NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1')
print('isrc: %s\nisnk: %s'%(iSrc,iSnk))

opTP = 'single'

phTP = 'unphased'
#phMP = 'd001_2.00'
phMP = 'unphased'

#ens = 'cl21_32_64_b6p3_m0p2350_m0p2050'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-main'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-1000'
ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-1200'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-all'


phMTagP = phasedTag(phTP,phMP)

# Plot one momentum, all effective energies
mTag = 'pz.-2'
mVec = momVector_Dict[ens][mTag]
zMom = mVec[2]
pmTpl = phMomTpl(phMTagP,zMom)

Nt = Nt_Dict_2pt[ens][pmTpl]

nRow = nRow2pt_Dict[ens][opTP][pmTpl]
xRng = np.arange(Nt)

i=0
osi=0
for t0 in T0List[ens][(phMTagP,opTP)]:
    dTpl2pt = (phMTagP,opTP,mTag,t0)
    if existsDataSet_2pt[ens][dTpl2pt]:

        for irow in range(1,nRow+1):
            ckey = (opTP,phMTagP,mTag,iSrc,iSnk,t0,irow)

            ax1.errorbar(xRng+osDctF[osi],  effEnergy_Allave[ens][ckey][0].real,
                         yerr = effEnergy_Allave[ens][ckey][1],
                         marker = mDct[i], color = cDct[i], ls='none', label=r'$t_0=%d$, row=%d'%(t0,irow),
                         ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle=fsDct[irow])

            ax1.axhline(dispRel(ens,amN[ens],mVec),lw=1,color='k',ls='--')

            osi+=1
        # for-irow
        i+=1
    else:
        print('Skipping ', dTpl2pt)

ckey = (opTP,phMTagP,mTag,iSrc,iSnk)
ax1.errorbar(xRng-0.2,  effEnergy_ave[ens][ckey][0].real,
             yerr = effEnergy_ave[ens][ckey][1],
             fmt = 'kX', label=r'$\vec{p}=(%d,%d,%d)$ Avg.'%(mVec[0],mVec[1],mVec[2]),
             ms = 9.0, elinewidth=2, capsize=ecs)#,fillstyle='none')

ax1.set_xlabel(r'$t/a$',fontsize=18)
ax1.set_ylabel(r'$aE_{\rm eff}(Pz=%d,t)$, %s'%(mVec[2],ensTag[ens]),fontsize=18)
ax1.set_xlim(0,18.5)
ax1.xaxis.set_ticks(np.arange(0, 18.5, 2))

if ens == 'cl21_32_64_b6p3_m0p2350_m0p2050':
    if np.abs(mVec[2]) >= 5:
        ax1.set_ylim(0.2,2.0)
    elif np.abs(mVec[2]) == 4 or np.abs(mVec[2]) == 3:
        ax1.set_ylim(0.4,1.2)
    else:
        ax1.set_ylim(0.4,0.9)
elif 'cl21_48_96_b6p3_m0p2416_m0p2050' in ens:
    if np.abs(mVec[2]) == 2:
        ax1.set_ylim(0.0,1.0)
    else:
        ax1.set_ylim(0.1,0.8)

    
ax1.legend(loc='best', ncol=2, numpoints=1, fontsize=18,
           labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
           markerscale=0.8)

fig1.savefig('plots/%s/aEeff.%s.all_t0.rowAll.pdf'%(ens,momFileTag(mVec)), bbox_inches='tight')


### Average two-point functions over momentum, compute averaged Eff. energy

In [ ]:

c2ptA_bins      = {}
c2ptA_ave       = {}

effEnergyA_bins      = {}
effEnergyA_ave       = {}

for ens in ensList:
    c2ptA_bins[ens]      = {}
    c2ptA_ave[ens]       = {}

    effEnergyA_bins[ens]      = {}
    effEnergyA_ave[ens]       = {}
    
    for opT in SrcSnkOpTypes[ens]:
        for iphT in phaseTypes[ens]:            
            for phM in Phases_MomTags_ave[(ens,iphT)]:
                phMTag = phasedTag(iphT,phM)
                phTpl = (phMTag,opT)        

                momTags = momTags_Dict_ave[ens][phTpl]


                for mTag in momTags:
                    mVec = momVector_Dict[ens][mTag]
                    zMom = mVec[2]
                    mFTag = momFileTag(mVec)
                    pmTpl = phMomTpl(phMTag,zMom)

                    srcOpList2pt = srcOpList2pt_Dict[ens][opT][pmTpl]        
                    snkOpList2pt = srcOpList2pt_Dict[ens][opT][pmTpl]

                    dTpl = (phMTag,opT,mTag)
                    Nbins =  Nbins_2pt_Dict[ens][dTpl]

                    Nt = Nt_Dict_2pt[ens][pmTpl]

                    for isrc, src in enumerate(srcOpList2pt):
                        for isnk, snk in enumerate(snkOpList2pt):
                            ckey = (opT,phMTag,mTag,isrc,isnk) # All info in one tuple

                            c2ptA_bins[ens][ckey] = np.zeros((Nbins,Nt),dtype=np.float128)
                            if mTag == 'pz.0':
                                c2ptA_bins[ens][ckey] = c2pt_bins[ens][ckey]
                            else:
                                mTagN = 'pz.-%d'%(mVec[2])
                                if iphT == 'unphased':
                                    phMN = 'unphased'
                                else:
                                    phMN = createPhMomTag('-'+getPhasedValueStr(phM))
                                phMTagN = phasedTag(iphT,phMN)

                                ckeyP = (opT,phMTag ,mTag ,isrc,isnk)
                                ckeyN = (opT,phMTagN,mTagN,isrc,isnk)

                                c2ptA_bins[ens][ckey] = 0.5 * (c2pt_bins[ens][ckeyP] + c2pt_bins[ens][ckeyN])


                            # Averaged Effective Energy
                            effEnergyA_bins[ens][ckey] = np.zeros((Nbins,Nt),dtype=np.float128)
                            for it in range(Nt):
                                effEnergyA_bins[ens][ckey][:,it] = np.log(c2ptA_bins[ens][ckey][:,it]/
                                                                     c2ptA_bins[ens][ckey][:,(it+1)%Nt])

                            effEnergyA_ave[ens][ckey] = jackknife_ave(effEnergyA_bins[ens][ckey],Nbins_2pt,tL=Nt)                    

                            c2ptA_ave[ens][ckey] = jackknife_ave(c2ptA_bins[ens][ckey],Nbins,tL=Nt)
                    # for-src-snk

                    print('Done: ',ens,phMTag, ', Momentum %s'%(mTag))
                # for momentum

           
print('\nDone\n')

### Perform constant fit on the Averaged Effective Energy

In [ ]:

tini = 6
tfin = 12
tFit = np.arange(tini,tfin+1)
Ndata = len(tFit)

E0_fit = {}
E0_cov = {}

E0_fit_ave = {}

for ens in ensList:
    E0_fit[ens] = {}
    E0_cov[ens] = {}

    E0_fit_ave[ens] = {}


    for opT in SrcSnkOpTypes[ens]:
        for iphT in phaseTypes[ens]:
            for phM in Phases_MomTags_ave[(ens,iphT)]:
                phMTag = phasedTag(iphT,phM)
                phTpl = (phMTag,opT)        

                momTags = momTags_Dict_ave[ens][phTpl]

                for mTag in momTags:
                    adTplE = (phMTag,opT,mTag)
                    if analyzeDataSetE[ens][adTplE]:
                        mVec = momVector_Dict[ens][mTag]
                        zMom = mVec[2]
                        pmTpl = phMomTpl(phMTag,zMom)

                        srcOpList2pt = srcOpList2pt_Dict[ens][opT][pmTpl]        
                        snkOpList2pt = srcOpList2pt_Dict[ens][opT][pmTpl]

                        dTpl = (phMTag,opT,mTag)
                        Nbins = Nbins_2pt_Dict[ens][dTpl]

                        for isrc, src in enumerate(srcOpList2pt):
                            for isnk, snk in enumerate(snkOpList2pt):
                                ckey = (opT,phMTag,mTag,isrc,isnk) # All info in one tuple

                                E0_fit[ens][ckey] = np.zeros(Nbins,dtype=np.float128)
                                E0_cov[ens][ckey] = np.zeros((Nbins,Ndata),dtype=np.float128)

                                E0_fitAll = np.zeros(Nbins,dtype=np.float128)
                                E0_covAll = np.zeros((Nbins,Ndata),dtype=np.float128)

                                nFit = 0
                                for ib in range(Nbins):
                                    ydata = effEnergyA_bins[ens][ckey][ib,tini:tfin+1]
                                    yerr  = effEnergyA_ave[ens][ckey][1][tini:tfin+1] 
                                    if (not np.isnan(ydata).any()):
                                        E0_fitAll[nFit] = PerformConstFit(ydata,yerr)
    #                                    E0_fitAll[nFit], E0_covAll[nFit] = scipyOpt.curve_fit(constFit,tFit,ydata,sigma=yerr)
                                        nFit += 1

                                E0_fit[ens][ckey] =  E0_fitAll[:nFit]

                                if (np.shape(E0_fit[ens][ckey])==(0,)) or (np.shape(E0_fit[ens][ckey])==(1,)):
                                    E0_fit_ave[ens][ckey] = (0,0)
                                else:
                                    E0_fit_ave[ens][ckey] = jackknife_ave(E0_fit[ens][ckey],Nbins=nFit,tL=1)
                    else:
                        print('Skipping dataset: ',adTplE)

                    print('Done: ',ens,phMTag, ', Momentum %s'%(mTag))
                # for-momentum
            
    
print('\nDone\n')


### Plot momentum-averaged Effective energy

In [ ]:
fig2, ax2 = plt.subplots(figsize=[14,8.75])

iSrc = 0
iSnk = 0
print('isrc: %s\nisnk: %s'%(iSrc,iSnk))

opTP = 'single'


#ens = 'cl21_32_64_b6p3_m0p2350_m0p2050'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-main'
ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-1000'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-1200'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-all'

# Plot average Eff. energy for all momenta
i=0
for phTP in phaseTypes[ens]:
    for phMP in Phases_MomTags_ave[(ens,phTP)]:
    
        phMTagP = phasedTag(phTP,phMP)


        for mTag in momTags_Dict_ave[ens][(phMTagP,opTP)]:
            mVec  = momVector_Dict[ens][mTag]
            zMom  = mVec[2]
            zMomN = -mVec[2]
            mTagN = MomTag(zMomN)
            mVecN = momVector_Dict[ens][mTagN]


            pmTpl = phMomTpl(phMTagP,zMom)

            Nt = Nt_Dict_2pt[ens][pmTpl]
            xRng = np.arange(Nt)

            adTplE = (phMTagP,opTP,mTag)
            if analyzeDataSetE[ens][adTplE]:# and (np.abs(zMom)<3):
                if phTP == 'unphased':
                    phMN = 'unphased'
                else:
                    phMN = createPhMomTag('-'+getPhasedValueStr(phMP))
                phMTagN = phasedTag(phTP,phMN)

                ckeyP = (opTP,phMTagP,mTag ,iSrc,iSnk)
                ckeyN = (opTP,phMTagN,mTagN,iSrc,iSnk)

                # Positive momentum
                ax2.errorbar(xRng,  effEnergy_ave[ens][ckeyP][0].real,
                             yerr = effEnergy_ave[ens][ckeyP][1],
                             marker = mDct[i], color = cDct[i], ls='none',
                             label=r'$\vec{p}=(%d,%d,%d)$'%(mVec[0],mVec[1],mVec[2]),
                             ms = 7.0, elinewidth=elw, capsize=ecs,fillstyle='full')

                if mTag != 'pz.0':
                    # Negative momentum
                    ax2.errorbar(xRng+osDct[1], effEnergy_ave[ens][ckeyN][0].real,
                                 yerr = effEnergy_ave[ens][ckeyN][1],
                                 marker = mDct[i], color = cDct[i], ls='none',
                                 label=r'$\vec{p}=(%d,%d,%d)$'%(mVecN[0],mVecN[1],mVecN[2]),
                                 ms = 7.0, elinewidth=elw, capsize=ecs,fillstyle='none')

                    # Momentum Averaged
                    ax2.errorbar(xRng-osDct[1], effEnergyA_ave[ens][ckeyP][0].real,
                                 yerr = effEnergyA_ave[ens][ckeyP][1],
                                 marker = 'x', color = cDct[i], ls='none',
                                 label=r'$\vec{p}=(%d,%d,%d)$ Avg.'%(mVec[0],mVec[1],mVec[2]),
                                 ms = 7.0, elinewidth=elw, capsize=ecs,fillstyle='full')


                    
                ax2.axhline(dispRel(ens,amN[ens],mVec),lw=1,color=cDct[i],ls='--')

                cFitv = E0_fit_ave[ens][ckeyP][0]
                cFite = E0_fit_ave[ens][ckeyP][1]
                if cFitv != 0 or cFite != 0:
                    ax2.fill_between(tFit, cFitv-cFite, cFitv+cFite, facecolor=cDct[i],
                                     alpha=0.4)#, label= 'const. fit')
                else:
                    print('Fit for ', ckeyP, 'was not proper')
                    
                print(E0_fit_ave[ens][ckeyP])

                i+=1
    
ax2.set_xlabel(r'$t/a$',fontsize=18)
ax2.set_ylabel(r'$aE_{\rm eff}(t)$, %s'%(ensTag[ens]),fontsize=18)
ax2.xaxis.set_ticks(np.arange(0, 22, 2))

if ens == 'cl21_32_64_b6p3_m0p2350_m0p2050':
    ax2.set_ylim(0.45,2)
elif 'cl21_48_96_b6p3_m0p2416_m0p2050' in ens:
    ax2.set_ylim(0.1,1.0)

    
# ax2.yaxis.set_ticks(np.arange(0.45, 0.86, 0.05))

ax2.set_xlim(0,18.2)
ax2.xaxis.set_ticks(np.arange(0, 18.2, 2))



ax2.legend(loc='best', ncol=4, numpoints=1, fontsize=18,
           labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
           markerscale=0.8)

fig2.savefig('plots/%s/aEeff.allMom.t0Ave.rowAve.pdf'%(ens),bbox_inches='tight')




### Plot momentum-averaged Effective energy for multiple ensembles

In [ ]:

fig3, ax3 = plt.subplots(figsize=[14,8.75])

iSrc = 0
iSnk = 0
print('isrc: %s\nisnk: %s'%(iSrc,iSnk))

opTP = 'single'
phTP = 'unphased'

ens_main = 'cl21_48_96_b6p3_m0p2416_m0p2050'

# Plot average Eff. energy for all momenta

ie = None
ens_str = 'all'
ens = constructFullEns(ens_main,ens_str)

phMP = 'unphased'
phMTagP = phasedTag(phTP,phMP)


i=0    
for mTag in momTags_Dict_ave[ens][(phMTagP,opTP)]:
    mVec  = momVector_Dict[ens][mTag]
    zMom  = mVec[2]

    pmTpl = phMomTpl(phMTagP,zMom)

    Nt = Nt_Dict_2pt[ens][pmTpl]
    xRng = np.arange(Nt)

    adTplE = (phMTagP,opTP,mTag)
    if analyzeDataSetE[ens][adTplE]:# and (np.abs(zMom)<3):
        ckeyP = (opTP,phMTagP,mTag ,iSrc,iSnk)

        # Momentum Averaged
        ax3.errorbar(xRng+osDct[i], effEnergyA_ave[ens][ckeyP][0].real,
                     yerr = effEnergyA_ave[ens][ckeyP][1],
                     marker = mDct[i], color = cDct[i], ls='none',
                     label=r'$\vec{p}=(%d,%d,%d)$'%(mVec[0],mVec[1],mVec[2]),
                     ms = 7.0, elinewidth=elw, capsize=ecs,fillstyle='full')

        ax3.axhline(dispRel(ens,amN[ens],mVec),lw=1,color=cDct[i],ls='--')

        cFitv = E0_fit_ave[ens][ckeyP][0]
        cFite = E0_fit_ave[ens][ckeyP][1]
        if cFitv != 0 or cFite != 0:
            ax3.fill_between(tFit, cFitv-cFite, cFitv+cFite, facecolor=cDct[i],
                             alpha=0.4)#, label= 'const. fit')
        else:
            print('Fit for ', ckeyP, 'was not proper')

        print(E0_fit_ave[ens][ckeyP])

        i+=1
    
ax3.set_xlabel(r'$t/a$',fontsize=18)
ax3.set_ylabel(r'$aE_{\rm eff}(t)$, %s'%(ensTag[ens]),fontsize=18)
ax3.xaxis.set_ticks(np.arange(0, 22, 2))

if ens == 'cl21_32_64_b6p3_m0p2350_m0p2050':
    ax3.set_ylim(0.45,2)
elif 'cl21_48_96_b6p3_m0p2416_m0p2050' in ens:
    ax3.set_ylim(0.1,1.0)

    
# ax3.yaxis.set_ticks(np.arange(0.45, 0.86, 0.05))

ax3.set_xlim(0,18.2)
ax3.xaxis.set_ticks(np.arange(0, 18.2, 2))



ax3.legend(loc='best', ncol=4, numpoints=1, fontsize=18,
           labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
           markerscale=0.8)

fig3.savefig('plots/%s/aEeff.allMom.t0Ave.rowAve.allStr.pdf'%(ens),bbox_inches='tight')




## Read / Load three-point functions, perform Jackknife sampling, average over $p_z$ and $z_3$

In [ ]:
c3pt_data_Source = {'cl21_32_64_b6p3_m0p2350_m0p2050': {'Read': False, 'Dump': False, 'Load': False}}


ens_main = 'cl21_48_96_b6p3_m0p2416_m0p2050'
for ens_stream in ensStreams[ens_main]:
    c3pt_data_Source[constructFullEns(ens_main,ens_stream)] = {'Read': True, 'Dump': True,'Load': False}

def get3ptFileName(phM,t0Tag,tsnkTag,src,snk,srcRow,snkRow,ins,insRow,mFTag,d,nvec):
    filePre = 'corr_3pt.baryon.n%d'%(nvec)

    srcTag = 'src_%s_%d'%(src,srcRow)
    snkTag = 'snk_%s_%d'%(snk,snkRow)
    insTag = 'ins_%s_%d'%(ins,insRow)

    dTag = dispTag(d)

    FileName = '%s.%s.%s.%s.%s.%s.%s.%s.%s.dat'%(filePre,phM,t0Tag,tsnkTag,srcTag,snkTag,insTag,dTag,mFTag)

    return FileName
#--------------------------

def get3ptFileDir(mainDir,t0Tag,mFTag,tsnkTag):
    return '%s/%s/%s/%s' % (mainDir,t0Tag,mFTag,tsnkTag)
#--------------------------------------------------------------


c3pt_Allbins_Re = {}
c3pt_Allbins_Im = {}
c3pt_Allave_Re  = {}
c3pt_Allave_Im  = {}

c3pt_rawAll = {}

print('\nDone\n')



In [ ]:

for ens in ensList:

    if c3pt_data_Source[ens]['Read']:

        c3pt_Allbins_Re[ens] = {}
        c3pt_Allbins_Im[ens] = {}
        c3pt_Allave_Re[ens]  = {}
        c3pt_Allave_Im[ens]  = {}

        c3pt_rawAll[ens] = {}

        for opT in SrcSnkOpTypes[ens]:
            for iphT in phaseTypes[ens]:
                for phM in Phases_MomTags[(ens,iphT)]:
                    phMTag = phasedTag(iphT,phM)
                    phTpl = (phMTag,opT)    
                    phT = None

                    momTags = momTags_Dict[ens][phTpl]

                    for mTag in momTags:
                        mVec = momVector_Dict[ens][mTag]
                        zMom = mVec[2]
                        mFTag = momFileTag(mVec)
                        pmTpl = phMomTpl(phMTag,zMom)

                        nRow = nRow3pt_Dict[ens][opT][pmTpl]

                        srcOpList3pt = srcOpList3pt_Dict[ens][opT][pmTpl]        
                        snkOpList3pt = srcOpList3pt_Dict[ens][opT][pmTpl]        

                        for tsnk in TsinkList[ens][(phMTag,opT)]:
                            tsnkTag = 'tsnk_%d'%(tsnk)
                            dTpl = (phMTag,opT,mTag,tsnk)
                            Ncfg = Ncfg_Dict_3pt[ens][dTpl]
                            Nt = tsnk

                            for it0,t0 in enumerate(T0List[ens][(phMTag,opT)]):
                                t0Tag = 't0_%d'%(t0)

                                dTpl3pt = (phMTag,opT,mTag,tsnk,t0)
                                dTpl2pt = (phMTag,opT,mTag,t0)
                                if existsDataSet_3pt[ens][dTpl3pt] and existsDataSet_2pt[ens][dTpl2pt]:
                                    print('Reading dataset: ', ens, dTpl3pt)
                                    for isrc, src in enumerate(srcOpList3pt):
                                        for isnk, snk in enumerate(snkOpList3pt):

                                            # Only unpolarized PDF for now
                                            irowSrc = 1
                                            irowSnk = 1
                                            gMat = 'gt'
                                            insOpName = gMatMap[gMat][0]
                                            insOpRow  = gMatMap[gMat][1]

                                            for d in dispLen[ens]:
                                                ckeyA = (opT,phMTag,mTag,tsnk,isrc,isnk,gMat,d,irowSrc,irowSnk,t0) # All info in one tuple
                                                c3pt_rawAll[ens][ckeyA] = np.zeros((Ncfg,Nt),dtype=np.complex128)

                                                c3ptFileDir  = get3ptFileDir(corr3ptMainDir[ens][phTpl],t0Tag,mFTag,tsnkTag)
                                                c3ptFileName = get3ptFileName(phM,t0Tag,tsnkTag,src,snk,irowSrc,irowSnk,
                                                                              insOpName,insOpRow,mFTag,d,Nvec[ens])               
                                                c3ptFile = '%s/%s' % (c3ptFileDir,c3ptFileName)

                                                with open(c3ptFile) as fp:
                                                    line = fp.readlines()
                                                    c = 0
                                                    for n in line:
                                                        it   = c%Nt
                                                        icfg = c//Nt
                                                        c3pt_rawAll[ens][ckeyA][icfg,it] = complex(np.float64(n.split()[1]),
                                                                                              np.float64(n.split()[2]))
                                                        c += 1
                                                # Reading file

                                                # Jackknife average on All three-point functions
                                                c3pt_Allbins_Re[ens][ckeyA], Nbins_tmp1 = jackknife_binning(c3pt_rawAll[ens][ckeyA].real)
                                                c3pt_Allbins_Im[ens][ckeyA], Nbins_tmp2 = jackknife_binning(c3pt_rawAll[ens][ckeyA].imag)
                                                c3pt_Allave_Re[ens][ckeyA] = jackknife_ave(c3pt_Allbins_Re[ens][ckeyA],Nbins_tmp1,tL=Nt)
                                                c3pt_Allave_Im[ens][ckeyA] = jackknife_ave(c3pt_Allbins_Im[ens][ckeyA],Nbins_tmp2,tL=Nt)
                                            # for-displacement
                                    # for-src-snk
                                else:
                                    print('### Skipping dataset:', dTpl3pt)
                            # for-t0
                            print('Done: ',ens,phMTag, ', Momentum %s'%(mTag), ', t_sink = %d'%(tsnk))
                        # for tsnk
                    # for momentum
    else:
        print('Skipping reading the raw three-point functions for ensemble %s'%(ens))

print('\nDone\n')

### Dump / Load raw three-point function using Pickle

In [ ]:
for ens in ensList:
    if c3pt_data_Source[ens]['Dump']:
        print('Dumping the raw three-point functions for ensemble %s...'%(ens))
        pickle.dump(c3pt_rawAll[ens]    , open('p_data/%s/c3pt_rawAll.p'%(ens)    , 'wb' ))
        pickle.dump(c3pt_Allbins_Re[ens], open('p_data/%s/c3pt_Allbins_Re.p'%(ens), 'wb' ))
        pickle.dump(c3pt_Allbins_Im[ens], open('p_data/%s/c3pt_Allbins_Im.p'%(ens), 'wb' ))
        pickle.dump(c3pt_Allave_Re[ens] , open('p_data/%s/c3pt_Allave_Re.p'%(ens) , 'wb' ))
        pickle.dump(c3pt_Allave_Im[ens] , open('p_data/%s/c3pt_Allave_Im.p'%(ens) , 'wb' ))
    elif c3pt_data_Source[ens]['Load']:
        print('Loading the raw three-point functions for ensemble %s...'%(ens))
        c3pt_rawAll[ens]     = pickle.load( open('p_data/%s/c3pt_rawAll.p'%(ens)    , 'rb' ) )
        c3pt_Allbins_Re[ens] = pickle.load( open('p_data/%s/c3pt_Allbins_Re.p'%(ens), 'rb' ) )
        c3pt_Allbins_Im[ens] = pickle.load( open('p_data/%s/c3pt_Allbins_Im.p'%(ens), 'rb' ) )
        c3pt_Allave_Re[ens]  = pickle.load( open('p_data/%s/c3pt_Allave_Re.p'%(ens) , 'rb' ) )
        c3pt_Allave_Im[ens]  = pickle.load( open('p_data/%s/c3pt_Allave_Im.p'%(ens) , 'rb' ) )
    else:
        print('Skipping dumping / loading of raw three-point functions for ensemble %s'%(ens))
    
print('\nDone\n')

### Average three-point functions over $t_0$, perform Jackknife sampling

In [ ]:
c3pt_sampling = {'cl21_32_64_b6p3_m0p2350_m0p2050': {'Evaluate': False, 'Dump': False, 'Load': False}}


ens_main = 'cl21_48_96_b6p3_m0p2416_m0p2050'
for ens_stream in ensStreams[ens_main]:
    c3pt_sampling[constructFullEns(ens_main,ens_stream)] = {'Evaluate': True, 'Dump': True,'Load': False}



c3pt_rawAve = {}

c3pt_bins_Re = {}
c3pt_bins_Im = {}

c3pt_ave_Re = {}
c3pt_ave_Im = {}

Nbins_3pt_Dict = {}


In [ ]:

for ens in ensList:
    c3pt_rawAve[ens] = {}

    c3pt_bins_Re[ens] = {}
    c3pt_bins_Im[ens] = {}

    c3pt_ave_Re[ens] = {}
    c3pt_ave_Im[ens] = {}

    Nbins_3pt_Dict[ens] = {}    

    if c3pt_sampling[ens]['Evaluate']:

        for opT in SrcSnkOpTypes[ens]:
            for iphT in phaseTypes[ens]:
                for phM in Phases_MomTags[(ens,iphT)]:
                    phMTag = phasedTag(iphT,phM)
                    phTpl = (phMTag,opT)    

                    momTags = momTags_Dict[ens][phTpl]

                    for mTag in momTags:
                        mVec = momVector_Dict[ens][mTag]
                        zMom = mVec[2]
                        mFTag = momFileTag(mVec)
                        pmTpl = phMomTpl(phMTag,zMom)

                        nRow = nRow3pt_Dict[ens][opT][pmTpl]

                        srcOpList3pt = srcOpList3pt_Dict[ens][opT][pmTpl]        
                        snkOpList3pt = srcOpList3pt_Dict[ens][opT][pmTpl]        

                        for tsnk in TsinkList[ens][(phMTag,opT)]:
                            tsnkTag = 'tsnk_%d'%(tsnk)
                            dTpl = (phMTag,opT,mTag,tsnk)
                            Ncfg = Ncfg_Dict_3pt[ens][dTpl]
                            Nt = tsnk

                            adTplM = (phMTag,opT,mTag,tsnk)
                            if analyzeDataSetM[ens][adTplM]:                
                                for isrc, src in enumerate(srcOpList3pt):
                                    for isnk, snk in enumerate(snkOpList3pt):

                                        # Only unpolarized PDF for now
                                        irowSrc = 1
                                        irowSnk = 1
                                        gMat = 'gt'

                                        for d in dispLen[ens]:
                                            ckey = (opT,phMTag,mTag,tsnk,isrc,isnk,gMat,d,irowSrc,irowSnk)

                                            # Average over t0
                                            c3pt_t0_ave = np.zeros((Ncfg,Nt),dtype=np.complex128)
                                            Nt0 = 0
                                            for it0,t0 in enumerate(T0List[ens][(phMTag,opT)]):
                                                ckeyA = (opT,phMTag,mTag,tsnk,isrc,isnk,gMat,d,irowSrc,irowSnk,t0)

                                                dTpl3pt = (phMTag,opT,mTag,tsnk,t0)
                                                dTpl2pt = (phMTag,opT,mTag,t0)
                                                if existsDataSet_3pt[ens][dTpl3pt] and existsDataSet_2pt[ens][dTpl2pt]:
                                                    Nt0 += 1
                                                    c3pt_t0_ave = c3pt_t0_ave + c3pt_rawAll[ens][ckeyA]                                        
                                            # for-t0
                                            c3pt_t0_ave = c3pt_t0_ave / float(Nt0)


                                            # Standard average and standard error
                                            c3pt_rawAve[ens][ckey] = (np.mean(c3pt_t0_ave,axis=0,dtype=np.complex128),
                                                                 np.std(c3pt_t0_ave,axis=0,dtype=np.complex128)/np.sqrt(Ncfg))

                                            # Jackknife-average on t0- and row-average            
                                            c3pt_bins_Re[ens][ckey], Nbins_3pt_Dict[ens][dTpl] = jackknife_binning(c3pt_t0_ave.real)
                                            c3pt_bins_Im[ens][ckey], Nbins_3pt_tmp3            = jackknife_binning(c3pt_t0_ave.imag)

                                            c3pt_ave_Re[ens][ckey] = jackknife_ave(c3pt_bins_Re[ens][ckey],Nbins_3pt_Dict[ens][dTpl],tL=Nt)
                                            c3pt_ave_Im[ens][ckey] = jackknife_ave(c3pt_bins_Im[ens][ckey],Nbins_3pt_Dict[ens][dTpl],tL=Nt)

                                    # for-displacement
                                # for-src-snk
                                print('Done: ',ens,phMTag, ', Momentum %s'%(mTag), ', t_sink = %d'%(tsnk))
                            # if-analyze
                        # for tsnk
                    # for momentum
    else:
        print('Skipping evaluating the three-point function samples for ensemble %s'%(ens))

           
print('\nDone\n')

### Write / Load non-averaged 3pt-function samples and average

In [ ]:
for ens in ensList:
    if c3pt_sampling[ens]['Dump']:
        print('Dumping the three-point function samples for ensemble %s...'%(ens))
        pickle.dump(c3pt_rawAve[ens], open('p_data/%s/c3pt_rawAve.p'%(ens), 'wb' ))

        pickle.dump(c3pt_bins_Re[ens], open('p_data/%s/c3pt_bins_Re.p'%(ens), 'wb' ))
        pickle.dump(c3pt_bins_Im[ens], open('p_data/%s/c3pt_bins_Im.p'%(ens), 'wb' ))

        pickle.dump(c3pt_ave_Re[ens], open('p_data/%s/c3pt_ave_Re.p'%(ens), 'wb' ))
        pickle.dump(c3pt_ave_Im[ens], open('p_data/%s/c3pt_ave_Im.p'%(ens), 'wb' ))

        pickle.dump(Nbins_3pt_Dict[ens], open('p_data/%s/Nbins_3pt_Dict.p'%(ens), 'wb' ))
    elif c3pt_sampling[ens]['Load']:
        print('Loading the three-point function samples for ensemble %s...'%(ens))
    #    c3pt_rawAve = pickle.load( open('c3pt_rawAve.p'%(ens), 'rb'))

        c3pt_bins_Re[ens] = pickle.load( open('p_data/%s/c3pt_bins_Re.p'%(ens), 'rb'))
        c3pt_bins_Im[ens] = pickle.load( open('p_data/%s/c3pt_bins_Im.p'%(ens), 'rb'))

        c3pt_ave_Re[ens] = pickle.load( open('p_data/%s/c3pt_ave_Re.p'%(ens), 'rb'))
        c3pt_ave_Im[ens] = pickle.load( open('p_data/%s/c3pt_ave_Im.p'%(ens), 'rb'))

        Nbins_3pt_Dict[ens] = pickle.load( open('p_data/%s/Nbins_3pt_Dict.p'%(ens), 'rb'))
    else:
        print('Skipping dumping/loading of three-point function samples for ensemble %s'%(ens))

print('\nDone\n')

### Average three-point function over $p_z$ and $z$ values

In [ ]:
c3pt_ave_sampling = {'cl21_32_64_b6p3_m0p2350_m0p2050': {'Evaluate': False, 'Dump': False, 'Load': False}}

ens_main = 'cl21_48_96_b6p3_m0p2416_m0p2050'
for ens_stream in ensStreams[ens_main]:
    c3pt_ave_sampling[constructFullEns(ens_main,ens_stream)] = {'Evaluate': True, 'Dump': True,'Load': False}



c3ptA_bins = {}
c3ptA_ave  = {}
for ens in ensList:
    c3ptA_bins[ens] = {'Re': {}, 'Im': {}}
    c3ptA_ave[ens]  = {'Re': {}, 'Im': {}}

In [ ]:

for ens in ensList:

    if c3pt_ave_sampling[ens]['Evaluate']:

        for opT in SrcSnkOpTypes[ens]:
            for iphT in phaseTypes[ens]:            
                for phM in Phases_MomTags_ave[(ens,iphT)]:
                    phMTag = phasedTag(iphT,phM)
                    phTpl = (phMTag,opT)        

                    momTags = momTags_Dict_ave[ens][phTpl]

                    for mTag in momTags:
                        mVec = momVector_Dict[ens][mTag]
                        zMom = mVec[2]
                        mFTag = momFileTag(mVec)
                        pmTpl = phMomTpl(phMTag,zMom)

                        nRow = nRow3pt_Dict[ens][opT][pmTpl]

                        srcOpList3pt = srcOpList3pt_Dict[ens][opT][pmTpl]        
                        snkOpList3pt = srcOpList3pt_Dict[ens][opT][pmTpl]        

                        for tsnk in TsinkList[ens][(phMTag,opT)]:
                            tsnkTag = 'tsnk_%d'%(tsnk)
                            dTpl = (phMTag,opT,mTag,tsnk)
                            Nbins = Nbins_3pt_Dict[ens][dTpl]
                            Nt = tsnk

                            for isrc, src in enumerate(srcOpList3pt):
                                for isnk, snk in enumerate(snkOpList3pt):

                                    # Only unpolarized PDF for now
                                    irowSrc = 1
                                    irowSnk = 1
                                    gMat = 'gt'

                                    for d in dispLen_ave[ens]:
                                        ckey = (opT,phMTag,mTag,tsnk,isrc,isnk,gMat,d,irowSrc,irowSnk)

                                        c3ptA_bins[ens]['Re'][ckey] = np.zeros((Nbins,Nt),dtype=np.float128)
                                        c3ptA_bins[ens]['Im'][ckey] = np.zeros((Nbins,Nt),dtype=np.float128)

                                        adTplM = (phMTag,opT,mTag,tsnk)
                                        if analyzeDataSetM[ens][adTplM]:                                    
                                            if mTag == 'pz.0':
                                                if d==0: # (pz=0, z=0)
                                                    c3ptA_bins[ens]['Re'][ckey] = c3pt_bins_Re[ens][ckey]
                                                    c3ptA_bins[ens]['Im'][ckey] = c3pt_bins_Im[ens][ckey]
                                                else: # (pz=0, z!=0)
                                                    ckey_Posd = (opT,phMTag,mTag,tsnk,isrc,isnk,gMat, d,irowSrc,irowSnk)
                                                    ckey_Negd = (opT,phMTag,mTag,tsnk,isrc,isnk,gMat,-d,irowSrc,irowSnk)
                                                    c3ptA_bins[ens]['Re'][ckey] = 0.5 * (c3pt_bins_Re[ens][ckey_Posd] + c3pt_bins_Re[ens][ckey_Negd])
                                                    c3ptA_bins[ens]['Im'][ckey] = 0.5 * (c3pt_bins_Re[ens][ckey_Posd] - c3pt_bins_Re[ens][ckey_Negd])
                                            else:
                                                mTagN = 'pz.-%d'%(mVec[2])
                                                if iphT == 'unphased':
                                                    phMN = 'unphased'
                                                else:
                                                    phMN = createPhMomTag('-'+getPhasedValueStr(phM))
                                                phMTagN = phasedTag(iphT,phMN)                                            

                                                if d==0: # (pz!=0, z=0)
                                                    ckey_PosP = (opT,phMTag ,mTag ,tsnk,isrc,isnk,gMat,d,irowSrc,irowSnk)
                                                    ckey_NegP = (opT,phMTagN,mTagN,tsnk,isrc,isnk,gMat,d,irowSrc,irowSnk)
                                                    c3ptA_bins[ens]['Re'][ckey] = 0.5 * (c3pt_bins_Re[ens][ckey_PosP] + c3pt_bins_Re[ens][ckey_NegP])
                                                    c3ptA_bins[ens]['Im'][ckey] = 0.5 * (c3pt_bins_Re[ens][ckey_PosP] - c3pt_bins_Re[ens][ckey_NegP])
                                                else: # (pz!=0, z!=0)
                                                    ckey_PosP_Posd = (opT,phMTag ,mTag ,tsnk,isrc,isnk,gMat, d,irowSrc,irowSnk)
                                                    ckey_PosP_Negd = (opT,phMTag ,mTag ,tsnk,isrc,isnk,gMat,-d,irowSrc,irowSnk)
                                                    ckey_NegP_Posd = (opT,phMTagN,mTagN,tsnk,isrc,isnk,gMat, d,irowSrc,irowSnk)
                                                    ckey_NegP_Negd = (opT,phMTagN,mTagN,tsnk,isrc,isnk,gMat,-d,irowSrc,irowSnk)

                                                    c3ptA_bins[ens]['Re'][ckey] = 0.25 * (c3pt_bins_Re[ens][ckey_PosP_Posd] + c3pt_bins_Re[ens][ckey_PosP_Negd] +
                                                                                          c3pt_bins_Re[ens][ckey_NegP_Posd] + c3pt_bins_Re[ens][ckey_NegP_Negd])

                                                    c3ptA_bins[ens]['Im'][ckey] = 0.25 * (c3pt_bins_Im[ens][ckey_PosP_Posd] - c3pt_bins_Im[ens][ckey_PosP_Negd] -
                                                                                          c3pt_bins_Im[ens][ckey_NegP_Posd] + c3pt_bins_Im[ens][ckey_NegP_Negd])
                                        # End-if

                                        # Jackknife-average on p,z-averaged three-point function
                                        for ri in ReIm:
                                            c3ptA_ave[ens][ri][ckey] = jackknife_ave(c3ptA_bins[ens][ri][ckey],Nbins_3pt_Dict[ens][dTpl],tL=Nt)
                                    # for-displacement
                            # for-src-snk-ins
                            print('Done: ',ens,phMTag, ', Momentum %s'%(mTag), ', t_sink = %d'%(tsnk))
                        # for tsnk
                    # for momentum
    else:
        print('Skipping evaluating the averaged three-point function samples for ensemble %s \n'%(ens))


print('\nDone\n')


### Write / Load 3pt-function p,z-averaged bins and average

In [ ]:
for ens in ensList:
    if c3pt_ave_sampling[ens]['Dump']:
        print('Dumping the averaged three-point function samples for ensemble %s...'%(ens))
        for ri in ReIm:
            pickle.dump(c3ptA_bins[ens][ri], open('p_data/%s/c3ptA_bins_%s.p'%(ens,ri), 'wb' ))
            pickle.dump(c3ptA_ave[ens][ri],  open('p_data/%s/c3ptA_ave_%s.p'%(ens,ri), 'wb' ))

        pickle.dump(Nbins_3pt_Dict[ens], open('p_data/%s/Nbins_3pt_Dict.p'%(ens), 'wb' ))
    elif c3pt_ave_sampling[ens]['Load']:
        print('Loading the averaged three-point function samples for ensemble %s...'%(ens))
        for ri in ReIm:
            c3ptA_bins[ens][ri] = pickle.load(open('p_data/%s/c3ptA_bins_%s.p'%(ens,ri), 'rb'))
            c3ptA_ave[ens][ri]  = pickle.load(open('p_data/%s/c3ptA_ave_%s.p'%(ens,ri), 'rb'))

        Nbins_3pt_Dict[ens] = pickle.load( open('p_data/%s/Nbins_3pt_Dict.p'%(ens), 'rb'))
    else:
        print('Skipping dumping/loading of averaged three-point function samples for ensemble %s'%(ens))



print('\nDone\n')

## Construct Ratio of Averaged three- and two-point functions, $R(t_s,t_{\rm ins}) = C_{3pt}(t_s,t_{\rm ins})\; /\; C_{2pt}(t_s)$, and perform Summation of Ratio


In [ ]:
ratio_Source = {'cl21_32_64_b6p3_m0p2350_m0p2050': {'Evaluate': False, 'Dump': False, 'Load': True}}

ens_main = 'cl21_48_96_b6p3_m0p2416_m0p2050'
for ens_stream in ensStreams[ens_main]:
    ratio_Source[constructFullEns(ens_main,ens_stream)] = {'Evaluate': True, 'Dump': True,'Load': False}


RA_bins = {}
RA_ave  = {}

RsumA_bins = {}
RsumA_ave  = {}

RrsumA_bins = {}
RrsumA_ave  = {}



In [ ]:

for ens in ensList:

    RA_bins[ens] = {}
    RA_ave[ens]  = {}

    RsumA_bins[ens] = {}
    RsumA_ave[ens]  = {}

    RrsumA_bins[ens] = {}
    RrsumA_ave[ens]  = {}    
    
    if ratio_Source[ens]['Evaluate']:

        for ri in ReIm:
            RA_bins[ens][ri] = {}
            RA_ave[ens][ri] = {}

            RsumA_bins[ens][ri] = {}
            RsumA_ave[ens][ri] = {}

            RrsumA_bins[ens][ri] = {}
            RrsumA_ave[ens][ri] = {}


        for opT in SrcSnkOpTypes[ens]:
            for iphT in phaseTypes[ens]:            
                for phM in Phases_MomTags_ave[(ens,iphT)]:
                    phMTag = phasedTag(iphT,phM)
                    phTpl = (phMTag,opT)

                    momTags = momTags_Dict_ave[ens][phTpl]

                    for mTag in momTags:
                        mVec = momVector_Dict[ens][mTag]
                        zMom = mVec[2]
                        mFTag = momFileTag(mVec)
                        pmTpl = phMomTpl(phMTag,zMom)

                        srcOpList = srcOpList3pt_Dict[ens][opT][pmTpl] # These are the same in two- and
                        snkOpList = srcOpList3pt_Dict[ens][opT][pmTpl] # three-point functions

                        for isrc, src in enumerate(srcOpList):
                            for isnk, snk in enumerate(snkOpList):
                                ckey2 = (opT,phMTag,mTag,isrc,isnk) # 2pt-tuple

                                for tsnk in TsinkList[ens][(phMTag,opT)]:
                                    tsnkTag = 'tsnk_%d'%(tsnk)
                                    Ntins = tsnk

                                    dTpl = (phMTag,opT,mTag,tsnk)
                                    Nbins = Nbins_3pt_Dict[ens][dTpl]

                                    adTplM = (phMTag,opT,mTag,tsnk)
                                    if analyzeDataSetM[ens][adTplM]:

                                        # Only unpolarized PDF for now
                                        irowSrc = 1
                                        irowSnk = 1
                                        gMat = 'gt'

                                        for z in dispLen_ave[ens]:
                                            ckey3 = (opT,phMTag,mTag,tsnk,isrc,isnk,gMat,z,irowSrc,irowSnk) # 3pt-tuple

                                            for ri in ReIm:
                                                RA_bins[ens][ri][ckey3] = np.zeros((Nbins,Ntins),dtype=np.float128)
                                                RsumA_bins[ens][ri][ckey3] = np.zeros(Nbins,dtype=np.float128)
                                                for tins in range(Ntins):
                                                    RA_bins[ens][ri][ckey3][:,tins] = c3ptA_bins[ens][ri][ckey3][:,tins] / c2ptA_bins[ens][ckey2][:,tsnk]

                                                    if tins>0: # Exclude source contact term
                                                        RsumA_bins[ens][ri][ckey3] += RA_bins[ens][ri][ckey3][:,tins]

                                                RA_ave[ens][ri][ckey3] = jackknife_ave(RA_bins[ens][ri][ckey3],Nbins,tL=Ntins)
                                                RsumA_ave[ens][ri][ckey3] = jackknife_ave(RsumA_bins[ens][ri][ckey3],Nbins,tL=1)
                                # for-tsink

                                # Evaluate reduced summed ratio
                                TsinkList_rs = TsinkList[ens][(phMTag,opT)][:-1]
                                for its,tsnk in enumerate(TsinkList_rs):
                                    tsnkH = TsinkList[ens][(phMTag,opT)][its+1]
                                    tsnkL = TsinkList[ens][(phMTag,opT)][its]

                                    dTpl = (phMTag,opT,mTag,tsnk)
                                    Nbins = Nbins_3pt_Dict[ens][dTpl]

                                    adTplM = (phMTag,opT,mTag,tsnk)
                                    if analyzeDataSetM[ens][adTplM]:

                                        # Only unpolarized PDF for now
                                        irowSrc = 1
                                        irowSnk = 1
                                        gMat = 'gt'

                                        for z in dispLen_ave[ens]:
                                            ckey3  = (opT,phMTag,mTag,tsnk ,isrc,isnk,gMat,z,irowSrc,irowSnk)
                                            ckey3H = (opT,phMTag,mTag,tsnkH,isrc,isnk,gMat,z,irowSrc,irowSnk)
                                            ckey3L = (opT,phMTag,mTag,tsnkL,isrc,isnk,gMat,z,irowSrc,irowSnk)
                                            for ri in ReIm:
                                                RrsumA_bins[ens][ri][ckey3] = (RsumA_bins[ens][ri][ckey3H] - RsumA_bins[ens][ri][ckey3L]) / (tsnkH - tsnkL)
                                                RrsumA_ave[ens][ri][ckey3] = jackknife_ave(RrsumA_bins[ens][ri][ckey3],Nbins,tL=1)
                                    # end-if
                                # for-tsink

                        print('Done: ',ens,phMTag, ', Momentum %s'%(mTag))
    else:
        print('Skipping evaluating the Ratio for ensemble %s'%(ens))


print('\nDone\n')

### Write / Load averaged Ratio of three- and two-point functions, $R(t_s,t_{\rm ins}) = C_{3pt}(t_s,t_{\rm ins})\; /\; C_{2pt}(t_s)$

In [ ]:
for ens in ensList:
    if ratio_Source[ens]['Dump']:
        print('Dumping the Ratio for ensemble %s...'%(ens))
        pickle.dump(RA_bins[ens], open('p_data/%s/RA_bins.p'%(ens), 'wb' ))
        pickle.dump(RA_ave[ens], open( 'p_data/%s/RA_ave.p'%(ens), 'wb' ))
        pickle.dump(RsumA_bins[ens], open('p_data/%s/RsumA_bins.p'%(ens), 'wb' ))
        pickle.dump(RsumA_ave[ens], open( 'p_data/%s/RsumA_ave.p'%(ens), 'wb' ))
        pickle.dump(RrsumA_bins[ens], open('p_data/%s/RrsumA_bins.p'%(ens), 'wb' ))
        pickle.dump(RrsumA_ave[ens], open( 'p_data/%s/RrsumA_ave.p'%(ens), 'wb' ))
    elif ratio_Source[ens]['Load']:
        print('Loading the Ratio for ensemble %s...'%(ens))
        RA_bins[ens] = pickle.load(open('p_data/%s/RA_bins.p'%(ens), 'rb'))
        RA_ave[ens] = pickle.load( open('p_data/%s/RA_ave.p'%(ens), 'rb'))
        RsumA_bins[ens] = pickle.load(open('p_data/%s/RsumA_bins.p'%(ens), 'rb'))
        RsumA_ave[ens] = pickle.load( open('p_data/%s/RsumA_ave.p'%(ens), 'rb'))
        RrsumA_bins[ens] = pickle.load(open('p_data/%s/RrsumA_bins.p'%(ens), 'rb'))
        RrsumA_ave[ens] = pickle.load( open('p_data/%s/RrsumA_ave.p'%(ens), 'rb'))

        Nbins_3pt_Dict[ens] = pickle.load( open('p_data/%s/Nbins_3pt_Dict.p'%(ens), 'rb'))
    else:
        print('Skipping dumping/loading of Ratio for ensemble %s'%(ens))


#print('Nbins:')
#print(Nbins_3pt_Dict)

print('\nDone\n')


## Perform / Load plateau fits on the Ratio

We perform constant fits on symmetrical ranges of the ratio for each value of $t_s$. In selecting the best plateau fit range for every $t_{s}$, the criterion is to select the first fit range of which $\chi^2<0.8$.

In [ ]:
plateauFit_Source = {'Evaluate': False, 'Dump': False, 'Load': True}

Mplat_bins = {}
Mplat_ave  = {}

chiPlat_bins = {}
chiPlat_ave  = {}

optPlatFit = {}

# Keep the same ranges for real, imaginary for now
eyePlatFit = {'Re': {4:0, 6:1, 8:1, 10:2, 12:3,14:4},
              'Im': {4:0, 6:1, 8:1, 10:2, 12:3,14:4}}


### Create generic structures

In [ ]:
if plateauFit_Source['Evaluate']:

    fitInfo_RP = {}
    Nfits_RP = {}
    for opT in SrcSnkOpTypes:
        for iphT in phaseTypes:            
            for phM in Phases_MomTags_ave[iphT]:
                phMTag = phasedTag(iphT,phM)
                phTpl = (phMTag,opT)
            
                fitInfo_RP[phTpl] = {}
                Nfits_RP[phTpl] = {}
                for tsnk in TsinkList[phTpl]:
                    fitInfo_RP[phTpl][tsnk] = {}
                    Nfits_RP[phTpl][tsnk] = tsnk//2-1
                    fitInfo_RP[phTpl][tsnk]['Nfits'] = Nfits_RP[phTpl][tsnk]

                    tini = 1 # Leave out the source point
                    tfin = tsnk-1
                    fitInfo_RP[phTpl][tsnk]['Rng'] = []
                    for nf in range(Nfits_RP[phTpl][tsnk]):
                        fitInfo_RP[phTpl][tsnk]['nf=%d'%(nf)] = {}
                        tstart = tini+nf
                        tstop  = tfin-nf
                        Npts = tstop-tstart+1
                        fitInfo_RP[phTpl][tsnk]['nf=%d'%(nf)]['xdata']  = np.arange(Npts+1)
                        fitInfo_RP[phTpl][tsnk]['nf=%d'%(nf)]['tstart'] = tstart
                        fitInfo_RP[phTpl][tsnk]['nf=%d'%(nf)]['tstop']  = tstop
                        fitInfo_RP[phTpl][tsnk]['nf=%d'%(nf)]['Npts']   = Npts

                        fitInfo_RP[phTpl][tsnk]['Rng'].append('%d-%d'%(tstart,tstop))

print('Done\n')

### Perform plateau fits

In [ ]:

if plateauFit_Source['Evaluate']:

    for ri in ReIm:
        Mplat_bins[ri] = {}
        Mplat_ave[ri]  = {}

        chiPlat_bins[ri] = {}
        chiPlat_ave[ri]  = {}

        optPlatFit[ri] = {}

    chiCrit = 0.8

    for opT in SrcSnkOpTypes:
        for iphT in phaseTypes:            
            for phM in Phases_MomTags_ave[iphT]:
                phMTag = phasedTag(iphT,phM)
                phTpl = (phMTag,opT)

                momTags = momTags_Dict_ave[phTpl]

                for mTag in momTags:
                    mVec = momVector_Dict[mTag]
                    zMom = mVec[2]
                    mFTag = momFileTag(mVec)
                    pmTpl = phMomTpl(phMTag,zMom)

                    srcOpList = srcOpList3pt_Dict[opT][pmTpl] # These are the same in two- and
                    snkOpList = srcOpList3pt_Dict[opT][pmTpl] # three-point functions

                    for tsnk in TsinkList[phTpl]:                        
                        dTpl = (phMTag,opT,mTag,tsnk)
                        Nbins = Nbins_3pt_Dict[dTpl]

                        adTplM = (phMTag,opT,mTag,tsnk)
                        if analyzeDataSetM[adTplM]:

                            for isrc, src in enumerate(srcOpList):
                                for isnk, snk in enumerate(snkOpList): 
                                    
                                    # Only upolarized PDF for now
                                    irowSrc = 1
                                    irowSnk = 1
                                    gMat = 'gt'

                                    for z in dispLen_ave:
                                        ckey3 = (opT,phMTag,mTag,tsnk,isrc,isnk,gMat,z,irowSrc,irowSnk)

                                        for ri in ReIm:
                                            Mplat_bins[ri][ckey3] = {}
                                            Mplat_ave[ri][ckey3] = {}
                                            chiPlat_bins[ri][ckey3] = {}
                                            chiPlat_ave[ri][ckey3] = {}
                                            optPlatFitFound = False
                                            optPlatFit[ri][ckey3] = None
                                            for nf in range(fitInfo_RP[phTpl][tsnk]['Nfits']):
                                                tstart = fitInfo_RP[phTpl][tsnk]['nf=%d'%(nf)]['tstart']
                                                tstop  = fitInfo_RP[phTpl][tsnk]['nf=%d'%(nf)]['tstop']
                                                Npts   = fitInfo_RP[phTpl][tsnk]['nf=%d'%(nf)]['Npts']

                                                # Doesn't matter for const. fit
                                                xdata = fitInfo_RP[phTpl][tsnk]['nf=%d'%(nf)]['xdata']

                                                Mplat_bins[ri][ckey3][nf] = np.zeros(Nbins,dtype=np.float128)
                                                chiPlat_bins[ri][ckey3][nf] = np.zeros(Nbins,dtype=np.float128)
                                                for nb in range(Nbins):
                                                    # Set fit values and error
                                                    fitVal = RA_bins[ri][ckey3][nb,tstart:tstop+1]
                                                    fitErr = RA_ave[ri][ckey3][1][tstart:tstop+1]

                                                    # Perform plateau fits
                                                    PerformConstFit(ydata,yerr)
                                                    Mplat_bins[ri][ckey3][nf][nb] = PerformConstFit(fitVal,fitErr)
#                                                    Mplat_bins[ri][ckey3][nf][nb], plCov = scipyOpt.curve_fit(constFit,xdata,fitVal,sigma=fitErr)

                                                    # Calculate chi^2
                                                    chiPlat_bins[ri][ckey3][nf][nb] = chiSq_const(xdata, fitVal, fitErr, Mplat_bins[ri][ckey3][nf][nb])
                                                    chiPlat_bins[ri][ckey3][nf][nb] /= (Npts-2)
                                                # for- Nbins

                                                Mplat_ave[ri][ckey3][nf]   = jackknife_ave(Mplat_bins[ri][ckey3][nf],Nbins,tL=1)
                                                chiPlat_ave[ri][ckey3][nf] = jackknife_ave(chiPlat_bins[ri][ckey3][nf],Nbins,tL=1)

                                                if chiPlat_ave[ri][ckey3][nf][0] <= chiCrit and not optPlatFitFound:
                                                    optPlatFit[ri][ckey3] = nf
                                                    optPlatFitFound = True
                                            # for Nfits                                    
                        # if analyze
                        print('Done: ',phMTag, ', Momentum %s'%(mTag), ', t_sink = %d'%(tsnk))
else:
    print('Skipping performing the plateau fits')
                    
print('\nDone\n')

In [ ]:
if plateauFit_Source['Dump']:
    print('Dumping the Plateau fits...')
    pickle.dump(Mplat_bins, open('p_data/Mplat_bins.p', 'wb' ))
    pickle.dump(Mplat_ave, open('p_data/Mplat_ave.p', 'wb' ))
    pickle.dump(chiPlat_bins, open('p_data/chiPlat_bins.p', 'wb' ))
    pickle.dump(chiPlat_ave, open('p_data/chiPlat_ave.p', 'wb' ))
    pickle.dump(optPlatFit, open('p_data/optPlatFit.p', 'wb' ))
elif plateauFit_Source['Load']:
    print('Loading the Plateau fits...')
    Mplat_bins = pickle.load(open('p_data/Mplat_bins.p', 'rb'))
    Mplat_ave = pickle.load(open('p_data/Mplat_ave.p', 'rb'))
    chiPlat_bins = pickle.load(open('p_data/chiPlat_bins.p', 'rb'))
    chiPlat_ave = pickle.load(open('p_data/chiPlat_ave.p', 'rb'))
    optPlatFit = pickle.load(open('p_data/optPlatFit.p', 'rb'))

    Nbins_3pt_Dict = pickle.load( open('p_data/Nbins_3pt_Dict.p', 'rb'))
else:
    print('Skipping dumping/loading of the Plateau fits')


print('\nDone\n')

## Perform/Load summation-method fits on the Ratio

Below we perform 4 types of fits on the summed ratio, $S(t_s) = \sum_{t_{ins}=1}^{t_s-1} \frac{C_{3pt}(t_s,t_{ins})}{C_{2pt}(t_s)}$

(i) Constant fit on the "reduced" summed ratio, $S_r(t_s) = S(t_s+1) - S(t_s)$, keeping and ignoring the lowest $t_s$ point

(ii) Linear fit of the form: $S(t_s) = Z + M t_s$, where $M$ is the matrix element, with $t_s$ starting from $4a,6a,8a,10a$.

(iii) 1- and 2- exponential fits of the form: $S(t_s) = Z + M t_s + At_s e^{-\Delta Et_s} +  Bt_s^2 e^{-\Delta Et_s}$, where $M$ is the matrix element

(iv) 1- and 2-exponential fits on the reduced summed ratio, of the form: $S_r(t_s) = M + Ae^{-\Delta E t_s} + Bt_se^{-\Delta E t_s}$

In [ ]:
summFit_Source = {'Evaluate': False, 'Dump': False, 'Load': True}

# Whether to perform Variable Projection fits
doVarPro = True

# Lowest starting point for linear fits
Lsum_tLow = [4,6,8]

if doVarPro:
    ExpFitTypes = fitTypesVP
else:
    ExpFitTypes = ['E1','E1r']
        
summFitTypes = ['Cr', 'L'] + ExpFitTypes

# Fit parameters of Constant, Linear, 1- and 2-Exponential fits
summFitParam = {'Cr': ['M'],
                'L' : ['M','b'],
                'E1': ['M','Z','A','dE']    , 'E1r': ['M','A','dE'],
                'E2': ['M','Z','A','B','dE'], 'E2r': ['M','A','B','dE']}

print('\nDone\n')

In [ ]:
# Bins, Averages, and Chi^2
summFitBins = {}
summFitAve  = {}
summFitChiAve = {}
for ft in summFitTypes:
    summFitBins[ft]   = {}
    summFitAve[ft]    = {}
    summFitChiAve[ft] = {}
    
# Counter of the 1- and 2-exp successful fits, and list with successful fits
summFitNf  = {}
binListExp = {}
for ft in ExpFitTypes:
    summFitNf[ft] = {}
    binListExp[ft] = {}
        
# Printing tag
prStr = {}

print('\nDone\n')


In [ ]:

if summFit_Source['Evaluate']:

    # 1- and 2-Exponential fits on regular- and reduced-summed ratio
    for ft in ExpFitTypes:
        for fp in summFitParam[ft]:
            summFitBins[ft][fp] = {'Re': {}, 'Im': {}}
            summFitAve[ft][fp]  = {'Re': {}, 'Im': {}}
        for ri in ReIm:
            summFitChiAve[ft][ri] = {}
            summFitNf[ft][ri]  = {}
            binListExp[ft][ri] = {}
    #------------------------

    # Constant fit on reduced-summed ratio keeping lowest (Ml) and ignoring lowest (Mh) point
    ft = 'Cr'
    for rsf in ['l','h']:
        dps = 'M_'+rsf
        summFitBins[ft][dps]   = {'Re': {}, 'Im': {}}
        summFitAve[ft][dps]    = {'Re': {}, 'Im': {}}
        summFitChiAve[ft][rsf] = {'Re': {}, 'Im': {}}
    #------------------------

    # Linear fit on regular-summed ratio
    ft = 'L'
    for sf in Lsum_tLow:
        sfs = 'l%d'%(sf)
        for fp in summFitParam[ft]:
            dps = fp+'_%s'%(sfs)
            summFitBins[ft][dps] = {'Re': {}, 'Im': {}}
            summFitAve[ft][dps]  = {'Re': {}, 'Im': {}}
        summFitChiAve[ft][sfs]   = {'Re': {}, 'Im': {}}
    #------------------------


    for opT in SrcSnkOpTypes:
        for iphT in phaseTypes:            
            for phM in Phases_MomTags_ave[iphT]:
                phMTag = phasedTag(iphT,phM)
                phTpl = (phMTag,opT)

                momTags = momTags_Dict_ave[(phMTag,opT)]

                # Fit ranges for reduced-summation fits
                TsinkList_rs = {'l': TsinkList[(phMTag,opT)][:-1],'h': TsinkList[(phMTag,opT)][1:-1]}
                Nx_rs = {}
                xData_rs = {}
                for rsf in ['l','h']:
                    Nx_rs[rsf] = len(TsinkList_rs[rsf])
                    xData_rs[rsf] = TsinkList_rs[rsf]


                # Fit ranges for linear fits on regular summed ratio fits
                TsinkList_sL = {}
                Nx_sL = {}
                xData_sL = {}
                for isf,sf in enumerate(Lsum_tLow):
                    sfs = 'l%d'%(sf)
                    isf = TsinkList[(phMTag,opT)].index(sf)
                    TsinkList_sL[sfs] = TsinkList[(phMTag,opT)][isf:]
                    Nx_sL[sfs] = len(TsinkList_sL[sfs])
                    xData_sL[sfs] = TsinkList_sL[sfs]


                # Fit range for the exponential fit on regular summed-ratio
                TsinkList_sE = TsinkList[(phMTag,opT)]
                Nx_sE = len(TsinkList_sE)
                xData_sE = TsinkList_sE

                for mTag in momTags:                    
                    mVec = momVector_Dict[mTag]
                    zMom = mVec[2]
                    mFTag = momFileTag(mVec)
                    pmTpl = phMomTpl(phMTag,zMom)

                    srcOpList = srcOpList3pt_Dict[opT][pmTpl] # These are the same in two- and
                    snkOpList = srcOpList3pt_Dict[opT][pmTpl] # three-point functions

                    for isrc, src in enumerate(srcOpList):
                        for isnk, snk in enumerate(snkOpList):
                            dTpl = (phMTag,opT,mTag,tsnk)
                            Nbins = Nbins_3pt_Dict[dTpl]

                            irowSrc = 1
                            irowSnk = 1
                            gMat = 'gt'
                            for z in dispLen_ave:
                                ckeyF = (opT,phMTag,mTag,isrc,isnk,gMat,z,irowSrc,irowSnk)

                                for ri in ReIm:
                                    chiBins = {}

                                    # 1- and 2-exponential fit on regular- and reduced-summed ratio
                                    nf_c = {} # Fit counters
                                    for ft in ExpFitTypes:
                                        for fp in summFitParam[ft]:
                                            summFitBins[ft][fp][ri][ckeyF]  = np.zeros(Nbins,dtype=np.float64)
                                        chiBins[ft] = np.zeros(Nbins,dtype=np.float64)
                                        nf_c[ft] = 0
                                        binListExp[ft][ri][ckeyF] = []
                                    
                                    # Constant fit on reduced-summed ratio
                                    ft = 'Cr'
                                    chiBins[ft] = {}
                                    for rsf in ['l','h']:
                                        dps = 'M_'+rsf
                                        summFitBins[ft][dps][ri][ckeyF] = np.zeros(Nbins,dtype=np.float64)
                                        chiBins[ft][rsf] = np.zeros(Nbins,dtype=np.float64)

                                    # Linear fit on regular summed ratio
                                    ft = 'L'
                                    chiBins['L'] = {}
                                    for sf in Lsum_tLow:
                                        sfs = 'l%d'%(sf)
                                        for fp in summFitParam[ft]:
                                            dps = fp+'_%s'%(sfs)
                                            summFitBins[ft][dps][ri][ckeyF] = np.zeros(Nbins,dtype=np.float64)
                                        chiBins[ft][sfs] = np.zeros(Nbins,dtype=np.float64)                                    
                                    
                                    for nb in range(Nbins):
                                        #----------------------------------
                                        # Fits for the regular-summed ratio
                                        #----------------------------------

                                        # Perform linear fit and calculate chi^2
                                        ft = 'L'
                                        for isf,sf in enumerate(Lsum_tLow):
                                            sfs = 'l%d'%(sf)

                                            sum_fitVal = np.zeros(Nx_sL[sfs],dtype=np.float64)
                                            sum_fitErr = np.zeros(Nx_sL[sfs],dtype=np.float64)
                                            for its,tsnk in enumerate(TsinkList_sL[sfs]):
                                                ckey3 = (opT,phMTag,mTag,tsnk,isrc,isnk,gMat,z,irowSrc,irowSnk) # 3pt-tuple
                                                sum_fitVal[its] = RsumA_bins[ri][ckey3][nb]                                    
                                                sum_fitErr[its] = RsumA_ave[ri][ckey3][1]

                                            fitParamsL, rsLCov = scipyOpt.curve_fit(linearFit,xData_sL[sfs],sum_fitVal,sigma=sum_fitErr)
                                            for ifp,fp in enumerate(summFitParam[ft]):
                                                dps = fp+'_%s'%(sfs)
                                                summFitBins[ft][dps][ri][ckeyF][nb] = fitParamsL[ifp]
                                            chiBins[ft][sfs][nb] = chiSq_linear(xData_sL[sfs], sum_fitVal, sum_fitErr, summFitBins['L']['M'+'_%s'%(sfs)][ri][ckeyF][nb],summFitBins['L']['b'+'_%s'%(sfs)][ri][ckeyF][nb])
                                            chiBins[ft][sfs][nb] /= (Nx_sL[sfs]-len(summFitParam[ft])-1)
                                        #-------------------


                                        # Perform exponential fit and calculate chi^2
                                        sum_fitVal = np.zeros(Nx_sE,dtype=np.float64)
                                        sum_fitErr = np.zeros(Nx_sE,dtype=np.float64)
                                        for its,tsnk in enumerate(TsinkList_sE):
                                            ckey3 = (opT,phMTag,mTag,tsnk,isrc,isnk,gMat,z,irowSrc,irowSnk) # 3pt-tuple
                                            sum_fitVal[its] = RsumA_bins[ri][ckey3][nb]                                    
                                            sum_fitErr[its] = RsumA_ave[ri][ckey3][1]                                   

                                        if doVarPro:
                                            xDataVP   = xData_sE
                                            yDataVP   = sum_fitVal
                                            errDataVP = sum_fitErr
                                            
                                            for ftE in fitTypesVPR:
                                                prStr[ftE] = 'VP'
                                                fitTVP = ftE
    
                                                try:
                                                    resVP = scipyOpt.minimize(chiSq_VP, x0 = [1.0])        
                                                    dE_fit = resVP.x[0] # The resulting dE from the fit                                            
                                                    Cp = np.dot(np.linalg.inv(PhiVPM(dE_fit)),Y(dE_fit)) # The rest of the fit parameters

                                                    summFitBins[ftE]['dE'][ri][ckeyF][nf_c[ftE]] = dE_fit
                                                    for fp in CVP[ftE]:
                                                        summFitBins[ftE][fp][ri][ckeyF][nf_c[ftE]] = Cp[CVP[ftE].index(fp)]

                                                    chiBins[ftE][nf_c[ftE]] = chiSq_VP(dE_fit)
                                                    chiBins[ftE][nf_c[ftE]] /= (Nx_sE-len(summFitParam[ftE]))

                                                    binListExp[ftE][ri][ckeyF].append(nb)
                                                    nf_c[ftE] += 1
                                                except:
                                                    pass
                                        else:
                                            prStr['E1'] = 'SF'
                    
                                            # Construct initial guesses
                                            guess_sE1 = np.zeros(4,dtype=np.float64)
                                            guess_sE1[0] = summFitBins['L']['M_l8'][ri][ckeyF][nb]
                                            fpGuess = summFitParam['E1'][1:]
                                            if nf_c['E1'] == 0:
                                                if z != 0:
                                                    ckeyG = (opT,phMTag,mTag,isrc,isnk,gMat,z-1,irowSrc,irowSnk)
                                                    for ifp,fp in enumerate(fpGuess):
                                                        testG = summFitAve['E1'][fp][ri][ckeyG][0]
                                                        if testG != None:
                                                            guess_sE1[ifp+1] = testG
                                                        else:
                                                            guess_sE1[ifp+1] = 0.1
                                                else:
                                                    if mTag != 'pz.0':
                                                        mTagG = MomTag(mVec[2]-1)
                                                        phMTagG = phMTag
                                                        if mTag == 'pz.+4':
                                                            phMTagG = phasedTag('unphased')
                                                        ckeyG = (opT,phMTagG,mTagG,isrc,isnk,gMat,z,irowSrc,irowSnk)
                                                        for ifp,fp in enumerate(fpGuess):
                                                            testG = summFitAve['E1'][fp][ri][ckeyG][0]
                                                            if testG != None:
                                                                guess_sE1[ifp+1] = testG
                                                            else:
                                                                guess_sE1[ifp+1] = 0.1
                                                    else:
                                                        for ifp,fp in enumerate(fpGuess):
                                                            guess_sE1[ifp+1] = 0.1
                                            else:
                                                for ifp,fp in enumerate(fpGuess):
                                                    guess_sE1[ifp+1] = summFitBins['E1'][fp][ri][ckeyF][nf_c['E1']-1]
                                            #-----------------------------

                                            # Construct fit parameter bounds
                                            minBnd = [None for ifp in range(len(summFitParam['E1']))]
                                            maxBnd = [None for ifp in range(len(summFitParam['E1']))]
                                            Mguess = summFitBins['L']['M_l8'][ri][ckeyF][nb]
                                            for ifp,fp in enumerate(summFitParam['E1']):
                                                minBnd[ifp] = -np.inf
                                                maxBnd[ifp] =  np.inf
                                            minBnd[0] = Mguess - 0.4*np.abs(Mguess)
                                            maxBnd[0] = Mguess + 0.4*np.abs(Mguess)

                                            E1_bnd = (minBnd,maxBnd)

                                            # Perform the fits
                                            try:
                                                if mTag == 'pz.0' and z==0:
                                                    fitParamsE1, sE1Cov = scipyOpt.curve_fit(exp1Fit,xData_sE,sum_fitVal,sigma=sum_fitErr,bounds=E1_bnd,method='trf')#,p0=guess_sE1)
                                                else:
                                                    fitParamsE1, sE1Cov = scipyOpt.curve_fit(exp1Fit,xData_sE,sum_fitVal,sigma=sum_fitErr,p0=guess_sE1)#,bounds=E1_bnd,method='trf')#,p0=guess_sE1)
                                                for ifp,fp in enumerate(summFitParam['E1']):
                                                    summFitBins['E1'][fp][ri][ckeyF][nf_c['E1']] = fitParamsE1[ifp]
                                                chiBins['E1'][nf_c['E1']] = chiSq_exp1(xData_sE, sum_fitVal, sum_fitErr, summFitBins['E1']['M'][ri][ckeyF][nf_c['E1']],summFitBins['E1']['Z'][ri][ckeyF][nf_c['E1']],summFitBins['E1']['A'][ri][ckeyF][nf_c['E1']],summFitBins['E1']['dE'][ri][ckeyF][nf_c['E1']])
                                                chiBins['E1'][nf_c['E1']] /= (Nx_sE-len(summFitParam['E1'])-1)
                                                nf_c['E1'] += 1
                                            except:
                                                pass
                                        # if doVarPro
                                        
                                        #---------------------------------------------
                                        
                                        
                                        #----------------------------------
                                        # Fits for the reduced-summed ratio
                                        #----------------------------------

                                        for rsf in ['h','l']:
                                            dps = 'M_'+rsf
                                            rsum_fitVal = np.zeros(Nx_rs[rsf],dtype=np.float64)
                                            rsum_fitErr = np.zeros(Nx_rs[rsf],dtype=np.float64)
                                            for its,tsnk in enumerate(TsinkList_rs[rsf]):
                                                ckey3 = (opT,phMTag,mTag,tsnk,isrc,isnk,gMat,z,irowSrc,irowSnk) # 3pt-tuple
                                                rsum_fitVal[its] = RrsumA_bins[ri][ckey3][nb]                                    
                                                rsum_fitErr[its] = RrsumA_ave[ri][ckey3][1]
                                                
                                            # Perform constant fit and calculate chi^2
#                                             summFitBins['Cr'][dps][ri][ckeyF][nb] = PerformConstFit(rsum_fitVal,rsum_fitErr)
#                                             chiBins['Cr'][rsf][nb] = chiSq_const(xData_rs[rsf], rsum_fitVal, rsum_fitErr, summFitBins['Cr'][dps][ri][ckeyF][nb])
#                                             chiBins['Cr'][rsf][nb] /= (Nx_rs[rsf]-len(summFitParam['Cr'])-1)


                                            # Perform 1-exponential fit and calculate chi^2
                                            if rsf == 'l':

                                                if doVarPro:                                                    
                                                    xDataVP   = xData_rs[rsf]
                                                    yDataVP   = rsum_fitVal
                                                    errDataVP = rsum_fitErr

                                                    for ftE in fitTypesVPRr:
                                                        prStr[ftE] = 'VP'
                                                        fitTVP = ftE

                                                        try:
                                                            resVP = scipyOpt.minimize(chiSq_VP, x0 = [1.0])        
                                                            dE_fit = resVP.x[0] # The resulting dE from the fit                                            
                                                            Cp = np.dot(np.linalg.inv(PhiVPM(dE_fit)),Y(dE_fit)) # The rest of the fit parameters

                                                            summFitBins[ftE]['dE'][ri][ckeyF][nf_c[ftE]] = dE_fit
                                                            for fp in CVP[ftE]:
                                                                summFitBins[ftE][fp][ri][ckeyF][nf_c[ftE]] = Cp[CVP[ftE].index(fp)]

                                                            chiBins[ftE][nf_c[ftE]] = chiSq_VP(dE_fit)
                                                            chiBins[ftE][nf_c[ftE]] /= (Nx_rs[rsf]-len(summFitParam[ftE]))

                                                            binListExp[ftE][ri][ckeyF].append(nb)
                                                            nf_c[ftE] += 1
                                                        except:
                                                            pass
                                                else:
                                                    prStr['E1r'] = 'SF'
                                                    prStr['E2r'] = 'SF'
                                                    
                                                    # Take care of initial guess
                                                    fpGuess = summFitParam['E1r'][1:]
                                                    guess_rsE1 = np.zeros(3,dtype=np.float64)
                                                    guess_rsE1[0] = summFitBins['E1']['M'][ri][ckeyF][nb]

                                                    # Take special cases into account
                                                    if( (ri == 'Im') and 
                                                        ((mTag == 'pz.+1') and (z==1 or z==2 or z==6 or z==7)) or
                                                        ((mTag == 'pz.+2') and (z==1)) or
                                                        ((mTag == 'pz.+3') and (z==1)) ):
                                                        mTagG0 = MomTag(0)
                                                        zG0 = z
                                                        ckeyG0  = (opT,phMTag,mTagG0,isrc,isnk,gMat,zG0,irowSrc,irowSnk)
                                                        for ifp,fp in enumerate(fpGuess):
                                                            testG0  = summFitAve['E1r'][fp][ri][ckeyG0][0]
                                                            if testG0 != None:
                                                                guess_rsE1[ifp+1] = testG0
                                                            else:
                                                                raise ValueError('Test guess for %s,z%d is None'%(mTag,z))
                                                    else:
                                                        if z != 0:
                                                            zG = z-1
                                                            zG0 = 0
                                                            if mTag != 'pz.0':
                                                                mTagG0 = MomTag(0)
                                                                phMTagG0 = phasedTag('unphased')
                                                                ckeyG   = (opT,phMTag  ,mTag  ,isrc,isnk,gMat,zG ,irowSrc,irowSnk)
                                                                ckeyG0  = (opT,phMTagG0,mTagG0,isrc,isnk,gMat,z  ,irowSrc,irowSnk)
                                                                ckeyG00 = (opT,phMTagG0,mTagG0,isrc,isnk,gMat,zG0,irowSrc,irowSnk)
                                                                for ifp,fp in enumerate(fpGuess):
                                                                    testG   = summFitAve['E1r'][fp][ri][ckeyG][0]
                                                                    testG0  = summFitAve['E1r'][fp][ri][ckeyG0][0]
                                                                    testG00 = summFitAve['E1r'][fp][ri][ckeyG00][0]
                                                                    if testG0 != None:
                                                                        guess_rsE1[ifp+1] = testG0
                                                                    else:
                                                                        if testG00 != None:
                                                                            guess_rsE1[ifp+1] = testG00
                                                                        else:
                                                                            guess_rsE1[ifp+1] = testG
                                                            else:
                                                                ckeyG   = (opT,phMTag,mTag,isrc,isnk,gMat,zG ,irowSrc,irowSnk)
                                                                ckeyG0  = (opT,phMTag,mTag,isrc,isnk,gMat,zG0,irowSrc,irowSnk)
                                                                for ifp,fp in enumerate(fpGuess):
                                                                    testG   = summFitAve['E1r'][fp][ri][ckeyG][0]
                                                                    testG0  = summFitAve['E1r'][fp][ri][ckeyG0][0]
                                                                    if testG != None:
                                                                        guess_rsE1[ifp+1] = testG
                                                                    else:
                                                                        guess_rsE1[ifp+1] = testG0
                                                        # if z != 0
                                                        else:
                                                            if mTag != 'pz.0':
                                                                mTagG = MomTag(0)
                                                                phMTagG = phasedTag('unphased')
                                                                ckeyG = (opT,phMTagG,mTagG,isrc,isnk,gMat,z,irowSrc,irowSnk)
                                                                for ifp,fp in enumerate(fpGuess):
                                                                    testG = summFitAve['E1r'][fp][ri][ckeyG][0]
                                                                    if testG != None:
                                                                        guess_rsE1[ifp+1] = testG
                                                                    else:
                                                                        guess_rsE1[ifp+1] = 0.1
                                                            else:
                                                                guess_rsE1[1] = 0.1 # A
                                                                guess_rsE1[2] = 0.1 # dE
                                                    #-----------------------------

                                                    # Set fit parameter bounds
                                                    minBnd = [None for ifp in range(len(summFitParam['E1r']))]
                                                    maxBnd = [None for ifp in range(len(summFitParam['E1r']))]
                                                    Mguess = summFitBins['L']['M_l8'][ri][ckeyF][nb] #E1sum_bins['M'][ri][ckeyF][nb]
                                                    for ifp,fp in enumerate(summFitParam['E1r']):
                                                        minBnd[ifp] = -np.inf
                                                        maxBnd[ifp] =  np.inf
                                                    minBnd[0] = Mguess - 0.18*np.abs(Mguess)
                                                    maxBnd[0] = Mguess + 0.18*np.abs(Mguess)

                                                    E1r_bnd = (minBnd,maxBnd)

                                                    # Perform the fits
                                                    try:
                                                        fitParamsE1r, sE1rCov = scipyOpt.curve_fit(exp1rFit,xData_rs[rsf],rsum_fitVal,sigma=rsum_fitErr,method='trf',bounds=E1r_bnd)#,p0=guess_rsE1)
                                                        for ifp,fp in enumerate(summFitParam['E1r']):
                                                            summFitBins['E1r'][fp][ri][ckeyF][nf_c['E1r']] = fitParamsE1r[ifp]
                                                        chiBins['E1r'][nf_c['E1r']] = chiSq_exp1r(xData_rs[rsf], rsum_fitVal, rsum_fitErr, summFitBins['E1r']['M'][ri][ckeyF][nf_c['E1r']],summFitBins['E1r']['A'][ri][ckeyF][nf_c['E1r']],summFitBins['E1r']['dE'][ri][ckeyF][nf_c['E1r']])
                                                        chiBins['E1r'][nf_c['E1r']] /= (Nx_rs[rsf]-len(summFitParam['E1r'])-1)
                                                        nf_c['E1r'] += 1
                                                    except:
                                                        pass
                                                # if doVarPro
                                        # for low-high
                                    # for-Nbins ---------------------------------------


                                    #-------------
                                    # Fit averages
                                    #-------------

                                    # Reduced summed-ratio Constant fit averages
                                    ft = 'Cr'
                                    for rsf in ['l','h']:
                                        dps = 'M_'+rsf
                                        summFitAve[ft][dps][ri][ckeyF] = jackknife_ave(summFitBins[ft][dps][ri][ckeyF],Nbins,tL=1)
                                        summFitChiAve[ft][rsf][ri][ckeyF] = jackknife_ave(chiBins[ft][rsf],Nbins,tL=1)
                                    #----------------------

                                    # Summed-ratio Linear fit averages
                                    ft = 'L'
                                    for isf,sf in enumerate(Lsum_tLow):
                                        sfs = 'l%d'%(sf)

                                        for fp in summFitParam['L']:
                                            dps = fp+'_%s'%(sfs)                                    
                                            summFitAve[ft][dps][ri][ckeyF] = jackknife_ave(summFitBins[ft][dps][ri][ckeyF],Nbins,tL=1)
                                        summFitChiAve[ft][sfs][ri][ckeyF] = jackknife_ave(chiBins[ft][sfs],Nbins,tL=1)
                                    #----------------------


                                    # 1-exponential fit averages
                                    for ft in ExpFitTypes:
                                        summFitNf[ft][ri][ckeyF] = nf_c[ft]
                                        
                                        # Slice fit bins for parameters and chi^2
                                        for fp in summFitParam[ft]:
                                            summFitBins[ft][fp][ri][ckeyF] = summFitBins[ft][fp][ri][ckeyF][:nf_c[ft]]
                                        chiBins[ft] = chiBins[ft][:nf_c[ft]]

                                        # Compute averages
                                        if nf_c[ft] == 0: # No successful fits
                                            for fp in summFitParam[ft]:
                                                 summFitAve[ft][fp][ri][ckeyF] = (None,None)
                                            summFitChiAve[ft][ri][ckeyF] = (None,None)
                                        elif nf_c[ft] == 1: # One successful fit
                                            for fp in summFitParam[ft]:
                                                 summFitAve[ft][fp][ri][ckeyF] = (summFitBins[ft][fp][ri][ckeyF][0],0.0)
                                            summFitChiAve[ft][ri][ckeyF] = (chiBins[ft][0], 0.0)
                                        else: # More than one successful fits
                                            for fp in summFitParam[ft]:
                                                summFitAve[ft][fp][ri][ckeyF] = jackknife_ave(summFitBins[ft][fp][ri][ckeyF],nf_c[ft],tL=1)
                                            summFitChiAve[ft][ri][ckeyF] = jackknife_ave(chiBins[ft],nf_c[ft],tL=1)
                                        #-----------------
                                        
                                        if not (ri=='Im' and z*mVec[2]==0):
                                            print( '%s %s-S: z = %d: nf_c[%s]  = %d: '%(prStr[ft],ri,z,ft,nf_c[ft]), summFitAve[ft]['M'][ri][ckeyF] )
                                    #------------------------------------------

                    print('Done: ',phMTag, ', Momentum %s'%(mTag))
else:
    print('Skipping performing the summation method fits')
                
print('\nDone\n')

In [ ]:
if summFit_Source['Dump']:
    print('Dumping the summation method fits...')
    
    for ft in summFitTypes:
        pickle.dump(summFitBins[ft]  , open('p_data/%ssum_bins.p'%(ft)  , 'wb' ))
        pickle.dump(summFitAve[ft]   , open('p_data/%ssum_ave.p'%(ft)   , 'wb' ))
        pickle.dump(summFitChiAve[ft], open('p_data/%ssum_chiAve.p'%(ft), 'wb' ))

    for ft in ExpFitTypes:
        pickle.dump(summFitNf[ft],  open('p_data/%ssummFitNf.p'%(ft), 'wb' ))
        pickle.dump(binListExp[ft], open('p_data/%s_binListExp.p'%(ft), 'wb' ))
        
        
elif summFit_Source['Load']:
    print('Loading the summation method fits...')
    
    for ft in summFitTypes:
        summFitBins[ft]   = pickle.load(open('p_data/%ssum_bins.p'%(ft)  , 'rb'))
        summFitAve[ft]    = pickle.load(open('p_data/%ssum_ave.p'%(ft)   , 'rb'))
        summFitChiAve[ft] = pickle.load(open('p_data/%ssum_chiAve.p'%(ft), 'rb'))

    for ft in ExpFitTypes:
        summFitNf[ft]  = pickle.load(open('p_data/%ssummFitNf.p'%(ft), 'rb'))
#        binListExp[ft] = pickle.load(open('p_data/%s_binListExp.p'%(ft), 'wb' ))
        

    Nbins_3pt_Dict = pickle.load( open('p_data/Nbins_3pt_Dict.p', 'rb'))
else:
    print('Skipping dumping/loading of the summation method fits')


print('\nDone\n')


### Create error bands for summation method fits

In [ ]:
summBands_Source = {'Evaluate': False, 'Dump': False, 'Load': True}

summBandTypes = ['L'] + ExpFitTypes

summBand = {}
for bt in summBandTypes:
    summBand[bt] = {}
    
FitModel = None


In [ ]:

if summBands_Source['Evaluate']:

    Nbpts = 251

    bt = 'L'
    for sf in Lsum_tLow:
        sfs = 'l%d'%(sf)
        summBand[bt][sfs] = {'Re':{}, 'Im':{}}

    for bt in ExpFitTypes:
        for ri in ReIm:
            summBand[bt][ri] = {}

    xStart = {}
    xEnd = {}
    dx = {}
    for bt in summBandTypes:
        xStart[bt] = {}
        xEnd[bt] = {}
        dx[bt] = {}

    for opT in SrcSnkOpTypes:
        for iphT in phaseTypes:            
            for phM in Phases_MomTags_ave[iphT]:
                phMTag = phasedTag(iphT,phM)
                phTpl = (phMTag,opT)

                momTags = momTags_Dict_ave[(phMTag,opT)]

                # Fit ranges for reduced-summation fits
                TsinkList_rs = {'l': TsinkList[(phMTag,opT)][:-1],'h': TsinkList[(phMTag,opT)][1:-1]}

                # Fit ranges for linear fits on regular summed ratio fits
                TsinkList_sL = {}
                for isf,sf in enumerate(Lsum_tLow):
                    sfs = 'l%d'%(sf)
                    isf = TsinkList[(phMTag,opT)].index(sf)
                    TsinkList_sL[sfs] = TsinkList[(phMTag,opT)][isf:]

                # Fit range for the exponential fits on regular summed-ratio
                TsinkList_sE = TsinkList[(phMTag,opT)]


                # Summed-ratio linear fit ranges
                bt = 'L'
                xStart[bt] = {}
                dx[bt] = {}
                for sf in Lsum_tLow:
                    sfs = 'l%d'%(sf)
                    xStart[bt][sfs] = TsinkList_sL[sfs][0]
                    xEnd[bt]        = TsinkList_sL[sfs][-1]+1
                    dx[bt][sfs]     = (xEnd[bt] - xStart[bt][sfs])  / (Nbpts-1)

                    
                # Summed-ratio exp-fit ranges
                for bt in fitTypesVPR:
                    xStart[bt] = TsinkList[(phMTag,opT)][0]-1
                    xEnd[bt]   = TsinkList[(phMTag,opT)][-1]+1
                    dx[bt]     = (xEnd[bt] - xStart[bt])  / (Nbpts-1)


                # Reduced-summed ratio exp-fit ranges
                for bt in fitTypesVPRr:
                    xStart[bt] = TsinkList_rs['l'][0]-1
                    xEnd[bt]   = TsinkList_rs['l'][-1]+1
                    dx[bt] = (xEnd[bt] - xStart[bt]) / (Nbpts-1)

                for mTag in momTags:
                    mVec = momVector_Dict[mTag]
                    zMom = mVec[2]
                    mFTag = momFileTag(mVec)
                    pmTpl = phMomTpl(phMTag,zMom)

                    srcOpList = srcOpList3pt_Dict[opT][pmTpl] # These are the same in two- and
                    snkOpList = srcOpList3pt_Dict[opT][pmTpl] # three-point functions

                    for isrc, src in enumerate(srcOpList):
                        for isnk, snk in enumerate(snkOpList):
                            dTpl = (phMTag,opT,mTag,tsnk)
                            Nbins = Nbins_3pt_Dict[dTpl]

                            irowSrc = 1
                            irowSnk = 1
                            gMat = 'gt'
                            for z in dispLen_ave:
                                ckeyF = (opT,phMTag,mTag,isrc,isnk,gMat,z,irowSrc,irowSnk)

                                for ri in ReIm:
                                    for sf in Lsum_tLow:
                                        sfs = 'l%d'%(sf)
                                        summBand['L'][sfs][ri][ckeyF]   = {'x'  : np.zeros(Nbpts),'v': np.zeros(Nbpts),'e': np.zeros(Nbpts)}

                                    for bt in ExpFitTypes:
                                        summBand[bt][ri][ckeyF] = {'x'  : np.zeros(Nbpts),'v': np.zeros(Nbpts),'e': np.zeros(Nbpts)}

                                    for ibp in range(Nbpts):

                                        # Linear fit
                                        bt = 'L'
                                        for sf in Lsum_tLow:
                                            sfs = 'l%d'%(sf)

                                            xi_L  = xStart[bt][sfs]  + ibp*dx[bt][sfs]

                                            dps = 'M_%s'%(sfs)
                                            if len(summFitBins[bt][dps][ri][ckeyF]) != Nbins:
                                                raise ValueError('Incorrect size of bins for linear fit %s, ckey '%(sfs),ckeyF)


                                            summBand[bt][sfs][ri][ckeyF]['x'][ibp] = xi_L
                                            summBand[bt][sfs][ri][ckeyF]['v'][ibp] = linearFit(xi_L, summFitAve[bt]['M_%s'%(sfs)][ri][ckeyF][0], summFitAve[bt]['b_%s'%(sfs)][ri][ckeyF][0])

                                            bandVarL  = np.zeros(Nbins)
                                            for nb in range(Nbins):
                                                bandVarL[nb]  = linearFit(xi_L, summFitBins[bt]['M_%s'%(sfs)][ri][ckeyF][nb], summFitBins[bt]['b_%s'%(sfs)][ri][ckeyF][nb])
                                            summBand[bt][sfs][ri][ckeyF]['e'][ibp]  = jackknife_ave(bandVarL ,Nbins,tL=1)[1]
                                        # for linear fit 4,6,....


                                        # Exponential fits
                                        for bt in ExpFitTypes:
                                            xi_E = xStart[bt] + ibp*dx[bt]

                                            nfE_c = summFitNf[bt][ri][ckeyF]
                                            
                                            summBand[bt][ri][ckeyF]['x'][ibp] = xi_E
                                            
                                            if nfE_c == 0:
                                                summBand[bt][ri][ckeyF]['v'][ibp] = None
                                                summBand[bt][ri][ckeyF]['e'][ibp] = None
                                            else:
                                                # Fit value
                                                Nprm = len(summFitParam[bt])
                                                Cp = np.zeros(Nprm)
                                                
                                                if summFitParam[bt].index('dE') != Nprm-1:
                                                    raise ValueError('dE is NOT the last parameter for %s!'%(bt))

                                                for ifp,fp in enumerate(CVP[bt]):
                                                    Cp[ifp] = summFitAve[bt][fp][ri][ckeyF][0]
                                                Cp[-1] = summFitAve[bt]['dE'][ri][ckeyF][0]
                                                
                                                summBand[bt][ri][ckeyF]['v'][ibp] = expModel(bt, xi_E, Cp)
                                                    
                                                # FIt error
                                                if nfE_c == 1:
                                                    summBand[bt][ri][ckeyF]['e'][ibp] = 0.0
                                                else:
                                                    bandVarE = np.zeros(nfE_c)
                                                    for nb in range(nfE_c):
                                                        for ifp,fp in enumerate(CVP[bt]):
                                                            Cp[ifp] = summFitBins[bt][fp][ri][ckeyF][nb]
                                                        Cp[-1] = summFitBins[bt]['dE'][ri][ckeyF][nb]
                                                        
                                                        bandVarE[nb] = expModel(bt, xi_E, Cp)

                                                    summBand[bt][ri][ckeyF]['e'][ibp] = jackknife_ave(bandVarE,nfE_c,tL=1)[1]
                                        #--------------------------------------------------------

                    print('Done: ',phMTag, ', Momentum %s'%(mTag))
else:
    print('Skipping evaluating the summation method fit bands')
                

print('\nDone\n')


In [ ]:
if summBands_Source['Dump']:
    print('Dumping the summation method fit bands...')
    for bt in summBandTypes:
        pickle.dump(summBand[bt], open('p_data/%ssum_band.p'%(bt), 'wb' ))     
elif summBands_Source['Load']:
    print('Loading the summation method fit bands...')
    for bt in summBandTypes:
        summBand[bt] = pickle.load(open('p_data/%ssum_band.p'%(bt), 'rb'))        
else:
    print('Skipping dumping/loading of the summation method fit bands')


print('\nDone\n')


### Plot summed ratio $R(t_s)$

In [ ]:
# Whether to plot the exp-fit band
plotExpBand = {'E1' : True, 'E2':  False,
               'E1r': True, 'E2r': False}

expBandLabel = {'E1' : 'summ. 1-exp fit'  , 'E2' : 'summ. 2-exp fit',
                'E1r': 'r-summ. 1-exp fit', 'E2r': 'r-summ. 2-exp fit'}

expBandCol = {'E1' : 'slategray'  , 'E2' : 'magenta',
              'E1r': 'orange', 'E2r': 'tomato'}


In [ ]:
figRS, axRS = plt.subplots(2, 2, sharex='col', figsize=[15,9]) #, sharey='row'
figRS.subplots_adjust(hspace=0.00,wspace=0.2)#,right=0.8)


# Examples of fits:
# Pz = 2, z = 1
# Pz = 2, z = 2
# Pz = 2, z = 3
# Pz = 1, z = 1
# Pz = 1, z = 2
# Pz = 1, z = 3
# Pz = 1, z = 5
# Pz = 1, z = 7



opTRS = 'single'
isrcRS = 0
isnkRS = 0
irowSrcRS = 1
irowSnkRS = 1

gMatRS = 'gt'

phTRS = 'unphased'
#phTRS = 'phased-d001_2.00'
mTagRS = 'pz.+1'
mVecRS = momVector_Dict[mTagRS]

dL = 1


Ntsink = len(TsinkList[(phTRS,opTRS)])
for iri,ri in enumerate(ReIm):
    tTsinkAnalyze = []    
    RsumA_plot = (np.zeros(Ntsink),np.zeros(Ntsink))
    for its, tsink in enumerate(TsinkList[(phTRS,opTRS)]):
        ckey3 = (opTRS,phTRS,mTagRS,tsink,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)

        adTplM = (phTRS,opTRS,mTagRS,tsink)
        if analyzeDataSetM[adTplM]:
            tTsinkAnalyze.append(tsink)
            for ive in range(2):
                RsumA_plot[ive][its] = RsumA_ave[ri][ckey3][ive]
    #----

    xRng = np.array(tTsinkAnalyze)
    
    # Plot summed-ratio
    axRS[iri][0].errorbar(xRng+osDctF[0],  RsumA_plot[0], yerr = RsumA_plot[1],
                 color = cDct[0], marker = mDct[0], ls = 'none',
                 ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full')
    
    
    # Plot summed-ratio linear fit band
    ckeyF = (opTRS,phTRS,mTagRS,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)
    ft = 'L'
    
    for isf,sf in enumerate(Lsum_tLow):
        sfs = 'l%s'%(sf)
        if ri == 'Re':
            lfLbl = r'linear fit $t_s^{\rm low} = %d$'%(sf)
        else:
            lfLbl = 'none'
        axRS[iri][0].plot(summBand[ft][sfs][ri][ckeyF]['x'], summBand[ft][sfs][ri][ckeyF]['v'], ls='-',color=cDct[isf],linewidth=0.8)
        axRS[iri][0].fill_between(summBand[ft][sfs][ri][ckeyF]['x'], summBand[ft][sfs][ri][ckeyF]['v']-summBand[ft][sfs][ri][ckeyF]['e'], summBand[ft][sfs][ri][ckeyF]['v']+summBand[ft][sfs][ri][ckeyF]['e'], facecolor=cDct[isf], alpha=0.20,label=lfLbl)
    #-----------------------------
    
    
    # Plot summed-ratio exponential fit band(s)
    ckeyF = (opTRS,phTRS,mTagRS,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)
    for ft in fitTypesVPR:
        if plotExpBand[ft]:    
            if ri == 'Re':
                efLbl = expBandLabel[ft]
            else:
                efLbl = 'none'
            if not np.isnan(summBand[ft][ri][ckeyF]['v']).any():
                axRS[iri][0].plot(summBand[ft][ri][ckeyF]['x'], summBand[ft][ri][ckeyF]['v'], ls='-', color=expBandCol[ft],linewidth=0.8)
                axRS[iri][0].fill_between(summBand[ft][ri][ckeyF]['x'], summBand[ft][ri][ckeyF]['v']-summBand[ft][ri][ckeyF]['e'], summBand[ft][ri][ckeyF]['v']+summBand[ft][ri][ckeyF]['e'], facecolor=expBandCol[ft], alpha=0.20,label=efLbl)
            else:
                print('%s: Skipping Part = %s , Mom = %s , z3 = %d'%(plotLabel[ft],ri,mTagRS,dL))
        
#----------------------------------------------------------------


# Plot reduced summed-ratio
Ntsink = len(TsinkList[(phTRS,opTRS)])

TsinkList_rs = {'l': TsinkList[(phTRS,opTRS)][:-1],'h': TsinkList[(phTRS,opTRS)][1:-1]}
TsinkList_plot = TsinkList[(phTRS,opTRS)][:-1]

for iri,ri in enumerate(ReIm):
    tTsinkAnalyze = []
    RrsumA_plot = (np.zeros(Ntsink-1),np.zeros(Ntsink-1))
    for its, tsink in enumerate(TsinkList_plot):
        ckey3 = (opTRS,phTRS,mTagRS,tsink,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)

        adTplM = (phTRS,opTRS,mTagRS,tsink)
        if analyzeDataSetM[adTplM]:
            tTsinkAnalyze.append(tsink)
            for ive in range(2):
                RrsumA_plot[ive][its] = RrsumA_ave[ri][ckey3][ive]
    #--------------------------

    xRng = np.array(tTsinkAnalyze)

    axRS[iri][1].errorbar(xRng+osDct[0],  RrsumA_plot[0], yerr = RrsumA_plot[1],
                 color = cDct[0], marker = mDct[0], ls = 'none',
                 ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full')
    
    
    # Plot regular-ratio linear fit slope
    ckeyF = (opTRS,phTRS,mTagRS,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)
    ft = 'L'
    
    for isf,sf in enumerate(Lsum_tLow):
        sfs = 'l%s'%(sf)

        sumLVal = summFitAve[ft]['M_%s'%(sfs)][ri][ckeyF]

        xLo = 2+0.3*isf
        xHi = 14-0.3*isf
        xSt = (xHi-xLo)/10
        sumLX = np.arange(xLo,14+0.01,xSt)
        sumLY = np.array([sumLVal[0] for i in range(len(sumLX))])
        sumLYerr = np.array([sumLVal[1] for i in range(len(sumLX))])

        print('sum-L value-%s %s = '%(sfs,ri), sumLVal)

        if ri == 'Re':
            lfLbl = r'summ. linear fit slope $t_s^{\rm low} = %d$'%(sf)
        else:
            lfLbl = 'none'
        axRS[iri][1].plot(sumLX, sumLY, ls='-',color=cDct[isf],linewidth=1.0)
        axRS[iri][1].fill_between( sumLX , sumLY-sumLYerr, sumLY+sumLYerr, facecolor=cDct[isf], alpha=0.20)#,label=lfLbl)  
    #-----------------------------

    
    # Plot regular-ratio 1-exp fit matrix element
    ckeyF = (opTRS,phTRS,mTagRS,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)
    for ft in fitTypesVPR:
        if plotExpBand[ft]:
    
            sumEVal = summFitAve[ft]['M'][ri][ckeyF]
            if sumEVal[0] != None:
                if not np.isnan(sumEVal[0]):

                    sumEX = np.arange(2,15)
                    sumEY = np.array([sumEVal[0] for i in range(len(sumEX))])
                    sumEYerr = np.array([sumEVal[1] for i in range(len(sumEX))])

                    print('%s value %s = '%(expBandLabel[ft],ri), sumEVal)

                    axRS[iri][1].plot(sumEX, sumEY, ls='-', color=expBandCol[ft],linewidth=1.0)
                    axRS[iri][1].fill_between( sumEX , sumEY-sumEYerr, sumEY+sumEYerr, facecolor=expBandCol[ft], alpha=0.25)#,label=efLbl)
    #-----------------------------

    
    # Plot reduced-summed ratio 1-exp fit matrix element
    ckeyF = (opTRS,phTRS,mTagRS,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)
    for ft in fitTypesVPRr:
        if plotExpBand[ft]:

            rsumEVal = summFitAve[ft]['M'][ri][ckeyF]
            if rsumEVal[0] != None:

                rsumEX = xRng
                rsumEY = np.array([rsumEVal[0] for i in range(len(rsumEX))])
                rsumEYerr = np.array([rsumEVal[1] for i in range(len(rsumEX))])

                if ri == 'Re':
                    refLbl = 'r-summ. 2-exp fit M'
                else:
                    refLbl = 'none'
                axRS[iri][1].plot(rsumEX, rsumEY, color=expBandCol[ft], ls='-',linewidth=1.0)
                axRS[iri][1].fill_between( rsumEX , rsumEY-rsumEYerr, rsumEY+rsumEYerr, facecolor=expBandCol[ft], alpha=0.20,label=refLbl)
    #-----------------------------
    
    
    # Plot reduced-summed ratio 1-exponential fit band
    ckeyF = (opTRS,phTRS,mTagRS,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)
    ft = 'E2r'
    
    if ri == 'Re':
        refLbl = 'r-summ. 2-exp fit'
    else:
        refLbl = 'none'
        
    if not np.isnan(summBand[ft][ri][ckeyF]['v']).any():
        axRS[iri][1].plot(summBand[ft][ri][ckeyF]['x'], summBand[ft][ri][ckeyF]['v'], color='saddlebrown', ls='-',linewidth=0.8)
        axRS[iri][1].fill_between(summBand[ft][ri][ckeyF]['x'], summBand[ft][ri][ckeyF]['v']-summBand[ft][ri][ckeyF]['e'], summBand[ft][ri][ckeyF]['v']+summBand[ft][ri][ckeyF]['e'], facecolor='saddlebrown', alpha=0.12,label=refLbl)
        rsumEVal = summFitAve[ft]['M'][ri][ckeyF]
        print('rsum-E2 value %s = '%(ri), rsumEVal)
    else:
        print('Skipping Part = %s , Mom = %s , z3 = %d'%(ri,mTagRS,dL))
    #-----------------------------
    
#-------------------------------------------------------------------------------


for iri,ri in enumerate(ReIm):
    axRS[iri][0].set_ylabel(r'%s[$R_{\rm sum}(p_z = %s;z_3 = %s)$]'%(ri,mVecRS[2],dL),fontsize=18)
    axRS[iri][1].set_ylabel(r'%s[$R_{\rm r-sum}(p_z = %s;z_3 = %s)$]'%(ri,mVecRS[2],dL),fontsize=18)

    axRS[iri][0].set_xlim(2,16)
    axRS[iri][0].xaxis.set_ticks(np.arange(2, 17, 2))
    axRS[iri][1].set_xlim(2,14)
    axRS[iri][1].xaxis.set_ticks(np.arange(2, 15, 2))

    
# axRS[0][1].set_ylim(-0.05,0.45)
# axRS[1][1].set_ylim(-0.5,0.5)

    
    
axRS[1][0].set_xlabel(r'$t_s/a$',fontsize=18)
axRS[1][1].set_xlabel(r'$t_s/a$',fontsize=18)

axRS[0][0].legend(loc='best', ncol=1, numpoints=1, fontsize=15,
           labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
           markerscale=0.8)

axRS[0][1].legend(loc='best', ncol=2, numpoints=1, fontsize=15,
           labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
           markerscale=0.8)


figRS.savefig('plots/summed_ratio_fits_%s_z%d_VP.pdf'%(mTagRS,dL),bbox_inches='tight')



## Perform 2-state fits on the Ratio

In [ ]:
twoStFit_Source = {'Evaluate': False, 'Dump': False, 'Load': True}

fitParam2stOpt = ['M','A01','A11','c1','dE']
#fitParam2stOpt = ['M','A01','A11','dE']
Nfprm = len(fitParam2stOpt)

# Define lowest tsink for the fit
twoSt_tLow_Gen = [4,6,8]
twoSt_tLow = {'pz.0' : [4,6,8],
              'pz.+1': [4,6,8],
              'pz.+2': [4,6,8],
              'pz.+3': [4,6,8],
              'pz.+4': [4,6,8],
              'pz.+5': [4,6],
              'pz.+6': [4,6]}

# Define highest tsink for the fit
twoSt_tHi = {}
for mTag in allMomenta:
    mVec = momVector_Dict[mTag]
    if mVec[2] <= 2:
        for z in dispLen_ave:
            twoSt_tHi[(mTag,z)] = 14
    elif mVec[2] == 3:
        for z in dispLen_ave:
            twoSt_tHi[(mTag,z)] = 12 # if z <= 4 else 14
    elif mVec[2] == 4:
        for z in dispLen_ave:
            twoSt_tHi[(mTag,z)] = 10 if z <= 5 else 12
    elif mVec[2] == 5:
        for z in dispLen_ave:
            twoSt_tHi[(mTag,z)] = 10 if z <= 3 else 12
    elif mVec[2] == 6:
        for z in dispLen_ave:
            twoSt_tHi[(mTag,z)] = 10 #if z <= 4 else 12

#print(twoSt_tHi)

#Order of running tLow
revTs = False
twoSt_Iter = {}
for iphT in phaseTypes:            
    for phM in Phases_MomTags_ave[iphT]:
        phMTag = phasedTag(iphT,phM)
        momTags = momTags_Dict_ave[(phMTag,opT)]

        for mTag in momTags:
            if revTs:
                twoSt_Iter[mTag] = twoSt_tLow[mTag][::-1]
            else:
                twoSt_Iter[mTag] = twoSt_tLow[mTag]

print('\nDone\n')


In [ ]:
M2stOpt_bins = {}
M2stOpt_ave  = {}
fit2stOpt_bins = {}
fit2stOpt_ave = {}
fit2stOpt_chiAve = {}

binList2st = {}

for tsT in twoSt_tLow_Gen:
    itsT = None
    tsT_s = 'ts%d'%(tsT)
    
    binList2st[tsT_s] = {'Re': {}, 'Im': {}}

    fit2stOpt_chiAve[tsT_s] = {'Re': {}, 'Im': {}}

    fit2stOpt_bins[tsT_s] = {}
    fit2stOpt_ave[tsT_s]  = {}
    for fp in fitParam2stOpt:
        fit2stOpt_bins[tsT_s][fp] = {'Re': {}, 'Im': {}}
        fit2stOpt_ave[tsT_s][fp]  = {'Re': {}, 'Im': {}}

    M2stOpt_bins[tsT_s] = {'Re': {}, 'Im': {}}
    M2stOpt_ave[tsT_s]  = {'Re': {}, 'Im': {}}
    
print('\nDone\n')


In [ ]:
# Whether to perform the two-state fit for the given momentum, z3 and tsLow

doAllTwoStFits = False
doStandardTwoStFits = False

doNonStrdTwoStFits = True

# 1st
# Mg_varFct = 0.5
# Pg_varFct = 0.7
# 2nd
# Mg_varFct = 0.25
# Pg_varFct = 0.9
# 3rd
# Mg_varFct = 0.1
# Pg_varFct = 0.9
# 4th
Mg_varFct = 0.02
Pg_varFct = 0.75

doTwoStateFitGen = {}
doTwoStateFit = {}
doStrdTwoStFit = {}

for opT in SrcSnkOpTypes:
    for iphT in phaseTypes:            
        for phM in Phases_MomTags_ave[iphT]:
            phMTag = phasedTag(iphT,phM)
            momTags = momTags_Dict_ave[(phMTag,opT)]

            for mTag in momTags:
                for tsT in twoSt_tLow[mTag]:
                    for z in dispLen_ave:
                        for ri in ReIm:
                            twoStTpl = (opT,phMTag,mTag,z,tsT,ri)
                            doTwoStateFit[twoStTpl]  = (doAllTwoStFits or doStandardTwoStFits)
                            doStrdTwoStFit[twoStTpl] = doStandardTwoStFits
                            doTwoStateFitGen[twoStTpl] = True


# Take care of non-standard fits
opT = 'single'

ri = 'Re'


RedoFitsPhase = 2

NonStrdFitTplList = {}
if RedoFitsPhase == 1: # 1st pass on non-standard fits without using VarPro    
    doVarPro_2st = False
    
    NonStrdFitTplList['Re'] = [('pz.0' ,[1],[4,6,8]),
                               ('pz.+1',[0],[4,6,8]),('pz.+1',[1],[4,6]),
                               ('pz.+2',[0],[6,8]),('pz.+2',[2],[8]),
                               ('pz.+3',[0],[4,6]),('pz.+3',[1],[8]),('pz.+3',[4,5,6],[4,6]),('pz.+3',[7],[4]),('pz.+3',[8],[8]),
                               ('pz.+5',[0],[6]),
                               ('pz.+6',[0,1,2,3,4,5,6,7,8],[4,6])]


    NonStrdFitTplList['Im'] = [('pz.+1',[3,4],[6]),('pz.+1',[5],[4,6]),
                               ('pz.+2',[1,2],[8]),('pz.+2',[4,6,7],[6]),
                               ('pz.+3',[5,7,8],[6]),
                               ('pz.+4',[7,8],[4,6,8]),
                               ('pz.+5',[1,2,3,4,5,6,7,8],[4,6]),
                               ('pz.+6',[1,2,3,4,5,6,7],[4]),('pz.+6',[8],[6])]
elif RedoFitsPhase == 2: # 2nd pass on failed fits, use VarPro
    doVarPro_2st = False
    
    NonStrdFitTplList['Re'] = [('pz.0' ,[1],[4,6,8]),
                               ('pz.+1',[0],[4,6,8]),('pz.+1',[1],[4,6]),
                               ('pz.+5',[0],[6])]

    NonStrdFitTplList['Im'] = []
#-------------------------------------------------------------------------------------



for ri in ReIm:
    for mTag,zList,tsList in NonStrdFitTplList[ri]:
        mVec = momVector_Dict[mTag]
        if mVec[2] < 4:
            phMTag = phasedTag('unphased')
        else:
            phMTag = phasedTag('phased','d001_2.00')
            
        for z3 in zList:
            for tsT in tsList:
                doTwoStateFit[(opT,phMTag,mTag,z3,tsT,ri)] = (doAllTwoStFits or doNonStrdTwoStFits)
                doStrdTwoStFit[(opT,phMTag,mTag,z3,tsT,ri)] = (not doNonStrdTwoStFits)

    
print('\nDone\n')


In [ ]:

if twoStFit_Source['Evaluate']:

    for opT in SrcSnkOpTypes:
        for iphT in phaseTypes:            
            for phM in Phases_MomTags_ave[iphT]:
                phMTag = phasedTag(iphT,phM)
                phTpl = (phMTag,opT)
                phT = None

                momTags = momTags_Dict_ave[(phMTag,opT)]

                for mTag in momTags:
                    mVec = momVector_Dict[mTag]
                    
                    for tsT in twoSt_Iter[mTag]:
                        tsT_s = 'ts%d'%(tsT)
                        mSkip10 = None
                        
                        zMom = mVec[2]
                        mFTag = momFileTag(mVec)
                        pmTpl = phMomTpl(phMTag,zMom)
                       
                        srcOpList = srcOpList3pt_Dict[opT][pmTpl] # These are the same in two- and
                        snkOpList = srcOpList3pt_Dict[opT][pmTpl] # three-point functions

                        for isrc, src in enumerate(srcOpList):
                            for isnk, snk in enumerate(snkOpList):  
                                
                                # Only upolarized PDF for now
                                irowSrc = 1
                                irowSnk = 1
                                gMat = 'gt'

                                for z in dispLen_ave:
                                    # Determine the number of fit points
                                    tsLow = TsinkList[(phMTag,opT)].index(tsT)
                                    tsHi  = TsinkList[(phMTag,opT)].index(twoSt_tHi[(mTag,z)])
                                    Tsink_2st = TsinkList[(phMTag,opT)][tsLow:tsHi+1]

                                    Nfpts = 0
                                    for tsnk in Tsink_2st:
                                        Nfpts += (tsnk-1) # (Exclude source-point)
                                    if z == dispLen_ave[0]:
                                        print(Tsink_2st, 'Nfpts = ',Nfpts)

                                    # For Variable Projection
                                    tsList2st_VP = Tsink_2st
                                    
                                    # Make sure number of bins is the same for all tsink
                                    dTpl = (phMTag,opT,mTag,Tsink_2st[0])
                                    Nbins = Nbins_3pt_Dict[dTpl]
                                    for tsnk in Tsink_2st[1:]:
                                        dTpl = (phMTag,opT,mTag,tsnk)
                                        if  Nbins_3pt_Dict[dTpl] != Nbins:
                                            raise ValueError('Inconsistency in Nbins for tsnk = ',tsnk)          
                                    
                                    for ri in ReIm:
                                        ckeyDoFit = (opT,phMTag,mTag,z,tsT,ri)
                                        ckeyF = (opT,phMTag,mTag,isrc,isnk,gMat,z,irowSrc,irowSnk) # 3pt-tuple
                                        if( doTwoStateFitGen[ckeyDoFit] and
                                           (doTwoStateFit[ckeyDoFit] or doAllTwoStFits) and
                                           (not(z*mVec[2]==0 and ri=='Im')) ):
                                            
#                                             if doStrdTwoStFit[ckeyDoFit]:
#                                                 print('Standard fit:')
#                                             else:
#                                                 if doVarPro_2st:
#                                                     print('VarPro')
#                                                 else:
#                                                     print('Non-Standard fit:')                                            
                                            
                                            M2stOpt_bins[tsT_s][ri][ckeyF] = np.zeros(Nbins)

                                            for fp in fitParam2stOpt:
                                                fit2stOpt_bins[tsT_s][fp][ri][ckeyF] = np.zeros(Nbins)
                                            fit2st_chiBins = np.zeros(Nbins)                            
                                            fit2st_chiBinsAll = np.zeros(Nbins)

                                            mC = 0
                                            binList2st[tsT_s][ri][ckeyF] = []
                                            
                                            for nb in range(Nbins):
                                                XData  = np.zeros(Nfpts)
                                                fitVal = np.zeros(Nfpts)
                                                fitErr = np.zeros(Nfpts)
                                                TsVar  = np.zeros(Nfpts)

                                                xData2st_VP   = {}
                                                yData2st_VP   = {}
                                                errData2st_VP = {}
                                                
                                                fIdx = 0
                                                for itsnk, tsnk in enumerate(Tsink_2st):
                                                    NtIns = tsnk-1
                                                    
                                                    xData2st_VP[tsnk]   = np.zeros(NtIns)
                                                    yData2st_VP[tsnk]   = np.zeros(NtIns)
                                                    errData2st_VP[tsnk] = np.zeros(NtIns)                                                    
                                                    
                                                    adTplM = (phMTag,opT,mTag,tsnk)
                                                    if analyzeDataSetM[adTplM]:
                                                        ckey3 = (opT,phMTag,mTag,tsnk,isrc,isnk,gMat,z,irowSrc,irowSnk) # 3pt-tuple

                                                        for tins in range(NtIns):
                                                            # Set fit values and error
                                                            XData[fIdx] = tins+1
                                                            fitVal[fIdx] = RA_bins[ri][ckey3][nb,tins+1]
                                                            fitErr[fIdx] = RA_ave[ri][ckey3][1][tins+1]

                                                            TsVar[fIdx] = tsnk # Tsink variable
                                                            
                                                            xData2st_VP[tsnk][tins]   = XData[fIdx]
                                                            yData2st_VP[tsnk][tins]   = fitVal[fIdx]
                                                            errData2st_VP[tsnk][tins] = fitErr[fIdx]

                                                            fIdx += 1
                                                        # for-tins
                                                # for-tsnk
                                                
                                                                                
                                                # Construct 2-state fit guesses
                                                guess_2st = np.array([1.0 for i in range(Nfprm)],dtype=np.float64)

                                                if mVec[2] < 2:
                                                    if mTag == 'pz.0': # For z=0 the default parameters are already set
                                                        if z != 0: # Use either previous z3, or z3=0, same Pz(=0)
                                                            zG0 = 0
                                                            zGp = z-1
                                                            ckeyG0 = (opT,phMTag,mTag,isrc,isnk,gMat,zG0,irowSrc,irowSnk)
                                                            ckeyGp = (opT,phMTag,mTag,isrc,isnk,gMat,zGp,irowSrc,irowSnk)
                                                            for ifp,fp in enumerate(fitParam2stOpt):
                                                                try:
                                                                    testG0 = fit2stOpt_bins[tsT_s][fp][ri][ckeyG0][nb]
                                                                except:
                                                                    testG0 = fit2stOpt_ave[tsT_s][fp][ri][ckeyG0][0]
                                                                if testG0 != None:
                                                                    guess_2st[ifp] = testG0
                                                                else:
                                                                    guess_2st[ifp] = 1.0
                                                    else:
                                                        if z == 0: # Use Pz=0, or previous Pz and same z3(=0) 
                                                            mTagG0 = MomTag(0)
                                                            mTagGp = MomTag(mVec[2]-1)
                                                            phMTagG0 = phasedTag('unphased')
                                                            phMTagGp = phMTag
                                                            if mTag == 'pz.+4':
                                                                phMTagG0 = phasedTag('unphased')
                                                            ckeyG0 = (opT,phMTagG0,mTagG0,isrc,isnk,gMat,z,irowSrc,irowSnk)
                                                            ckeyGp = (opT,phMTagGp,mTagGp,isrc,isnk,gMat,z,irowSrc,irowSnk)
                                                            for ifp,fp in enumerate(fitParam2stOpt):
                                                                try:
                                                                    testG0 = fit2stOpt_bins[tsT_s][fp][ri][ckeyG0][nb]
                                                                except:
                                                                    testG0 = fit2stOpt_ave[tsT_s][fp][ri][ckeyG0][0]
                                                                if testG0 != None:
                                                                    guess_2st[ifp] = testG0
                                                                else:
                                                                    guess_2st[ifp] = 1.0
                                                        else: # Use Pz=0, same z3
                                                            if ri=='Re' or (ri=='Im' and not(mTag=='pz.+1')):
                                                                if ri=='Re':
                                                                    mTagG0 = MomTag(0)
                                                                else:
                                                                    mTagG0 = MomTag(1)
                                                                phMTagG0 = phasedTag('unphased')
                                                                ckeyG0 = (opT,phMTagG0,mTagG0,isrc,isnk,gMat,z ,irowSrc,irowSnk)
                                                                for ifp,fp in enumerate(fitParam2stOpt):
                                                                    try:
                                                                        testG0 = fit2stOpt_bins[tsT_s][fp][ri][ckeyG0][nb]
                                                                    except:
                                                                        testG0 = fit2stOpt_ave[tsT_s][fp][ri][ckeyG0][0]
                                                                    if testG0 != None:
                                                                        guess_2st[ifp] = testG0
                                                                    else:
                                                                        guess_2st[ifp] = 1.0
                                                    # Pz != 0

                                                # Set the matrix element to the result of a good fit
                                                Mguess = summFitAve['L']['M_l8'][ri][ckeyF][0]
                                                guess_2st[0] = Mguess
                                                #------------------

                                                # Set fit parameter bounds
                                                minBnd = [None for ifp in range(len(fitParam2stOpt))]
                                                maxBnd = [None for ifp in range(len(fitParam2stOpt))]
                                                for ifp,fp in enumerate(fitParam2stOpt):
                                                    if mC == 0:
                                                        bndGuess = guess_2st[ifp]
                                                    else:
                                                        bndGuess = fit2stOpt_bins[tsT_s][fp][ri][ckeyF][mC-1]

                                                    if doStrdTwoStFit[ckeyDoFit]:
                                                        minBnd[ifp] = -np.inf
                                                        maxBnd[ifp] =  np.inf
                                                    else:
                                                        minBnd[ifp] = guess_2st[ifp] - Pg_varFct*np.abs(bndGuess)
                                                        maxBnd[ifp] = guess_2st[ifp] + Pg_varFct*np.abs(bndGuess)

                                                minBnd[0] = Mguess - Mg_varFct*np.abs(Mguess)
                                                maxBnd[0] = Mguess + Mg_varFct*np.abs(Mguess)

                                                twoSt_bnd = (minBnd,maxBnd)                                                

                                                try:
                                                    XDataTpl = (XData,TsVar)
                                                    if doStrdTwoStFit[ckeyDoFit]:
                                                        if nb == 0:
                                                            print(ckeyDoFit, ': Standard fit')
                                                        fParams_2st, covMat_2st = scipyOpt.curve_fit(twoStateFitOpt,XDataTpl,fitVal,sigma=fitErr,p0=guess_2st)

                                                        for ifp,fp in enumerate(fitParam2stOpt):
                                                            fit2stOpt_bins[tsT_s][fp][ri][ckeyF][mC] = fParams_2st[ifp]

                                                        fit2st_chiBinsAll[mC] = chiSq_twoStateOpt(XDataTpl,fitVal,fitErr,
                                                                                                  fit2stOpt_bins[tsT_s]['M'][ri][ckeyF][mC] ,fit2stOpt_bins[tsT_s]['A01'][ri][ckeyF][mC],fit2stOpt_bins[tsT_s]['A11'][ri][ckeyF][mC],
                                                                                                  fit2stOpt_bins[tsT_s]['c1'][ri][ckeyF][mC],fit2stOpt_bins[tsT_s]['dE'][ri][ckeyF][mC])
                                                        fit2st_chiBinsAll[mC] /= (Nfpts-Nfprm-1)
                                                    else:
                                                        if doVarPro_2st:
                                                            if nb == 0:
                                                                print(ckeyDoFit, ': VarPro')
                                                            vpGuess = np.array([1.0,1.0])
                                                            resVP = scipyOpt.minimize(chiSq_VP_2st, x0 = vpGuess)
                                                            dE_fit = resVP.x[0] # The resulting dE from the fits
                                                            c1_fit = resVP.x[1] # The resulting c1 from the fits
                                                            Cp = np.dot(np.linalg.inv(PhiMVP_2st(resVP.x)),Y_2st(resVP.x)) # The rest of the fit parameters

                                                            fit2stOpt_bins[tsT_s]['dE'][ri][ckeyF][mC] = dE_fit
                                                            fit2stOpt_bins[tsT_s]['c1'][ri][ckeyF][mC] = c1_fit
                                                            for fp in CVP_2st:
                                                                fit2stOpt_bins[tsT_s][fp][ri][ckeyF][mC] = Cp[CVP_2st.index(fp)]

                                                            fit2st_chiBinsAll[mC] = chiSq_VP_2st(resVP.x) / (Nfpts-Nfprm-1)
                                                        else:
                                                            if nb == 0:
                                                                print(ckeyDoFit, ': Non-Standard fit')
                                                            fParams_2st, covMat_2st = scipyOpt.curve_fit(twoStateFitOpt,XDataTpl,fitVal,sigma=fitErr,bounds=twoSt_bnd,method='trf')#p0=guess_2st)

                                                            for ifp,fp in enumerate(fitParam2stOpt):
                                                                fit2stOpt_bins[tsT_s][fp][ri][ckeyF][mC] = fParams_2st[ifp]

                                                            fit2st_chiBinsAll[mC] = chiSq_twoStateOpt(XDataTpl,fitVal,fitErr,
                                                                                                      fit2stOpt_bins[tsT_s]['M'][ri][ckeyF][mC] ,fit2stOpt_bins[tsT_s]['A01'][ri][ckeyF][mC],fit2stOpt_bins[tsT_s]['A11'][ri][ckeyF][mC],
                                                                                                      fit2stOpt_bins[tsT_s]['c1'][ri][ckeyF][mC],fit2stOpt_bins[tsT_s]['dE'][ri][ckeyF][mC])
                                                            fit2st_chiBinsAll[mC] /= (Nfpts-Nfprm-1)

                                                    # The matrix element
                                                    Melem = fit2stOpt_bins[tsT_s]['M'][ri][ckeyF][mC]

                                                    if not np.isnan(Melem):
                                                        M2stOpt_bins[tsT_s][ri][ckeyF][mC] = Melem
                                                        fit2st_chiBins[mC] = fit2st_chiBinsAll[mC]
                                                        binList2st[tsT_s][ri][ckeyF].append(nb)                                                            
                                                        mC += 1
                                                        if(mC%70==0 and doVarPro_2st):
                                                            print(mC, Melem)
                                                except:
                                                    pass
                                            # for-nbins

                                            for fp in fitParam2stOpt:
                                                fit2stOpt_bins[tsT_s][fp][ri][ckeyF] =  fit2stOpt_bins[tsT_s][fp][ri][ckeyF][:mC]

                                            M2stOpt_bins[tsT_s][ri][ckeyF] = M2stOpt_bins[tsT_s][ri][ckeyF][:mC]
                                            fit2st_chiBins = fit2st_chiBins[:mC]

                                            if mC == 0: # No successful fits
                                                M2stOpt_ave[tsT_s][ri][ckeyF] = (None,None)
                                                fit2stOpt_chiAve[tsT_s][ri][ckeyF] = (None,None)
                                                for fp in fitParam2stOpt:
                                                    fit2stOpt_ave[tsT_s][fp][ri][ckeyF] = (None,None)
                                            elif mC == 1: # 1 successful fit
                                                M2stOpt_ave[tsT_s][ri][ckeyF] = (M2stOpt_bins[tsT_s][ri][ckeyF][0],0.0)
                                                fit2stOpt_chiAve[tsT_s][ri][ckeyF] = (fit2st_chiBins[0],0.0)
                                                for fp in fitParam2stOpt:
                                                    fit2stOpt_ave[tsT_s][fp][ri][ckeyF] = (fit2stOpt_bins[tsT_s][fp][ri][ckeyF][0],0.0)
                                            else:
                                                M2stOpt_ave[tsT_s][ri][ckeyF] = jackknife_ave(M2stOpt_bins[tsT_s][ri][ckeyF],mC,tL=1)
                                                fit2stOpt_chiAve[tsT_s][ri][ckeyF] = jackknife_ave(fit2st_chiBins,mC,tL=1)
                                                for fp in fitParam2stOpt:
                                                    fit2stOpt_ave[tsT_s][fp][ri][ckeyF] = jackknife_ave(fit2stOpt_bins[tsT_s][fp][ri][ckeyF],mC,tL=1)


                                            print('%s - mom = %s , z = %d: mC = %d: '%(ri,mTag,z,mC),M2stOpt_ave[tsT_s][ri][ckeyF])#, ckeyF)#,fit2stOpt_chiAve[tsT_s][ri][ckeyF])
                                        # if not (pz*z == 0 and Imag)
                                    # Re-im

                        print('Done: Momentum %s, ts_low = %d'%(mTag,tsT))
                    # for-momentum
                # for two-state ts-ranges
            
else:
    print('Skipping performing two-state fits')
        
print('\nDone\n')                                    



In [ ]:

if twoStFit_Source['Dump']:
    print('Dumping the two-state fits...')
    pickle.dump(M2stOpt_bins, open('p_data/M2stOpt_bins.p', 'wb' ))
    pickle.dump(M2stOpt_ave, open('p_data/M2stOpt_ave.p', 'wb' ))
    pickle.dump(fit2stOpt_bins, open('p_data/fit2stOpt_bins.p', 'wb' ))
    pickle.dump(fit2stOpt_ave, open('p_data/fit2stOpt_ave.p', 'wb' ))
    pickle.dump(fit2stOpt_chiAve, open('p_data/fit2stOpt_chiAve.p', 'wb' ))
    pickle.dump(binList2st, open('p_data/binList2st.p', 'wb' ))
elif twoStFit_Source['Load']:
    print('Loading the two-state fits...')
    M2stOpt_bins     = pickle.load(open('p_data/M2stOpt_bins.p', 'rb'))
    M2stOpt_ave      = pickle.load(open('p_data/M2stOpt_ave.p', 'rb'))
    fit2stOpt_bins   = pickle.load(open('p_data/fit2stOpt_bins.p', 'rb'))
    fit2stOpt_ave    = pickle.load(open('p_data/fit2stOpt_ave.p', 'rb'))
    fit2stOpt_chiAve = pickle.load(open('p_data/fit2stOpt_chiAve.p', 'rb'))
    binList2st       = pickle.load(open('p_data/binList2st.p', 'rb'))
else:
    print('Skipping dumping/loading of the two-state fits')

print('\nDone\n')


## Multi-plots with all fits

#### Dictionary with 'eye' plateau ranges

In [ ]:
# twoSt_mrk = '>'
# twoSt_col = 'blueviolet'

# Lsum_mrk = 'H'
# Lsum_col = 'deeppink'

Lsum_mrk = '>'
Lsum_col = 'saddlebrown' # 'blueviolet'

twoSt_mrk = 'X'
twoSt_col = 'hotpink' #'deeppink'


E1sum_mrk = '<'
E1sum_col = 'tomato'

E1rsum_mrk = 'v'
E1rsum_col = 'goldenrod'


yLimFct = {'Re':{'pz.0' : {0:0.99, 1:1.01},
                 'pz.+1': {0:0.99, 1:1.01},
                 'pz.+2': {0:0.95, 1:1.05},
                 'pz.+3': {0:0.90, 1:1.10},
                 'pz.+4': {0:0.90, 1:1.10},
                 'pz.+5': {0:0.92, 1:1.08},
                 'pz.+6': {0:0.70, 1:1.50}},
           'Im':{'pz.+1': {0:0.95, 1:1.05},
                 'pz.+2': {0:0.95, 1:1.05},
                 'pz.+3': {0:0.90, 1:1.10},
                 'pz.+4': {0:0.95, 1:1.05},
                 'pz.+5': {0:0.95, 1:1.05},
                 'pz.+6': {0:0.90, 1:1.10}}}

fadeVal = 0.4

print('\nDone\n')


#### Define optimal $t_{\rm snk}$ of plateau, summation and two-state fits for plotting and for ITDs

In [ ]:
iFit = 3

tsinkOpt = {}

fTp = 'Plat'
tsinkOpt[fTp] = {'Re': {}, 'Im':{}}
ri = 'Re'
for mTag in allMomenta:
    mVec = momVector_Dict[mTag]
    if mVec[2] <= 2:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 12
    elif mVec[2] == 3:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 10
    elif mVec[2] == 4:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 10 #if z <= 5 else 12
    elif mVec[2] >= 5:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 10

ri = 'Im'
for mTag in allMomenta:
    mVec = momVector_Dict[mTag]
    if mVec[2] <= 1:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 14
    elif mVec[2] == 2:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 12
    elif mVec[2] == 3:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 10
    elif mVec[2] == 4:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 10 if z <= 3 else 12
    elif mVec[2] >= 5:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 10

#------------------------------------          
           
    
fTp = 'L-summ'
tsinkOpt[fTp] = {'Re': {}, 'Im':{}}
ri = 'Re'
for mTag in allMomenta:
    mVec = momVector_Dict[mTag]
    if mVec[2] == 0:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 6
    elif mVec[2] == 1 or mVec[2] == 2:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 6 if z <= 6 else 8
    elif mVec[2] == 3:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 6
    elif mVec[2] == 4:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 6 if z <= 5 else 8
    elif mVec[2] >= 5:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 6

ri = 'Im'
for mTag in allMomenta:
    mVec = momVector_Dict[mTag]
    if mVec[2] <= 2:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 8
    else:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 6

#------------------------------------            


fTp = '2-state'
tsinkOpt[fTp] = {'Re': {}, 'Im':{}}
ri = 'Re'
for mTag in allMomenta:
    mVec = momVector_Dict[mTag]
    if mVec[2] <= 1:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 8 if z != 2 else 6
    elif mVec[2] == 2:
        for z in dispLen_ave:
            if z == 0:                
                tsinkOpt[fTp][ri][(mTag,z)] = 4
            else:
                tsinkOpt[fTp][ri][(mTag,z)] = 6 if z <= 2 else 8
    elif mVec[2] == 3:
        for z in dispLen_ave:            
            tsinkOpt[fTp][ri][(mTag,z)] = 6 if (z == 1 or z == 8) else 8
    elif mVec[2] == 4:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 8
    elif mVec[2] == 5:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 4 if z == 0 else 6
    elif mVec[2] == 6:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 6

            

ri = 'Im'
for mTag in allMomenta:
    mVec = momVector_Dict[mTag]
    if mVec[2] <= 1:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 8
    elif mVec[2] == 2:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 6 if z <= 3 else 8
    elif mVec[2] == 3:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 6 if (z <= 4 or z == 6) else 4
    elif mVec[2] == 4:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 4 if (z == 4) else 6
    elif mVec[2] == 5:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 6
    elif mVec[2] == 6:
        for z in dispLen_ave:
            tsinkOpt[fTp][ri][(mTag,z)] = 6 if z <= 7 else 4

                                        
print('\nDone\n')


#### Whether to consider a 2-state fit for a given (Pz,z) combination

In [ ]:

calc2st = {}       
for tsT in twoSt_tLow_Gen:
    tsT_s = 'ts%d'%(tsT)
    
    calc2st[tsT_s] = {'Re': {}, 'Im': {}}

failed2st_tpl = []    
# failed2st_tpl = [('Re','pz.0',1) ,
#                  ('Re','pz.+1',0),('Re','pz.+1',1),
#                  ('Re','pz.+6',0),('Re','pz.+6',1),('Re','pz.+6',2),('Re','pz.+6',3),('Re','pz.+6',4),
#                  ('Re','pz.+6',5),('Re','pz.+6',6),('Re','pz.+6',7),('Re','pz.+6',8),
#                  ('Im','pz.+4',7),('Im','pz.+1',8),
#                  ('Im','pz.+5',1),('Im','pz.+5',2),('Im','pz.+5',3),('Im','pz.+5',6),('Im','pz.+5',7),('Im','pz.+5',8)]


for opT in SrcSnkOpTypes:
    for iphT in phaseTypes:            
        for phM in Phases_MomTags_ave[iphT]:
            phMTag = phasedTag(iphT,phM)
            phTpl = (phMTag,opT)

            momTags = momTags_Dict_ave[(phMTag,opT)]

            for mTag in momTags:
                mVec = momVector_Dict[mTag]
                for tsT in twoSt_Iter[mTag]:
                    tsT_s = 'ts%d'%(tsT)

                    zMom = mVec[2]
                    pmTpl = phMomTpl(phMTag,zMom)

                    srcOpList = srcOpList3pt_Dict[opT][pmTpl] # These are the same in two- and
                    snkOpList = srcOpList3pt_Dict[opT][pmTpl] # three-point functions

                    for isrc, src in enumerate(srcOpList):
                        for isnk, snk in enumerate(snkOpList):  

                            # Only upolarized PDF for now
                            irowSrc = 1
                            irowSnk = 1
                            gMat = 'gt'

                            for z in dispLen_ave:
                                ckeyF = (opT,phMTag,mTag,isrc,isnk,gMat,z,irowSrc,irowSnk)

                                for ri in ReIm:
                                    if(z*mVec[2]==0 and ri=='Im'):
                                        calc2st[tsT_s][ri][ckeyF] = False
                                    else:
                                        calc2st[tsT_s][ri][ckeyF] = True
                                        for fTpl in failed2st_tpl:
                                            riT,mTagT,zT = fTpl
                                            if(riT == ri and mTagT == mTag and zT == z):
                                                calc2st[tsT_s][ri][ckeyF] = False
                                        
print('\nDone\n')

### Vector Charge with optimal fit ranges

In [ ]:
ckey_ZV = ('single','unphased','pz.0',tsinkOpt['2-state']['Re'][('pz.0',0)],0,0,'gt',0,1,1)

ZV  = Mplat_ave['Re'][ckey_ZV][0]
print('ZV = ',ZV)

In [ ]:
opTR = 'single'
isrcR = 0
isnkR = 0
irowSrcR = 1
irowSnkR = 1
gMatR = 'gt'

plot_rSummE1 = False
plot_SummE1  = False

Lsum_tLow = [4,6,8]

ri = 'Im'

for mTagR in allMomenta:    
    if ri == 'Im' and mTagR == 'pz.0':
        continue
    
    print(mTagR)
    
    #mTagR = 'pz.0'
    mVecR = momVector_Dict[mTagR]

    if mVecR[2] <= 3:
        phTR = 'unphased'
    else:
        phTR = 'phased-d001_2.00'
    ophTpl = (phTR,opTR)
    tSinkLowest = TsinkList[(phTR,opTR)][0]

    # Plot ranges for 2-state fits
    twoSt_tLow_Plot = twoSt_tLow[mTagR]
    if (mVecR[2] == 3 and ri == 'Im'):
        try:
            twoSt_tLow_Plot.remove(8)
        except:
            pass
      
    
    # Plot ranges for summation method
    Lsum_tLow_Plot = Lsum_tLow
    if (mVecR[2] == 5 and ri == 'Re') or (mVecR[2] == 3 and ri == 'Im'):
        try:
            Lsum_tLow_Plot.remove(8)
        except:
            pass
        

    maxRngX = max(Lsum_tLow_Plot) if max(Lsum_tLow_Plot) > max(twoSt_tLow_Plot) else max(twoSt_tLow_Plot)

    print('L-summ fit range : ', Lsum_tLow_Plot)
    print('2-state fit range: ', twoSt_tLow_Plot)
    #-------------------------------


    # What to plot
    if ri == 'Re':
        dispPlot = [0,2,5,8] #[0,1,2,3,4,5,6,7,8]
    else:
        dispPlot = [1,2,5,8] #[1,2,3,4,5,6,7,8]
    nFitT = 3
    nDisp = len(dispPlot)
    bAx = nDisp - 1
    tAx = 0
    lAx = 0


    figFM, axFM = plt.subplots(nDisp, nFitT, sharex='col', sharey='row', figsize=[16,12])
    figFM.subplots_adjust(hspace=0.00,wspace=0.03)#,left=0.8)

    gs = gridspec.GridSpec(nDisp, nFitT, width_ratios=[1.5,0.8,1.1])
    #-------------------------------------------------------


    for idL,dL in enumerate(dispPlot):
        TsinkList_Plot_Plat = []
        tsinkOpt[fTp][ri][(mTag,z)]
        upRngTPlat = twoSt_tHi[(mTagR,dL)]
#         if mVecR[2] == 4:
#             upRngTPlat = tsinkOpt['Plat'][ri][(mTagR,dL)]
        for t in range(tSinkLowest,upRngTPlat+1,2):
            TsinkList_Plot_Plat.append(t)
        print('z = %d, Plat fit range   : '%(dL), TsinkList_Plot_Plat)


        for ifT in range(nFitT):
            axFM[idL][ifT] = plt.subplot(gs[ifT+nFitT*idL])


        # Plot the ratio for all tsinks (leftmost axis)
        axIdx = 0
        i = 0
        tsRng = {}

        raMaxVal = -1e-6
        raMinVal = 1e6
        for tsink in TsinkList[(phTR,opTR)]:#TsinkList_Plot_Plat:
            fStyle = 'full'
            faded = 1.0 if tsink in TsinkList_Plot_Plat else fadeVal

            tsRng[tsink] = np.arange(tsink) - tsink/2

            adTplM = (phTR,opTR,mTagR,tsink)
            if analyzeDataSetM[adTplM]:
                ckey = (opTR,phTR,mTagR,tsink,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

                axFM[idL][axIdx].errorbar(tsRng[tsink][1:]+osDctF[i]/2,  RA_ave[ri][ckey][0][1:],
                             yerr = RA_ave[ri][ckey][1][1:],
                             color = cDct[i], marker = mDct[i], ls = 'none',
                             label=r'$t_s=%da$'%(tsink),
                             ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle=fStyle,alpha=faded)
                i+=1        
                #--------

                if tsink in TsinkList_Plot_Plat:
                    tsnkMid = None
                    for tins in range(1,tsink):
                        if RA_ave[ri][ckey][0][tins] > raMaxVal:
                            raMaxVal = RA_ave[ri][ckey][0][tins]
                            ckeyMax = ckey
                            tinsMax = tins
                        if RA_ave[ri][ckey][0][tins] < raMinVal:
                            raMinVal = RA_ave[ri][ckey][0][tins]
                            ckeyMin = ckey
                            tinsMin = tins
            #if-analyze
        # for tsink

        # limits
        axFM[idL][axIdx].xaxis.set_ticks(np.arange(-8, 9, 2))
        axFM[idL][axIdx].set_xlim(-8,8)

        # Set y-limits based on the values and errors of largest tsep in 2-state fit
        tsnkErr = TsinkList_Plot_Plat[-1]  
        ckeyErr = ckey = (opTR,phTR,mTagR,tsnkErr,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)
        errOff  = RA_ave[ri][ckeyErr][1][tsnkErr//2]
        yLimMin = raMinVal - errOff
        yLimMax = raMaxVal + errOff    
        yLim0 = yLimFct[ri][mTagR][0]*yLimMin
        yLim1 = yLimFct[ri][mTagR][1]*yLimMax

        axFM[idL][axIdx].set_ylim(yLim0,yLim1)

        ylims = axFM[idL][0].get_ylim()    
        #----------------------------------------------------------------------------------------------



        # Plot plateau fits for all tsep (center axis)    
        axIdx = 1
        for itsP, tsnkP in enumerate(TsinkList[(phTR,opTR)]): #enumerate(TsinkList_Plot_Plat):
            ckeyPlat = (opTR,phTR,mTagR,tsnkP,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)
            faded = 1.0 if tsnkP in TsinkList_Plot_Plat else fadeVal

            iFit = optPlatFit[ri][ckeyPlat]
            if iFit == None:
                iFit = eyePlatFit[ri][tsnkP]

            if tsnkP == 4:
                platVal = (RA_ave[ri][ckeyPlat][0][2],RA_ave[ri][ckeyPlat][1][2])
            else:
                platVal = Mplat_ave[ri][ckeyPlat][iFit]
            platLab = r'$t_s = %da$'%(tsnkP)

            pXrng = np.array([tsnkP])
            axFM[idL][axIdx].errorbar(pXrng, platVal[0], yerr = platVal[1],
                                      marker = mDct[itsP], color = cDct[itsP], ls='none',ms=7.0,
                                      elinewidth = elw, capsize=ecs, fillstyle='full',alpha=faded)#,label=platLab)

        # limits
        axFM[idL][axIdx].set_xlim(2.5,15.5)
        axFM[idL][axIdx].set_ylim(ylims[0],ylims[1])    
        #----------------------------------------------------------------------------------------------



        # Plot summed ratio linear fits for all ts_low, 1-exp fits and two-state fits (rightmost axis)
        axIdx = 2

        # Summed ratio linear fits
        ckeySL = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)
        for isf,sf in enumerate(Lsum_tLow_Plot):
            sfs = 'l%s'%(sf)

            sumLVal = summFitAve['L']['M_%s'%(sfs)][ri][ckeySL]

            if isf == 0:
                sumLLab = 'Summ. linear'
            else:
                sumLLab = None        

            pXrng = np.array([sf])-0.15
            axFM[idL][axIdx].errorbar(pXrng, sumLVal[0], yerr = sumLVal[1],
                                      marker = Lsum_mrk, color = Lsum_col, ls='none',ms=7.0,
                                      elinewidth = elw, capsize=ecs, fillstyle='full',label = sumLLab)        
        #---------------------------------


        # Two-state fits
        ckey2st = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)
        if ((dL*mVecR[2] != 0) or ri=='Re'):# and (mTagR != 'pz.+3'):
            for itsT, tsT in enumerate(twoSt_tLow_Plot):
                tsT_s = 'ts%d'%(tsT)

                twoStVal = M2stOpt_ave[tsT_s][ri][ckey2st]        

                if itsT == 0:
                    twoStLab = '2-state'
                else:
                    twoStLab = None        

                if twoStVal[0] != None:

                    pXrng = np.array([tsT])+0.15
                    axFM[idL][axIdx].errorbar(pXrng, twoStVal[0], yerr = twoStVal[1],
                                              marker = twoSt_mrk, color = twoSt_col, ls='none',ms=7.0,
                                              elinewidth = elw, capsize=ecs, fillstyle='full',label = twoStLab)        
                else:
                    print('Skipping 2-state fit for tS = %d'%(tsT), ckey2st)            
        else:
            print('Skipping 2-state fit for ', ckey2st)
        #---------------------------------


        # 1-exp fits for regular- and reduced-summed ratio
        ckeyE1 = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)
        tE1s = maxRngX+1 # Lsum_tLow_Plot[-1]
        e1Idx = 0
        if plot_SummE1 or plot_rSummE1:
            axFM[idL][axIdx].axvline(tE1s,lw=0.7,color='k',ls='--')    

        # Regular-summed ratio 1-exp
        if plot_SummE1:
            e1Idx = 1
            sumE1Val = summFitAve['E1']['M'][ri][ckeyE1]

            E1fLbl = 'Summ. 1-exp fit'

            pXrng = np.array([tE1s+e1Idx])
            axFM[idL][axIdx].errorbar(pXrng, sumE1Val[0], yerr = sumE1Val[1],
                                      marker = E1sum_mrk, color = E1sum_col, ls='none',ms=7.0,
                                      elinewidth = elw, capsize=ecs, fillstyle='full',label = E1fLbl)

            if sumE1Val[0] == None:
                print('Skipping summ. 1-exp fit for ', ckeyE1)
            else:
                if np.isnan(sumE1Val[0]):
                    print('Skipping summ. 1-exp fit for ', ckeyE1)
        #---------------------------------


        # Reduced-summed ratio 1-exp
        if plot_rSummE1:
            e1Idx = 2
            rsumE1Val = E1rsum_ave['M'][ri][ckeyE1]

            rE1fLbl = 'r-Summ. 1-exp fit'

            pXrng = np.array([tE1s+e1Idx])
            axFM[idL][axIdx].errorbar(pXrng, rsumE1Val[0], yerr = rsumE1Val[1],
                                      marker = E1rsum_mrk, color = E1rsum_col, ls='none',ms=7.0,
                                      elinewidth = elw, capsize=ecs, fillstyle='full',label = rE1fLbl)

            if rsumE1Val[0] == None:
                print('Skipping r-summ. 1-exp fit for ', ckeyE1)
            else:
                if np.isnan(rsumE1Val[0]):
                    print('Skipping r-summ. 1-exp fit for ', ckeyE1)
        #---------------------------------

        # limits
        tSR = Lsum_tLow_Plot[0]-1
#         print('Axis range:', mTagR,np.arange(tSR, tE1s+e1Idx+1, 2))
        axFM[idL][axIdx].set_xlim(tSR,tE1s+e1Idx+1)
#        axFM[idL][axIdx].xaxis.set_ticks(np.arange(Lsum_tLow_Plot[0], tE1s+e1Idx+1, 2))
        axFM[idL][axIdx].set_ylim(ylims[0],ylims[1])
        #----------------------------------------------------------------------------------------------

        

        # Plot plateau, summation and 2-state bands
        tsnkP = tsinkOpt['Plat'][ri][(mTagR,dL)]
        tsnkIdx = TsinkList[(phTR,opTR)].index(tsnkP) #TsinkList_Plot_Plat.index(tsnkP)
        ckeyPlat = (opTR,phTR,mTagR,tsnkP,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

        iFit = optPlatFit[ri][ckeyPlat]
        if iFit == None:
            iFit = eyePlatFit[ri][tsnkP]
        platVal = Mplat_ave[ri][ckeyPlat][iFit]

        tsnkLsum = tsinkOpt['L-summ'][ri][(mTagR,dL)]
        sfs = 'l%d'%(tsnkLsum)
        ckeySL   = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)
        LsumVal = summFitAve['L']['M_%s'%(sfs)][ri][ckeySL]

        tsnk2st = tsinkOpt['2-state'][ri][(mTagR,dL)]
        tsT_s = 'ts%d'%(tsnk2st)
        ckey2st = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)
        twoStVal = M2stOpt_ave[tsT_s][ri][ckey2st]    

        for iAx in range(nFitT):
            xlims = axFM[idL][iAx].get_xlim()
            if iAx == 0:
                bOffsL = 0.4#(xlims[1] - xlims[0])*0.03
                bOffsR = 0
            elif iAx == nFitT-1:
                bOffsR = 0.3#(xlims[1] - xlims[0])*0.03
                bOffsL = 0
            else:
                bOffsL = 0
                bOffsR = 0

            ibF = 0
            bandX = np.arange(xlims[0]+bOffsL*ibF,xlims[1]-bOffsR*ibF+0.1,0.1)

            platY = np.array([platVal[0] for i in range(len(bandX))])
            platYerr = np.array([platVal[1] for i in range(len(bandX))])

            axFM[idL][iAx].plot(bandX, platY, ls='-', color=cDct[tsnkIdx],linewidth=1.0,alpha=0.5)
            axFM[idL][iAx].fill_between(bandX, platY-platYerr, platY+platYerr, facecolor=cDct[tsnkIdx], alpha=0.18)#,label=r'Plat. $t_s=%d$'%(tsinkPlat))  
            #-------

            ibF += 1 
            bandX = np.arange(xlims[0]+bOffsL*ibF,xlims[1]-bOffsR*ibF+0.1,0.1)

            sumLY = np.array([LsumVal[0] for i in range(len(bandX))])
            sumLYerr = np.array([LsumVal[1] for i in range(len(bandX))])

            axFM[idL][iAx].plot(bandX, sumLY, ls='-', color=Lsum_col,linewidth=1.0,alpha=0.5)
            axFM[idL][iAx].fill_between(bandX, sumLY-sumLYerr, sumLY+sumLYerr, facecolor=Lsum_col, alpha=0.12)#,label=r'Plat. $t_s=%d$'%(tsinkPlat)) 
            #-------

            if twoStVal[0] != None:
                ibF += 1
                bandX = np.arange(xlims[0]+bOffsL*ibF,xlims[1]-bOffsR*ibF+0.1,0.1)

                sumLY = np.array([twoStVal[0] for i in range(len(bandX))])
                sumLYerr = np.array([twoStVal[1] for i in range(len(bandX))])

                axFM[idL][iAx].plot(bandX, sumLY, ls='-', color=twoSt_col,linewidth=1.0,alpha=0.5)
                axFM[idL][iAx].fill_between(bandX, sumLY-sumLYerr, sumLY+sumLYerr, facecolor=twoSt_col, alpha=0.18)#,label=r'Plat. $t_s=%d$'%(tsinkPlat)) 

        #----------------------------------------------------------------------------------------------    



        # Remove ytick labels except for leftmost and bottom axes
        if idL != bAx:
            for iAx in range(nFitT):
                axFM[idL][iAx].xaxis.set_ticklabels([])


        axFM[idL][1].yaxis.set_ticklabels([])
        axFM[idL][2].yaxis.set_ticklabels([])


        # Create pz,z3 label/text, and plateau text
        cstLabY = ylims[1] - (ylims[1]-ylims[0])*0.15
        axFM[idL][0].text(3,cstLabY,r'$p_z = %d$ , $z_3 = %d$'%(mVecR[2],dL),fontsize=16)
        if idL == 0:
            axFM[idL][1].text(4,cstLabY,'Plateau fits',fontsize=16)



    # X-labels
    axFM[bAx][0].set_xlabel(r'$[t_{\rm ins} - t_s/2]/a$',fontsize=18)

    axFM[bAx][1].xaxis.set_ticks(np.arange(4, 15, 2))
    axFM[bAx][1].set_xlabel(r'$t_s/a$',fontsize=18)

    axFM[bAx][2].xaxis.set_ticks(np.arange(4, twoSt_tLow_Plot[-1]+1, 2))
    axFM[bAx][2].set_xlabel(r'$t_s^{\rm low}/a$',fontsize=18)





    # Y-label
    figFM.text(0.06,0.5, r'$\mathbf{%s}[R(p_z,z_3)]$' % (ri), ha="left", va="center", rotation=90,fontsize=18)
    # axFM[2][0].set_ylabel(r'Re[$R(p_z=%d;z_3)$]'%(mVecR[2]),fontsize=18)
    # axFM[2][1].set_ylabel(r'Im[$R(p_z=%d;z_3)$]'%(mVecR[2]),fontsize=18)

    # Legend
    axFM[tAx][0].legend(loc='best', ncol=3, numpoints=1, fontsize=14,
                        labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
                        markerscale=0.8)
    axFM[tAx][2].legend(loc='best', ncol=2, numpoints=1, fontsize=14,
                        labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
                        markerscale=0.8)



    figFM.savefig('plots/matElem_4plot_%s_%s.pdf'%(ri,mTagR),bbox_inches='tight')



### Plot rest-frame density $\mathcal{M}(p_z=0;z_3^2)$ using optimal fit ranges

In [ ]:

figRFD, axRFD = plt.subplots(figsize=[10,6.25])

opTRS = 'single'
phTRS = 'unphased'
isrcRS = 0
isnkRS = 0
irowSrcRS = 1
irowSnkRS = 1

gMatRS = 'gt'

mTagRS = 'pz.0'
nf=0

ri = 'Re'

xRng = np.array(dispLen_ave)


restFrD_PL  = (np.zeros(len(dispLen_ave)),np.zeros(len(dispLen_ave)))
restFrD_SL  = (np.zeros(len(dispLen_ave)),np.zeros(len(dispLen_ave)))
restFrD_2st = (np.zeros(len(dispLen_ave)),np.zeros(len(dispLen_ave)))

for idL,dL in enumerate(dispLen_ave):
    optTPlat = tsinkOpt['Plat'][ri][(mTagRS,dL)]
    ckeyPL = (opTRS,phTRS,mTagRS,optTPlat,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)

    
    sf = tsinkOpt['L-summ'][ri][(mTagRS,dL)]
    sfs = 'l%s'%(sf)
    ckeySL = (opTRS,phTRS,mTagRS,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)    
    
    tsT = tsinkOpt['2-state'][ri][(mTagRS,dL)]
    tsT_s = 'ts%d'%(tsT)
    ckey2st = (opTRS,phTRS,mTagRS,isrcRS,isnkRS,gMatRS,dL,irowSrcRS,irowSnkRS)

    adTplM = (phTRS,opTRS,mTagRS,optTPlat)
    if analyzeDataSetM[adTplM]:               
        iFit = optPlatFit[ri][ckeyPL]
        if iFit == None:
            iFit = eyePlatFit[ri][optTPlat]        

        for i in range(2):
            restFrD_PL[i][idL] = Mplat_ave[ri][ckeyPL][iFit][i]     / ZV[0]
            restFrD_SL[i][idL] = summFitAve['L']['M_%s'%(sfs)][ri][ckeySL][i] / ZV[0]

            if calc2st[tsT_s][ri][ckey2st]:
                restFrD_2st[i][idL] = M2stOpt_ave[tsT_s][ri][ckey2st][i] / ZV[0]
            else:
                restFrD_2st[i][idL] = None

it = 0
axRFD.errorbar(xRng+osDctF[it], restFrD_PL[0], yerr = restFrD_PL[1],
             color = cDct[it], marker = mDct[it], ls = 'none', label = r'Plateau',
             ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full') # label = r'Plat., $t_s = %da$'%(tsink),

it+=1
axRFD.errorbar(xRng+osDctF[it+1], restFrD_SL[0], yerr = restFrD_SL[1],
             color = cDct[it], marker = mDct[it], ls = 'none', label = r'Summation', # $t_s^{\rm low} = %da$'%(sf),
             ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full')

it+=1
axRFD.errorbar(xRng+osDctF[it+1], restFrD_2st[0], yerr = restFrD_2st[1],
             color = cDct[it], marker = mDct[it], ls = 'none', label = r'2-state', # $t_s^{\rm low} = %da$'%(sf),
             ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full')




axRFD.set_xlabel(r'$z_3/a$',fontsize=18)
axRFD.set_ylabel(r'%s[$\mathcal{M}(P_z=0,z_3^2)$]'%(ri),fontsize=18)

axRFD.xaxis.set_ticks(np.arange(-1, 9, 1))
#axRFD.set_ylim(0.5,1.2)
#axRFD.yaxis.set_ticks(np.arange(0.45, 0.86, 0.05))

axRFD.legend(loc='best', ncol=1, numpoints=1, fontsize=18,
          labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
          markerscale=0.8)


figRFD.savefig('plots/rest_frame_density_%s.pdf'%(ri),bbox_inches='tight')


## Evaluate reduced ITD using optimal fit ranges

In [ ]:

def get_tITD(fTp_,ri_,mTag_,z_):
    t_itd = {}
    t_itd['c']    = tsinkOpt[fTp_][ri_][(mTag_,z_)]
    t_itd['z0']   = tsinkOpt[fTp_][ri_][(mTag_,0)]                            
    t_itd['p0']   = tsinkOpt[fTp_][ri_][('pz.0',z_)]
    t_itd['p0z0'] = tsinkOpt[fTp_][ri_][('pz.0',0)]
    return t_itd
#----------------------------------------
                           
def get_keyITD(fTp_,opT_,phMTag_,mTag_,isrc_,isnk_,gMat_,z_,irowSrc_,irowSnk_,tsPL=-1):
    ckey_itd = {}
    if fTp_ == 'Plat':
        if tsPL == -1:
            raise ValueError('get_keyITD: Got Plateau fit type but got no t_sink' )

        ckey_itd['c']    = (opT_,phMTag_   ,mTag_ ,tsPL['c']   ,isrc_,isnk_,gMat_,z_,irowSrc_,irowSnk_)
        ckey_itd['z0']   = (opT_,phMTag_   ,mTag_ ,tsPL['z0']  ,isrc_,isnk_,gMat_,0 ,irowSrc_,irowSnk_)
        ckey_itd['p0']   = (opT_,'unphased','pz.0',tsPL['p0']  ,isrc_,isnk_,gMat_,z_,irowSrc_,irowSnk_)
        ckey_itd['p0z0'] = (opT_,'unphased','pz.0',tsPL['p0z0'],isrc_,isnk_,gMat_,0 ,irowSrc_,irowSnk_)
    elif fTp_ == 'L-summ' or fTp_ == '2-state':
        ckey_itd['c']    = (opT_,phMTag_  ,mTag_ ,isrc_,isnk_,gMat_,z_,irowSrc_,irowSnk_)
        ckey_itd['z0']   = (opT_,phMTag_  ,mTag_ ,isrc_,isnk_,gMat_,0 ,irowSrc_,irowSnk_)
        ckey_itd['p0']   = (opT_,'unphased','pz.0',isrc_,isnk_,gMat_,z_,irowSrc_,irowSnk_)
        ckey_itd['p0z0'] = (opT_,'unphased','pz.0',isrc_,isnk_,gMat_,0 ,irowSrc_,irowSnk_)        
    else:
        raise ValueError('get_keyITD: Unsupported fit type')
        
    return ckey_itd
#----------------------------------------

def commonBins(a, b, c, d):
    a_set = set(a)
    b_set = set(b)
    c_set = set(c)
    d_set = set(d)
    if (a_set & b_set & c_set & d_set):
        return list(a_set & b_set & c_set & d_set)
    else:
#        print("No common elements")
        return []
#----------------------------------------


pzPosList = ['c','z0','p0','p0z0']
pzPosListOffs = ['z0','p0','p0z0']

rITD_bins = {}
rITD_ave  = {}
for fT in ['Plat','L-summ','2-state']:
    rITD_bins[fT] = {}
    rITD_ave[fT]  = {}
    for ri in ReIm:
        rITD_bins[fT][ri] = {}
        rITD_ave[fT][ri]  = {}
        
        
ckeyITD = {}


        
for opT in SrcSnkOpTypes:
    for iphT in phaseTypes:            
        for phM in Phases_MomTags_ave[iphT]:
            phMTag = phasedTag(iphT,phM)
            phTpl = (phMTag,opT)
            phT = None

            momTags = momTags_Dict_ave[(phMTag,opT)]

            for mTag in momTags:
                mVec = momVector_Dict[mTag]
                zMom = mVec[2]
                mFTag = momFileTag(mVec)
                pmTpl = phMomTpl(phMTag,zMom)

                srcOpList = srcOpList3pt_Dict[opT][pmTpl] # These are the same in two- and
                snkOpList = srcOpList3pt_Dict[opT][pmTpl] # three-point functions


                isrc = 0
                isnk = 0

                # Only upolarized PDF for now
                irowSrc = 1
                irowSnk = 1
                gMat = 'gt'
                
                for z in dispLen_ave:                    
                    tITD = {}
                    iFit = {}                    
                    ckeyITD[(mTag,z)] = {}

                    # Determine keys and optimal fit ranges
                    for fTp in ['Plat', 'L-summ', '2-state']:
                        ckeyITD[(mTag,z)][fTp] = {} #{'c': {}, 'z0': {}, 'p0':{}, 'p0z0':{}}
                        iFit[fTp] = {}
                        for ri in ReIm:
                            tITD[fTp] = get_tITD(fTp,ri,mTag,z)

                            ckeyITD[(mTag,z)][fTp][ri] = {}
                            ckeyITD[(mTag,z)][fTp][ri] = get_keyITD(fTp,opT,phMTag,mTag,isrc,isnk,gMat,z,irowSrc,irowSnk,tITD[fTp])

                            iFit[fTp][ri] = {}
                            if fTp == 'Plat':
                                for pzV in pzPosList:
                                    iFit[fTp][ri][pzV] = optPlatFit[ri][ckeyITD[(mTag,z)][fTp][ri][pzV]]
                                    if iFit[fTp][ri][pzV] == None:
                                        iFit[fTp][ri][pzV] = eyePlatFit[ri][tITD[fTp][pzV]]                   
                            elif fTp == 'L-summ':
                                for pzV in pzPosList:
                                    iFit[fTp][ri][pzV] = 'M_l%d'%(tITD[fTp][pzV])
                            elif fTp == '2-state':
                                for pzV in pzPosList:
                                    iFit[fTp][ri][pzV] = 'ts%d'%(tITD[fTp][pzV])
                        # Real-Imag
                    # for fit-type
                    
                    #--------------------------------------------------------------
                    # Plateau ITD
                    fTp = 'Plat'
                    
                    # Make sure Nbins are compatible
                    for ri in ReIm:
                        for pzV in pzPosListOffs:
                            if( np.shape(Mplat_bins[ri][ckeyITD[(mTag,z)][fTp][ri]['c']][iFit[fTp][ri]['c']]) !=
                                np.shape(Mplat_bins['Re'][ckeyITD[(mTag,z)][fTp]['Re'][pzV]][iFit[fTp]['Re'][pzV]]) ):
                                raise ValueError('Shape mismatch between \'c\' and %s bins'%(pzV))
                                
                        # Determine ITDs
                        rITD_bins['Plat'][ri][ckeyITD[(mTag,z)][fTp][ri]['c']] = \
                        ( (Mplat_bins[ri][ckeyITD[(mTag,z)][fTp][ri]['c']]   [iFit[fTp][ri]['c']]    / Mplat_bins['Re'][ckeyITD[(mTag,z)][fTp]['Re']['z0']][iFit[fTp]['Re']['z0']]) *
                          (Mplat_bins['Re'][ckeyITD[(mTag,z)][fTp]['Re']['p0z0']][iFit[fTp]['Re']['p0z0']] / Mplat_bins['Re'][ckeyITD[(mTag,z)][fTp]['Re']['p0']][iFit[fTp]['Re']['p0']]))
                        
                        
                        ckeyI = ckeyITD[(mTag,z)][fTp][ri]['c']
                        rITD_ave[fTp][ri][ckeyI] = jackknife_ave(rITD_bins[fTp][ri][ckeyI],np.shape(rITD_bins[fTp][ri][ckeyI])[0],tL=1)
                    #--------------------------------------------------------------

                    
                    # Summation method ITD
                    fTp = 'L-summ'
                    
                    # Make sure Nbins are compatible
                    for ri in ReIm:
                        for pzV in pzPosListOffs:
                            if( np.shape(summFitBins['L'][iFit[fTp][ri]['c']]  [ri]  [ckeyITD[(mTag,z)][fTp][ri]['c']]) !=
                                np.shape(summFitBins['L'][iFit[fTp]['Re'][pzV]]['Re'][ckeyITD[(mTag,z)][fTp]['Re'][pzV]]) ):
                                raise ValueError('Shape mismatch between \'c\' and %s bins'%(pzV))
                                
                        # Determine ITDs
                        rITD_bins[fTp][ri][ckeyITD[(mTag,z)][fTp][ri]['c']] = \
                        ( (summFitBins['L'][iFit[fTp][ri]['c']]   [ri][ckeyITD[(mTag,z)][fTp][ri]['c']]    / summFitBins['L'][iFit[fTp]['Re']['z0']]['Re'][ckeyITD[(mTag,z)][fTp]['Re']['z0']]) *
                          (summFitBins['L'][iFit[fTp]['Re']['p0z0']]['Re'][ckeyITD[(mTag,z)][fTp]['Re']['p0z0']] / summFitBins['L'][iFit[fTp]['Re']['p0']]['Re'][ckeyITD[(mTag,z)][fTp]['Re']['p0']]))
                                
                            
                        ckeyI = ckeyITD[(mTag,z)][fTp][ri]['c']
                        rITD_ave[fTp][ri][ckeyI] = jackknife_ave(rITD_bins[fTp][ri][ckeyI],np.shape(rITD_bins[fTp][ri][ckeyI])[0],tL=1)
                    #--------------------------------------------------------------

                    
                    # 2-state ITD
                    fTp = '2-state'
                    
                    if(calc2st[iFit[fTp]['Re']['z0']  ]['Re'][ckeyITD[(mTag,z)][fTp]['Re']['z0']  ] and
                       calc2st[iFit[fTp]['Re']['p0']  ]['Re'][ckeyITD[(mTag,z)][fTp]['Re']['p0']  ] and
                       calc2st[iFit[fTp]['Re']['p0z0']]['Re'][ckeyITD[(mTag,z)][fTp]['Re']['p0z0']]):
                        
                        for ri in ReIm:
                            c2st_b = calc2st[iFit[fTp][ri]['c']][ri][ckeyITD[(mTag,z)][fTp][ri]['c']]
                            if(c2st_b):
                                
                                # Find common bins in matrix elements involved
                                commBinsList = commonBins(binList2st[iFit[fTp][ri  ]['c']   ][ri  ][ckeyITD[(mTag,z)][fTp][ri  ]['c']   ],
                                                          binList2st[iFit[fTp]['Re']['z0']  ]['Re'][ckeyITD[(mTag,z)][fTp]['Re']['z0']  ],
                                                          binList2st[iFit[fTp]['Re']['p0']  ]['Re'][ckeyITD[(mTag,z)][fTp]['Re']['p0']  ],
                                                          binList2st[iFit[fTp]['Re']['p0z0']]['Re'][ckeyITD[(mTag,z)][fTp]['Re']['p0z0']])

                                nITD_2st = len(commBinsList)                            
                                if nITD_2st == 0:
                                    print('No common bins for (%s,%s,%d). Rejecting this point'%(ri,mTag,z))
                                    rITD_ave[fTp][ri][ckeyITD[(mTag,z)][fTp][ri]['c']] = (None,None)
                                else:
                                    rITD_bins[fTp][ri][ckeyITD[(mTag,z)][fTp][ri]['c']] = np.zeros(nITD_2st)
                                    
                                    idx2st = {}
                                    for ib,b in enumerate(commBinsList):
                                        # Find indices of common bins for each position
                                        idx2st['c'] = binList2st[iFit[fTp][ri]['c']][ri][ckeyITD[(mTag,z)][fTp][ri]['c']].index(b)
                                        for pzV in pzPosListOffs:
                                            idx2st[pzV] = binList2st[iFit[fTp]['Re'][pzV]]['Re'][ckeyITD[(mTag,z)][fTp]['Re'][pzV]].index(b)

                                        # Determine ITDs
                                        rITD_bins[fTp][ri][ckeyITD[(mTag,z)][fTp][ri]['c']][ib] = \
                                        ( (M2stOpt_bins[iFit[fTp][ri]['c']][ri][ckeyITD[(mTag,z)][fTp][ri]['c']][ib]    / M2stOpt_bins[iFit[fTp]['Re']['z0']]['Re'][ckeyITD[(mTag,z)][fTp]['Re']['z0']][ib]) *
                                          (M2stOpt_bins[iFit[fTp]['Re']['p0z0']]['Re'][ckeyITD[(mTag,z)][fTp]['Re']['p0z0']][ib] / M2stOpt_bins[iFit[fTp]['Re']['p0']]['Re'][ckeyITD[(mTag,z)][fTp]['Re']['p0']][ib]))
                                    # common bins
                                    
                                    ckeyI = ckeyITD[(mTag,z)][fTp][ri]['c']
                                    
                                    if nITD_2st == 1:
                                        rITD_ave[fTp][ri][ckeyI] = (rITD_bins[fTp][ri][ckeyITD[(mTag,z)][fTp][ri]['c']],0.0)
                                    else:
                                        rITD_ave[fTp][ri][ckeyI] = jackknife_ave(rITD_bins[fTp][ri][ckeyI],np.shape(rITD_bins[fTp][ri][ckeyI])[0],tL=1)
                                # if commBinsList
                            # if calc2st
                            else:
                                if not(ri == 'Im' and mVec[2]*z==0):
                                    print('Will not calculate (%s,%s,%d). Rejecting this point'%(ri,mTag,z))
                                rITD_ave[fTp][ri][ckeyITD[(mTag,z)][fTp][ri]['c']] = (None,None)
                        # Real-Imag
                    else:
                        for ri in ReIm:
                            print('Will not calculate (%s,%s,%d). Rejecting this point'%(ri,mTag,z))
                            rITD_ave[fTp][ri][ckeyITD[(mTag,z)][fTp][ri]['c']] = (None,None)

                            
                print('Done: ',phMTag, ', Momentum %s'%(mTag))

print('\nDone\n')

### Dump ITDs in HDF5 files

In [ ]:

h5_file = h5py.File('h5_data/ITDs_plat_summ.hdf5', 'w')

for opT in SrcSnkOpTypes:
    for iphT in phaseTypes:            
        for phM in Phases_MomTags_ave[iphT]:
            phMTag = phasedTag(iphT,phM)
            phTpl = (phMTag,opT)
            phT = None

            momTags = momTags_Dict_ave[(phMTag,opT)]

            for mTag in momTags:
                mVec = momVector_Dict[mTag]
                zMom = mVec[2]
                mFTag = momFileTag(mVec)
                pmTpl = phMomTpl(phMTag,zMom)

                srcOpList = srcOpList3pt_Dict[opT][pmTpl] # These are the same in two- and
                snkOpList = srcOpList3pt_Dict[opT][pmTpl] # three-point functions

                isrc = 0
                isnk = 0

                # Only upolarized PDF for now
                irowSrc = 1
                irowSnk = 1
                gMat = 'gt'

                for z in dispLen_ave:
                    for ri in ReIm:

                        for fTp in ['Plat','L-summ']:
                            grp = '%s/Pz_%d/z_%d/%s'%(fTp,mVec[2],z,ri)

                            dset_str_bins = grp + '/jack'
                            dset_str_ave  = grp + '/ave'

                            dset_bins = h5_file.create_dataset(dset_str_bins, data=rITD_bins[fTp][ri][ckeyITD[(mTag,z)][fTp][ri]['c']])#, dtype='f')                        

                            dset_ave  = h5_file.create_dataset(dset_str_ave, data=rITD_ave[fTp][ri][ckeyITD[(mTag,z)][fTp][ri]['c']])#, dtype='f')                        

                                

h5_file.close()
                        
                        
print('\nDone\n')                        
                        

In [ ]:


figITDnuF, axITDnuF = plt.subplots(figsize=[10,6.25])

opTRS = 'single'
isrcRS = 0
isnkRS = 0
irowSrcRS = 1
irowSnkRS = 1

gMatRS = 'gt'

ri = 'Re'

plcITD = {'Plat':'Plateau', 'L-summ':'Summation', '2-state':'2-state'}
flITD = {'Plat':'plat', 'L-summ':'summ', '2-state':'2-state'}

for ift,fTp in enumerate(['Plat','L-summ','2-state']):
    
    figITDz3 , axITDz3  = plt.subplots(figsize=[10,6.25])
    figITDnuP, axITDnuP = plt.subplots(figsize=[10,6.25])
    figITDnuZ, axITDnuZ = plt.subplots(figsize=[10,6.25])
    
    ip = 0
    for iphT in phaseTypes:            
        for phM in Phases_MomTags_ave[iphT]:
            phMTag = phasedTag(iphT,phM)
            phTpl = (phMTag,opT)
            phT = None

            ophTpl = (phMTag,opTRS)
            momTags = momTags_Dict_ave[ophTpl]

            xRng = np.array(dispLen_ave)

            for mTag in momTags:
                mVec = momVector_Dict[mTag]

                rITD_plot = (np.zeros(len(dispLen_ave)),np.zeros(len(dispLen_ave)))
                izc = 0
                for idL,dL in enumerate(dispLen_ave):
                    ckey = ckeyITD[(mTag,dL)][fTp][ri]['c']

                    nu = mVec[2]*dL*uM

                    for i in range(2):
                        rITD_plot[i][idL] = rITD_ave[fTp][ri][ckey][i]

                    if idL==0:
                        nLblP = r'$p_z = %d$'%(mVec[2])
                    else:
                        nLblP = ''

                    if mTag == 'pz.0':
                        nLblZ = r'$z_3 = %d$'%(dL)
                    else:
                        nLblZ = ''

                    # Color based on Pz
                    axITDnuP.errorbar(nu, rITD_ave[fTp][ri][ckey][0], yerr = rITD_ave[fTp][ri][ckey][1],
                         color = cDct[ip], marker = mDct[ip], ls = 'none', label = nLblP,
                         ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full')

                    # Color based on z3
                    axITDnuZ.errorbar(nu, rITD_ave[fTp][ri][ckey][0], yerr = rITD_ave[fTp][ri][ckey][1],
                         color = cDct[izc], marker = mDct[izc], ls = 'none', label = nLblZ,
                         ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full')
                    
                    # Fit type comparison
                    axITDnuF.errorbar(nu+osDct[ift], rITD_ave[fTp][ri][ckey][0], yerr = rITD_ave[fTp][ri][ckey][1],
                         color = cDct[ift], marker = mDct[ift], ls = 'none', label = plcITD[fTp] if (izc==0 and ip==0) else None,
                         ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full')

                    izc += 1
                #-------------------------------

                axITDz3.errorbar(xRng+osDctF[ip]/2,  rITD_plot[0], yerr = rITD_plot[1],
                             color = cDct[ip], marker = mDct[ip], ls = 'none', label = r'$p_z = %d$'%(mVec[2]),
                             ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full')

                ip+=1

                
    axITDnuP.set_xlim(-0.5,11)
    axITDnuZ.set_xlim(-0.5,11)
    axITDz3.xaxis.set_ticks(np.arange(-1, 9, 1))
    axITDnuP.xaxis.set_ticks(np.arange(0, 10.5, 1))
    axITDnuZ.xaxis.set_ticks(np.arange(0, 10.5, 1))
    
    

    axITDz3.set_ylim(-0.4,1.2)
    axITDnuP.set_ylim(-0.4,1.2)
    axITDnuZ.set_ylim(-0.4,1.2)


    axITDz3.set_xlabel(r'$z_3/a$',fontsize=18)
    axITDz3.set_ylabel(r'%s[$\mathcal{M}_{\rm %s}(p_z,z_3^2)$]'%(ri,flITD[fTp]),fontsize=18)
    axITDnuP.set_xlabel(r'$\nu$',fontsize=18)
    axITDnuP.set_ylabel(r'%s[$\mathcal{M}_{\rm %s}(\nu,z_3^2)$]'%(ri,flITD[fTp]),fontsize=18)
    axITDnuZ.set_xlabel(r'$\nu$',fontsize=18)
    axITDnuZ.set_ylabel(r'%s[$\mathcal{M}_{\rm %s}(\nu,z_3^2)$]'%(ri,flITD[fTp]),fontsize=18)


    axITDz3.legend(loc='best', ncol=2, numpoints=1, fontsize=18,
              labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
              markerscale=0.8)

    axITDnuP.legend(loc='best', ncol=2, numpoints=1, fontsize=18,
              labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
              markerscale=0.8)

    axITDnuZ.legend(loc='best', ncol=2, numpoints=1, fontsize=18,
              labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
              markerscale=0.8)



    figITDz3.savefig('plots/rITD_z3_%s_%s.pdf'%(fTp,ri),bbox_inches='tight')
    figITDnuP.savefig('plots/rITD_nu_%s_%s_Pcol.pdf'%(fTp,ri),bbox_inches='tight')
    figITDnuZ.savefig('plots/rITD_nu_%s_%s_Zcol.pdf'%(fTp,ri),bbox_inches='tight')
# End fit type

axITDnuF.set_xlim(-0.5,11)
axITDnuF.xaxis.set_ticks(np.arange(0, 10.5, 1))
axITDnuF.set_ylim(-0.4,1.2)
axITDnuF.set_xlabel(r'$\nu$',fontsize=18)
axITDnuF.set_ylabel(r'%s[$\mathcal{M}(\nu,z_3^2)$]'%(ri),fontsize=18)

axITDnuF.legend(loc='best', ncol=2, numpoints=1, fontsize=18,
          labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
          markerscale=0.8)

figITDnuF.savefig('plots/rITD_nu_comp_%s.pdf'%(ri),bbox_inches='tight')


### Plot ratio $R(t_s,t_{\rm ins})$

In [ ]:
figR = [None, None]
axR  = [None, None]

figR[0], axR[0] = plt.subplots(figsize=[10,6.25])
figR[1], axR[1] = plt.subplots(figsize=[10,6.25])

Fct0 = {'Re': 0.95, 'Im': 0.85}
Fct1 = {'Re': 1.05, 'Im': 1.15}


opTR = 'single'
phTR = 'unphased'
ophTpl = (phTR,opTR)
isrcR = 0
isnkR = 0
irowSrcR = 1
irowSnkR = 1

#ens = 'cl21_32_64_b6p3_m0p2350_m0p2050'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-main'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-1000'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-1200'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-all'

gMatR = 'gt'

mTagR = 'pz.0'

dL = 2

# xRini = -8
# xRfin = 9
# xRoff = 0.3

# xLo = 2+0.3*isf
# xHi = 14-0.3*isf
# xSt = (xHi-xLo)/10
# sumLX = np.arange(xLo,14+0.01,xSt)

# fitBandRng = np.arange(-8,10)

for iri,ri in enumerate(ReIm):
    i = 0
    tsRng = {}
    for tsink in TsinkList[ens][(phTR,opTR)]:

        tsRng[tsink] = np.arange(tsink) - tsink/2

        adTplM = (phTR,opTR,mTagR,tsink)
        if analyzeDataSetM[ens][adTplM]:
            ckey = (opTR,phTR,mTagR,tsink,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

            axR[iri].errorbar(tsRng[tsink][1:]+osDctF[i]/2,  RA_ave[ens][ri][ckey][0][1:],
                         yerr = RA_ave[ens][ri][ckey][1][1:],
                         color = cDct[i], marker = mDct[i], ls = 'none',
                         label=r'$t_s=%d$'%(tsink),
                         ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle='full')
            i+=1
    #-------------------------------------------------------------------------------
 

#     # Plot plateau fit        
#     tsinkPlat = 12
#     iFit = 3

#     ckeyFit = (opTR,phTR,mTagR,tsinkPlat,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

#     ti = fitInfo_RP[ophTpl][tsinkPlat]['nf=%d'%(iFit)]['tstart']
#     tf = fitInfo_RP[ophTpl][tsinkPlat]['nf=%d'%(iFit)]['tstop']
#     tsnkIdx = TsinkList[(phTR,opTR)].index(tsinkPlat)

#     platVal = Mplat_ave[ri][ckeyFit][iFit]

#     platX = (tsRng[tsinkPlat][ti:tf+1] + osDctF[tsnkIdx]/2)
#     platY = np.array([platVal[0] for i in range(len(platX))])
#     platYerr = np.array([platVal[1] for i in range(len(platX))])

#     print('Plateau value %s = '%(ri), platVal)

#     axR[iri].plot(platX, platY, ls='-', color = cDct[tsnkIdx],linewidth=2.0)
#     axR[iri].fill_between(platX, platY-platYerr, platY+platYerr, facecolor=cDct[tsnkIdx], alpha=0.25,label=r'Plat. $t_s=%d$'%(tsinkPlat))  

#     if ri == 'Im':
#         if(np.abs(platVal[0]-platVal[1])<0.01 ):
#             axR[iri].axhline(0,ls='--',color='k',lw=1)

#     axR[iri].set_xlabel(r'$[t_{\rm ins} -t_s/2]/a$',fontsize=18)
#     axR[iri].set_ylabel(r'%s[$R(p_z;z_3;t_{\rm ins},t_s)$]'%(ri),fontsize=18)
#     #----------------------------------------------------------------------------------------------

    
#     # Plot 1-exp summation method band
#     ckeyF = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

#     E1sumVal = E1sum_ave['M'][ri][ckeyF]
#     print('E1-summ. value %s = '%(ri), E1sumVal)

#     iFc = 0
#     xLo = xRini+xRoff*iFc
#     xHi = xRfin-xRoff*iFc
#     xSt = (xHi-xLo)/10
    
#     E1sumX = np.arange(xLo,xRfin+0.01,xSt)
#     E1sumY = np.array([E1sumVal[0] for i in range(len(E1sumX))])
#     E1sumYerr = np.array([E1sumVal[1] for i in range(len(E1sumX))])

#     E1sumLbl = 'Summ. 1-exp'

#     axR[iri].plot(E1sumX, E1sumY,ls='-',color='slategray',linewidth=1.0)
#     axR[iri].fill_between(E1sumX , E1sumY-E1sumYerr, E1sumY+E1sumYerr, facecolor='slategray', alpha=0.25, label=E1sumLbl)     
#     #------------------------------------------------

#     # Plot linear fit band
#     fs = 8
#     sfs = 'l%d'%(fs)
#     ckeyF = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

#     LsumVal = Lsum_ave['M_%s'%(sfs)][ri][ckeyF]
#     print('L-summ. value %s %s = '%(sfs, ri), LsumVal)

#     iFc += 1
#     xLo = xRini+xRoff*iFc
#     xHi = xRfin-xRoff*iFc
#     xSt = (xHi-xLo)/10

#     LsumX = np.arange(xLo,xRfin+0.01,xSt)
#     LsumY = np.array([LsumVal[0] for i in range(len(LsumX))])
#     LsumYerr = np.array([LsumVal[1] for i in range(len(LsumX))])

#     LsumLbl = r'Summ. linear $t_s^{\rm low} = %d$'%(fs)

#     axR[iri].plot(LsumX, LsumY,ls='-',color='tomato',linewidth=1.0)
#     axR[iri].fill_between(LsumX , LsumY-LsumYerr, LsumY+LsumYerr, facecolor='tomato', alpha=0.15, label=LsumLbl)     
#     #------------------------------------------------

    
            
#     # Plot 2-state band
#     ckeyF = (opTR,phTR,mTagR,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

#     tsT = 8
#     tsT_s = 'ts%d'%(tsT)
    
#     if M2st_ave[tsT_s][ri][ckeyF][0] != None:
#         twoStVal = M2st_ave[tsT_s][ri][ckeyF]
#         print('2-state value %s = '%(ri), twoStVal)

#         iFc += 1
#         xLo = xRini+xRoff*iFc
#         xHi = xRfin-xRoff*iFc
#         xSt = (xHi-xLo)/10
        
#         twoStX = np.arange(xLo,xRfin+0.01,xSt)
#         twoStY = np.array([twoStVal[0] for i in range(len(twoStX))])
#         twoStYerr = np.array([twoStVal[1] for i in range(len(twoStX))])

#         twoStLbl = r'2-state, $t_s^{\rm low} = %d$'%(tsT)

#         axR[iri].plot(twoStX, twoStY, ls='-', color='darkorchid',linewidth=1.0)
#         axR[iri].fill_between(twoStX , twoStY-twoStYerr, twoStY+twoStYerr, facecolor='darkorchid', alpha=0.15, label=twoStLbl)
#     else:
#         print('Skipping 2-state fit for ', ckeyF)
#     #------------------------------------------------

    
#     axR[iri].legend(loc='best', ncol=3, numpoints=1, fontsize=18,
#            labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
#            markerscale=0.8)

#     axR[iri].xaxis.set_ticks(np.arange(-8, 9, 2))
#     axR[iri].set_xlim(-8,9)
    
#-------------------------------------------------------------------------------

#     yLim0 = Fct0[ri]*(E1sumVal[0]-E1sumVal[1])
#     yLim1 = Fct1[ri]*(E1sumVal[0]+E1sumVal[1])
#     axR[iri].set_ylim(yLim0,yLim1)




In [ ]:
figR = [None, None]
axR  = [None, None]

figR[0], axR[0] = plt.subplots(figsize=[10,6.25])
figR[1], axR[1] = plt.subplots(figsize=[10,6.25])


opTR = 'single'
phTR = 'unphased'
ophTpl = (phTR,opTR)
isrcR = 0
isnkR = 0
irowSrcR = 1
irowSnkR = 1

#ens = 'cl21_32_64_b6p3_m0p2350_m0p2050'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-main'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-1000'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-1200'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-all'
ens = None

ens_main = 'cl21_48_96_b6p3_m0p2416_m0p2050'
ensM = constructFullEns(ens_main,'all')

gMatR = 'gt'

tsink = 8

Pz_z_comb = [('pz.0',0),('pz.0',7),('pz.+1',3),('pz.+2',5),
             ('pz.+1',1),('pz.+1',8),('pz.+2',2),('pz.+2',6)]

mTagR,dL = Pz_z_comb[2]

mVec = momVector_Dict[ensM][mTagR]

sc=0
for ens_str in ensStreams[ens_main]:
    ens = constructFullEns(ens_main,ens_str)    

    if ens_str == 'all':
        fstl = 'full'
    else:
        fstl = 'none'

    for iri,ri in enumerate(ReIm):
#        i = 0
#        for tsink in TsinkList[ens][(phTR,opTR)]:

        tsRng[tsink] = np.arange(tsink) - tsink/2

        adTplM = (phTR,opTR,mTagR,tsink)
        if analyzeDataSetM[ens][adTplM]:
            ckey = (opTR,phTR,mTagR,tsink,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

            axR[iri].errorbar(tsRng[tsink][1:]+osDctF[sc]/1.2,  RA_ave[ens][ri][ckey][0][1:],
                         yerr = RA_ave[ens][ri][ckey][1][1:],
                         color = cDct[sc], marker = mDct[sc], ls = 'none',
                         label=r'%s'%(ensTag[ens]),
                         ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle=fstl)
    #            i+=1
    #-------------------------------------------------------------------------------

        axR[iri].set_xlabel(r'$[t_{\rm ins} - t_s/2]/a$',fontsize=18)
        axR[iri].xaxis.set_ticks(np.arange(-4, 4.1, 1))
        axR[iri].set_xlim(-4.2,4.2)


        axR[iri].legend(loc='best', ncol=2, numpoints=1, fontsize=18,
                        labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
                        markerscale=0.8)

        axR[iri].set_ylabel(r'%s[$R(p_z=%d;z_3=%d,t_{\rm sep} = %d)$]'%(ri,mVec[2],dL,tsink),fontsize=18)
    # End-loop Re-Im

    sc+=1    
#-------------------------------------------------------------------------------


for iri,ri in enumerate(ReIm):
    figR[iri].savefig('plots/%s-all/matElem.%s.tsep_%d.%s.z_%d.pdf'%(ens_main,ri,tsink,mTagR,dL),bbox_inches='tight')




In [ ]:
figC = [None, None]
axC  = [None, None]

figC[0], axC[0] = plt.subplots(figsize=[10,6.25])
figC[1], axC[1] = plt.subplots(figsize=[10,6.25])


opTR = 'single'
phTR = 'unphased'
ophTpl = (phTR,opTR)
isrcR = 0
isnkR = 0
irowSrcR = 1
irowSnkR = 1

#ens = 'cl21_32_64_b6p3_m0p2350_m0p2050'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-main'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-1000'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-1200'
#ens = 'cl21_48_96_b6p3_m0p2416_m0p2050-all'
ens = None

ens_main = 'cl21_48_96_b6p3_m0p2416_m0p2050'
ensM = constructFullEns(ens_main,'all')

gMatR = 'gt'

tsink = 8

Pz_z_comb = [('pz.0',0),('pz.0',7),('pz.0',3),('pz.+1',3),('pz.+1',4),('pz.+2',5),
             ('pz.+1',1),('pz.+1',7),('pz.+1',8),('pz.+2',2),('pz.+2',3),('pz.+2',6)]

mTagR,dL = Pz_z_comb[10]

mVec = momVector_Dict[ensM][mTagR]

sc=0
for ens in ['cl21_32_64_b6p3_m0p2350_m0p2050', 'cl21_48_96_b6p3_m0p2416_m0p2050-all']:

    if ens == 'cl21_32_64_b6p3_m0p2350_m0p2050':
        fstl = 'none'
    else:
        fstl = 'full'


    for iri,ri in enumerate(ReIm):
        i = 0
        for tsink in [6,8]:

            tsRng[tsink] = np.arange(tsink) - tsink/2

            adTplM = (phTR,opTR,mTagR,tsink)
            if analyzeDataSetM[ens][adTplM]:
                ckey = (opTR,phTR,mTagR,tsink,isrcR,isnkR,gMatR,dL,irowSrcR,irowSnkR)

                axC[iri].errorbar(tsRng[tsink][1:]+osDctF[i]/1.2,  RA_ave[ens][ri][ckey][0][1:],
                             yerr = RA_ave[ens][ri][ckey][1][1:],
                             color = cDct[i], marker = mDct[sc], ls = 'none',
                             label=r'$t_{\rm sep} = %d$, %s'%(tsink,ensTag[ens]),
                             ms = 8.0, elinewidth=elw, capsize=ecs,fillstyle=fstl)
                i+=1
        #--------------

        axC[iri].set_xlabel(r'$[t_{\rm ins} - t_s/2]/a$',fontsize=18)
        axC[iri].xaxis.set_ticks(np.arange(-4, 4.1, 1))
        axC[iri].set_xlim(-4.2,4.2)


        axC[iri].legend(loc='best', ncol=2, numpoints=1, fontsize=18,
                        labelspacing=0.2,borderpad=0.2,columnspacing=0.2,
                        markerscale=0.8)

        axC[iri].set_ylabel(r'%s[$R(p_z=%d;z_3=%d)$]'%(ri,mVec[2],dL),fontsize=18)
    # End-loop Re-Im

    sc+=1    
#-------------------------------------------------------------------------------


for iri,ri in enumerate(ReIm):
    figC[iri].savefig('plots/%s-all/matElem.EnsComp.%s.tsep_%d.%s.z_%d.pdf'%(ens_main,ri,tsink,mTagR,dL),bbox_inches='tight')



